In [1]:
#!/usr/bin/env python
# coding: utf-8

#Created by Leonardo G. Dettori

# ======================================================================================================================#
# ======================================================================================================================#
# ======================================================================================================================#

#imports useful libraries
import wx
import os
import shutil
from pokemon_functions import*

#Defines the Main Interface (Frame) of the Pokemon Team Builder Program, which holds a menu bar and the Main Panel
class Pokemon_Team_Builder_Frame(wx.Frame):
    
    def __init__(self):
        wx.Frame.__init__(self, parent=None, title='Pokemon Team Builder')
        
        #Calls Pokemon_Team_Builder_Panel
        self.panel = Pokemon_Team_Builder_Panel(self)
        #Creates a menu
        self.create_menu()
        #Exhibits the Frame to the user
        self.Show()
        #Maximizes the Frame
        self.Maximize(True)

        
        #Creates/resets dex_list to keep track of all pokemon available in the selected dex folder
        Pokemon_Team_Builder_Frame.dex_list = []
    
    #Defines the menu
    def create_menu(self):
        #Creates a menu bar
        menu_bar = wx.MenuBar()
        
        #Creates the databse menu
        database_menu = wx.Menu()
        #Adds Open Databse Folder item to the database menu
        open_database_folder_menu_item = database_menu.Append(wx.ID_ANY, 'Open Database Folder', 'Open a folder with .poke and .team files')
        #Adds database menu to the menubar
        menu_bar.Append(database_menu, '&Database')        
        #Binds open_database_folder_menu_item to on_open_folder function
        self.Bind(event=wx.EVT_MENU, handler=self.on_open_folder, source=open_database_folder_menu_item,)
        
        #Creates the dex menu
        dex_menu = wx.Menu()
        #Adds Open Dex Folder item to the dex menu
        open_dex_folder_menu_item = dex_menu.Append(wx.ID_ANY, 'Open PokeDex Folder', 'Open a folder with the .pokedexswsh files from Serebii.net')
        #Adds dex menu to the menubar
        menu_bar.Append(dex_menu, '&Pokedex')        
        #Binds open_dex_folder_menu_item to on_open_folder_dex function
        self.Bind(event=wx.EVT_MENU, handler=self.on_open_folder_dex, source=open_dex_folder_menu_item,)  
        
        #Sets up menu bar
        self.SetMenuBar(menu_bar)
        
    #Defines on_open_folder function to get information from the provided database
    def on_open_folder(self, event):
        #Desfines title of the dialog box that will be used to select database's directory
        title = "Choose a directory:"
        #Calls a default Directory Dialog Box (dlg)
        #(this type of dialog box allows the user to only open directories)
        dlg = wx.DirDialog(self, title, style=wx.DD_DEFAULT_STYLE)
        #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown
        #If the user presses the dialog box’s OK button, the program can get the user’s path choice via the dialog’s .GetPath() function
        if dlg.ShowModal() == wx.ID_OK:
            
            #Gets path to the Databse directory
            Pokemon_Team_Builder_Frame.data_path = str(dlg.GetPath())+'/'
            
            #Passes the selected Database Path to the functions which will handle the pokemon list and the team list on the program's main panel, respectively
            self.panel.update_pokemon_listing()
            self.panel.update_team_listing()
            
        #Closes default Direcotry Dialog Box (dlg) using Destroy()  
        #(this is the appropriate function to close dialog boxes without harming the program)
        dlg.Destroy()
        
    #Defines on_open_folder_dex function to get information from the provided Pokedex Folder downloaded from Serebii.net
    def on_open_folder_dex(self, event):
        #Desfines title of the dialog box that will be used to select dex's directory
        title = "Choose a directory:"
        #Calls a default Directory Dialog Box (dlg)
        #(this type of dialog box allows the user to only open directories)
        dlg = wx.DirDialog(self, title, style=wx.DD_DEFAULT_STYLE)
        #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown
        #If the user presses the dialog box’s OK button, the program can get the user’s path choice via the dialog’s .GetPath() function
        if dlg.ShowModal() == wx.ID_OK:
            
            #Gets path to the Dex directory
            Pokemon_Team_Builder_Frame.dex_path = str(dlg.GetPath())+'/'
            
            #Uses the selected Dex Path to assemble a list of all pokemon available in the Dex directory
            #Creates/resets dex_list to keep track of all pokemon available in the selected dex folder
            Pokemon_Team_Builder_Frame.dex_list = []
            #Compiles list of files within the provided databse directory
            self.files = next(os.walk(Pokemon_Team_Builder_Frame.dex_path))[2]
            #print(self.files)
            #Compiles the list of available pokemon dex entries
            for f in self.files:
                if f.endswith('.pokedexswsh'):  #pokemon files
                    Pokemon_Team_Builder_Frame.dex_list.append(f)#.replace('.pokedexswsh',''))
            #print(Pokemon_Team_Builder_Frame.dex_list)

            
        #Closes default Direcotry Dialog Box (dlg) using Destroy()  
        #(this is the appropriate function to close dialog boxes without harming the program)
        dlg.Destroy()
        

        
#Defines Pokemon Team Builder program's main panel, which holds the two lists (Pokemon, Team) and their buttons (Add, Edit, Delete)
class Pokemon_Team_Builder_Panel(wx.Panel):
    
    def __init__(self, parent):
        super().__init__(parent)
        
        #Defines Main Sizer for the panel
        main_sizer = wx.BoxSizer(wx.HORIZONTAL)
        #Defines Pokemon Sizer, which will go inside Main Sizer
        pokemon_sizer = wx.BoxSizer(wx.VERTICAL)
        #Defines Team Sizer, which will go inside Main Sizer
        team_sizer = wx.BoxSizer(wx.VERTICAL)
        
        #Adds Pokemon Sizer to Main Sizer
        main_sizer.Add(pokemon_sizer, 2, wx.ALL | wx.EXPAND, 10)
        #Adds Team Sizer to Main Sizer
        main_sizer.Add(team_sizer, 1, wx.ALL | wx.EXPAND, 10)
        
        #Defines a dictionary to handle all pokemon and all team entries in the database
        self.Database = {}
        self.Database['All_Pokemon'] = {}
        self.Database['All_Teams'] = {}
        
        # ============================= #  Defines Static Texts to use as List Titles
        
        #Defines font style for the statict text
        font = wx.Font(18, wx.DECORATIVE, wx.ITALIC, wx.NORMAL)
        
        #Creates Pokemon List Title as a Static Text and sets font style
        self.pokemon_title = wx.StaticText(self, label="Pokemon Entries: ")
        self.pokemon_title.SetFont(font)
        #Creates Team List Title as a Static Text and sets font style
        self.team_title = wx.StaticText(self, label="Team Entries: ")
        self.team_title.SetFont(font)
        #Adds Pokemon List Title to Pokemon Sizer
        pokemon_sizer.Add(self.pokemon_title, 0, wx.ALL | wx.CENTER, 5)
        #Adds Team List Title to Team Sizer
        team_sizer.Add(self.team_title, 0, wx.ALL | wx.CENTER, 5)
        
        # ============================= #  Defines Lists and adds them to respective sizers

        #Defines Pokemon List
        self.pokemon_list_ctrl = wx.ListCtrl(self, size=(-1, 100), style=wx.LC_REPORT | wx.BORDER_SUNKEN)
        #Defines Column Headers in a python list
        self.pokemon_headers = ['Name','Type 1','Type 2','Ability','Item','Move 1','Move 2','Move 3','Move 4']
        #Defines Column Widths in a python list
        self.pokemon_headers_widths = [150,70,70,100,100,100,100,100,100]
        #Creates/resets helpful counter
        pokemon_header_counter = 0
        #Uses a loop to create one column per header
        for header in self.pokemon_headers:
            #Inserts Column in Pokemon List
            self.pokemon_list_ctrl.InsertColumn(pokemon_header_counter, header, width=self.pokemon_headers_widths[pokemon_header_counter])
            #moves counter to netx iteration
            pokemon_header_counter = pokemon_header_counter + 1            
        #For header customization
        # Attributes (textcolor, backgroundcolor, font)
        font_bold = wx.Font(wx.FontInfo(10).Bold())
        head_txt_colr = wx.Colour(10,35,106)
        head_bac_colr = wx.Colour('white') #background not working
        # Currently it is implemented only for wxMSW and does nothing in the other ports       
        self.pokemon_list_ctrl.SetHeaderAttr(wx.ItemAttr(head_txt_colr, head_bac_colr, font_bold))
        
        #Defines Team List
        self.team_list_ctrl = wx.ListCtrl(self, size=(-1, 100), style=wx.LC_REPORT | wx.BORDER_SUNKEN)
        #Defines Column Headers in a python list
        self.team_headers = ['Team Name','Pokemon 1','Pokemon 2','Pokemon 3','Pokemon 4','Pokemon 5','Pokemon 6']
        #Defines Column Widths in a python list
        self.team_headers_widths = [150,150,150,150,150,150,150]
        #Creates/resets helpful counter
        team_header_counter = 0
        #Uses a loop to create one column per header
        for header in self.team_headers:
            #Inserts Column in Team List
            self.team_list_ctrl.InsertColumn(team_header_counter, header, width=self.team_headers_widths[team_header_counter])
            #moves counter to netx iteration
            team_header_counter = team_header_counter + 1  
        #For header customization
        # Attributes (textcolor, backgroundcolor, font)
        font_bold = wx.Font(wx.FontInfo(10).Bold())
        head_txt_colr = wx.Colour(10,35,106)
        head_bac_colr = wx.Colour('white') #background not working
        # Currently it is implemented only for wxMSW and does nothing in the other ports       
        self.team_list_ctrl.SetHeaderAttr(wx.ItemAttr(head_txt_colr, head_bac_colr, font_bold))
        
        #Binds double-click events to each list
        #For pokemon list
        self.Bind(wx.EVT_LIST_ITEM_ACTIVATED, self.OnClick_pokemon_list, self.pokemon_list_ctrl)
        #For team list
        self.Bind(wx.EVT_LIST_ITEM_ACTIVATED, self.OnClick_team_list, self.team_list_ctrl)
        
        #Binds column left-click events to each list
        #For pokemon list
        self.Bind(wx.EVT_LIST_COL_CLICK, self.OnColumnLeftClick_pokemon_list, self.pokemon_list_ctrl)
        #For team list
        self.Bind(wx.EVT_LIST_COL_CLICK, self.OnColumnLeftClick_team_list, self.team_list_ctrl)
        
        #Adds Pokemon List to the Pokemon Sizer
        pokemon_sizer.Add(self.pokemon_list_ctrl, 1, wx.ALL | wx.EXPAND, 5)
        #Adds Team List to the Team Sizer
        team_sizer.Add(self.team_list_ctrl, 1, wx.ALL | wx.EXPAND, 5)
        
        # ============================= #  Defines Buttons and Binds them to Functions
        
        #Defines Pokemon Buttons
        
        #Defines Pokemon Add Button
        pokemon_add_button = wx.Button(self, label='Add')
        #Binds Pokemon Add Button to on_add_pokemon function
        pokemon_add_button.Bind(wx.EVT_BUTTON, self.on_add_pokemon)
        
        #Defines Pokemon Edit Button
        pokemon_edit_button = wx.Button(self, label='Edit')
        #Binds Pokemon Edit Button to on_edit_pokemon function
        pokemon_edit_button.Bind(wx.EVT_BUTTON, self.on_edit_pokemon)
        
        #Defines Pokemon Delete Button
        pokemon_delete_button = wx.Button(self, label='Delete')
        #Binds Pokemon Delete Button to on_delete_pokemon function
        pokemon_delete_button.Bind(wx.EVT_BUTTON, self.on_delete_pokemon)
        
        #Defines Team Buttons
        
        #Defines Team Add Button
        team_add_button = wx.Button(self, label='Add')
        #Binds Team Add Button to on_add_team function
        team_add_button.Bind(wx.EVT_BUTTON, self.on_add_team)
        
        #Defines Team Edit Button
        team_edit_button = wx.Button(self, label='Edit')
        #Binds Team Edit Button to on_edit_team function
        team_edit_button.Bind(wx.EVT_BUTTON, self.on_edit_team)
        
        #Defines Team Delete Button
        team_delete_button = wx.Button(self, label='Delete')
        #Binds Team Delete Button to on_delete_team function
        team_delete_button.Bind(wx.EVT_BUTTON, self.on_delete_team)
        
        # ============================= # Defines Button Sizers, adds Buttons to Sizers and Sizers to outer Sizers
        
        #Defines Pokemon Button Sizer to go inside Pokemon Sizer
        pokemon_button_sizer = wx.BoxSizer(wx.HORIZONTAL)        
        #Adds Pokemon Add Button to Pokemon Button Sizer
        pokemon_button_sizer.Add(pokemon_add_button, 0, wx.ALL | wx.CENTER, 5)
        #Adds Pokemon Edit Button to Pokemon Button Sizer
        pokemon_button_sizer.Add(pokemon_edit_button, 0, wx.ALL | wx.CENTER, 5) 
        #Adds Pokemon Delete Button to Pokemon Button Sizer
        pokemon_button_sizer.Add(pokemon_delete_button, 0, wx.ALL | wx.CENTER, 5)
        #Adds Pokemon Button Sizer to Pokemon Sizer
        pokemon_sizer.Add(pokemon_button_sizer, 0, wx.ALL | wx.CENTER, 5)
        
        #Defines Team Button Sizer to go inside Team Sizer
        team_button_sizer = wx.BoxSizer(wx.HORIZONTAL)
        #Adds Team Add Button to Team Button Sizer
        team_button_sizer.Add(team_add_button, 0, wx.ALL | wx.CENTER, 5)
        #Adds Team Edit Button to Team Button Sizer
        team_button_sizer.Add(team_edit_button, 0, wx.ALL | wx.CENTER, 5) 
        #Adds Team Delete Button to Team Button Sizer
        team_button_sizer.Add(team_delete_button, 0, wx.ALL | wx.CENTER, 5)
        #Adds Team Button Sizer to Team Sizer
        team_sizer.Add(team_button_sizer, 0, wx.ALL | wx.CENTER, 5)
        
        # ============================= #  Sets up Main Sizer
               
        #Sets up Main Sizer for the panel
        self.SetSizer(main_sizer)
        
    # =========================================================== # Defines functions for the Add, Edit and Delete Buttons (Pokemon and Team)
    
    #Defines function for Add pokemon entry button. It links the button to a dialog box for adding pokemon entries
    def on_add_pokemon(self, event):
        #Sets up Add Pokemon Dialog Box
        dlg = AddPokemonDialog()
        #Displays Add Pokemon Dialog Box
        dlg.ShowModal()
        #Updates the Pokemon List in the panel with the newly added pokemon entry
        self.update_pokemon_listing()
        #Closes Add Pokemon Dialog Box
        dlg.Destroy()
    
    #Defines function for Edit pokemon entry button. It links the button to a dialog box for editing pokemon entries
    def on_edit_pokemon(self, event):
        #Gets information of the selected pokemon from Pokemon List in the main panel
        selection = self.pokemon_list_ctrl.GetFocusedItem()
        #Button only functions if something was selected
        if selection >= 0:
            #print(self.pokemon_list_ctrl.GetItemText(selection,col=0))
            #Sets up Edit Pokemon Dialog Box taking selected pokemon name as argument
            dlg = EditPokemonDialog(self.pokemon_list_ctrl.GetItemText(selection,col=0))
            #Displays Edit Pokemon Dialog Box
            dlg.ShowModal()
            #Updates the Pokemon List in the panel with the freshly edited pokemon entry
            self.update_pokemon_listing()
            #Closes Edit Pokemon Dialog Box
            dlg.Destroy()
        
        #IF the user hasn't made a selection in the Pokemon List
        else:
            #Displays a default error message dialog telling the user to make a selection in the Pokemon List
            #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
            dlg = wx.MessageDialog(self, caption='Error!', message = 'No item was selected! Please select an item from Pokemon Entries.', style= wx.OK | wx.ICON_ERROR)
            dlg.ShowModal()
            #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
            #Closes the message dialog box    
            dlg.Destroy()
            
    #Defines function for Delete pokemon entry button. It links the button to a default dialog box for deleting pokemon entries
    def on_delete_pokemon(self, event):
        #Gets information of the selected pokemon from Pokemon List in the main panel
        selection = self.pokemon_list_ctrl.GetFocusedItem()
        #Button only functions if something was selected
        if selection >= 0:
            #Gets pokemon name from selected item at Pokemon List
            poke_name = self.pokemon_list_ctrl.GetItemText(selection,col=0)
            #Calls a default dialog box to ask user if they would like to delete pokemon entry
            #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
            dlg_attention = wx.MessageDialog(self, caption='Attention!', message = 'Would you like to permanently delete the following pokemon entry?\n\n'+str(poke_name), style= wx.OK | wx.CANCEL | wx.ICON_WARNING)
            #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown
            #If the user presses the attention dialog box’s OK button, the program can get the user’s path choice via the dialog’s .GetPath() function
            if dlg_attention.ShowModal() == wx.ID_OK:
                #Deletes pokemon entry from the specified database
                delete_pokemon(Pokemon_Team_Builder_Frame.data_path,poke_name)
                #Displays a default message to say that the Entry was Successfully Deleted from the Database
                #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
                dlg = wx.MessageDialog(self, caption='Success!', message = 'Pokemon entry was successfully deleted from database!', style= wx.OK | wx.ICON_INFORMATION)
                dlg.ShowModal()
                #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
                #Closes the message dialog box    
                dlg.Destroy()
                #Closes the attention message dialog box    
                dlg_attention.Destroy()
                #Updates the Pokemon List in the panel 
                self.update_pokemon_listing()
            
            #When the user doesn't want to delete the selected pokemon entry, the message box closes
            else:
                #Closes the attention message dialog box    
                dlg_attention.Destroy()
                
        #IF the user hasn't made a selection in the Pokemon List
        else:
            #Displays a default error message dialog telling the user to make a selection in the Pokemon List
            #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
            dlg = wx.MessageDialog(self, caption='Error!', message = 'No item was selected! Please select an item from Pokemon Entries.', style= wx.OK | wx.ICON_ERROR)
            dlg.ShowModal()
            #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
            #Closes the message dialog box    
            dlg.Destroy()
            
    #Defines function for Add team entry button. It links the button to a dialog box for adding team entries
    def on_add_team(self, event):
        #Sets up Add Team Dialog Box
        dlg = AddTeamDialog()
        #Displays Add Team Dialog Box
        dlg.ShowModal()
        #Updates the Team List in the panel with the newly added pokemon entry
        self.update_team_listing()
        #Closes Add Team Dialog Box
        dlg.Destroy()
        
    #Defines function for Edit team entry button. It links the button to a dialog box for editing team entries
    def on_edit_team(self, event):
        #Gets information of the selected team from Team List in the main panel
        selection = self.team_list_ctrl.GetFocusedItem()
        #Button only functions if something was selected
        if selection >= 0:
            #print(self.team_list_ctrl.GetItemText(selection,col=0))
            #Sets up Edit Team Dialog Box taking selected team name as argument
            dlg = EditTeamDialog(self.team_list_ctrl.GetItemText(selection,col=0))
            #Displays Edit Team Dialog Box
            dlg.ShowModal()
            #Updates the Team List in the panel with the newly edited team entry
            self.update_team_listing()
            #Closes Edit Pokemon Dialog Box
            dlg.Destroy()
        
        #IF the user hasn't made a selection in the Team List
        else:
            #Displays a default error message dialog telling the user to make a selection in the Team List
            #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
            dlg = wx.MessageDialog(self, caption='Error!', message = 'No item was selected! Please select an item from Team Entries.', style= wx.OK | wx.ICON_ERROR)
            dlg.ShowModal()
            #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
            #Closes the message dialog box    
            dlg.Destroy()        
        
    #Defines function for Delete team entry button. It links the button to a default dialog box for deleting team entries
    def on_delete_team(self, event):
        #Gets information of the selected team from Team List in the main panel
        selection = self.team_list_ctrl.GetFocusedItem()
        #Button only functions if something was selected
        if selection >= 0:
            #Gets team name from selected item at Team List
            team_name = self.team_list_ctrl.GetItemText(selection,col=0)
            #Calls a default dialog box to ask user if they would like to delete team entry
            #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
            dlg_attention = wx.MessageDialog(self, caption='Attention!', message = 'Would you like to permanently delete the following team entry?\n\n'+str(team_name), style= wx.OK | wx.CANCEL | wx.ICON_WARNING)
            #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown
            #If the user presses the attention dialog box’s OK button, the program can get the user’s path choice via the dialog’s .GetPath() function
            if dlg_attention.ShowModal() == wx.ID_OK:
                #Deletes pokemon entry from the specified database
                delete_team(Pokemon_Team_Builder_Frame.data_path,team_name)
                #Displays a default message to say that the Entry was Successfully Deleted from the Database
                #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
                dlg = wx.MessageDialog(self, caption='Success!', message = 'Team entry was successfully deleted from database!', style= wx.OK | wx.ICON_INFORMATION)
                dlg.ShowModal()
                #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
                #Closes the message dialog box    
                dlg.Destroy()
                #Closes the attention message dialog box    
                dlg_attention.Destroy()
                #Updates the Team List in the panel 
                self.update_team_listing()
            
            #When the user doesn't want to delete the selected team entry, the message box closes
            else:
                #Closes the attention message dialog box    
                dlg_attention.Destroy()
                
        #IF the user hasn't made a selection in the Team List
        else:
            #Displays a default error message dialog telling the user to make a selection in the Team List
            #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
            dlg = wx.MessageDialog(self, caption='Error!', message = 'No item was selected! Please select an item from Team Entries.', style= wx.OK | wx.ICON_ERROR)
            dlg.ShowModal()
            #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
            #Closes the message dialog box    
            dlg.Destroy()
            
    # =========================================================== # Defines functions for double-click on each list
    
    #Defines function for double-click on pokemon list. It links the double-click to a dialog box for viewing pokemon entries
    def OnClick_pokemon_list(self, event):
        #Gets information of the selected pokemon from Pokemon List in the main panel
        selection = self.pokemon_list_ctrl.GetFocusedItem()
        #Button only functions if something was selected
        if selection >= 0:
            #Loads pokemon information
            self.clicked_pokemon = read_pokemon(Pokemon_Team_Builder_Frame.data_path, self.pokemon_list_ctrl.GetItemText(selection,col=0))            
            #print(self.pokemon_list_ctrl.GetItemText(selection,col=0))
            #Sets up view Pokemon Dialog Box using pre-loaded dictionary as argument
            dlg = ViewPokemonDialog_from_dict(self.clicked_pokemon, preview='no')
            #Displays ViewPokemonDialog_from_dict Box
            dlg.Show()
        
        #IF the user hasn't made a selection in the Pokemon List
        else:
            #Displays a default error message dialog telling the user to make a selection in the Pokemon List
            #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
            dlg = wx.MessageDialog(self, caption='Error!', message = 'No item was selected! Please select an item from Pokemon Entries.', style= wx.OK | wx.ICON_ERROR)
            dlg.ShowModal()
            #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
            #Closes the message dialog box    
            dlg.Destroy()
    
    #Defines function for double-click on team list. It links the double-click to a dialog box for viewing team entries
    def OnClick_team_list(self, event):
        #Gets information of the selected team from Team List in the main panel
        selection = self.team_list_ctrl.GetFocusedItem()
        #Button only functions if something was selected
        if selection >= 0:
            #Loads team information
            self.clicked_team = read_team(Pokemon_Team_Builder_Frame.data_path, self.team_list_ctrl.GetItemText(selection,col=0))            
            #print(self.team_list_ctrl.GetItemText(selection,col=0))
            #Sets up PreViewTeamDialog using pre-loaded dictionary as argument
            dlg = PreViewTeamDialog(self.clicked_team, preview='no')
            #Displays PreViewTeamDialog Box
            dlg.Show()
        
        #IF the user hasn't made a selection in the Team List
        else:
            #Displays a default error message dialog telling the user to make a selection in the Team List
            #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
            dlg = wx.MessageDialog(self, caption='Error!', message = 'No item was selected! Please select an item from Team Entries.', style= wx.OK | wx.ICON_ERROR)
            dlg.ShowModal()
            #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
            #Closes the message dialog box    
            dlg.Destroy()
            
    # =========================================================== # Defines functions for column left-click on each list
    
    #Defines function for column left-click on pokemon list. It sorts the column
    def OnColumnLeftClick_pokemon_list(self, event):
        selection = event.GetColumn()
        #print(self.pokemon_headers[selection])
        #Sorts and updates Pokemon List Ctrl
        self.update_pokemon_listing(sorting=self.pokemon_headers[selection])
        
    #Defines function for column left-click on team list. It sorts the column
    def OnColumnLeftClick_team_list(self, event):
        selection = event.GetColumn()
        print(self.team_headers[selection])
        #Sorts and updates Team List Ctrl
        #The function is no implemented yet
        #self.update_team_listing(sorting=self.team_headers[selection])
                        

     # =========================================================== #       
            
    #Defines function that updates the Team List with database information
    def update_team_listing(self, sorting='None'):
        
        #Resets Team List in the Panel
        self.team_list_ctrl.ClearAll()
        
        #Resets useful variables
        self.files = []  #Keeps track of all files in the database
        Pokemon_Team_Builder_Frame.teams_available = []  #Keeps track of all teams in the database

        #Resets dictionary to handle all pokemon entries and all team entries in the database
        self.Database['All_Teams'].clear()
        self.Database['All_Teams'] = {}

        #Compiles list of files within the provided databse directory
        self.files = next(os.walk(Pokemon_Team_Builder_Frame.data_path))[2]
        #print(self.files)

        #Compiles a python list of available teams within the database
        for f in self.files:
            if f.endswith('.team'):  #team files
                Pokemon_Team_Builder_Frame.teams_available.append(f.replace('.team',''))

        #Uses lists to load database entries into subdictionary for team entries
        #Loads all team entries from database into Database Python dictionary, entry by entry
        database_counter = 0 # resets helpful counter
        for i in Pokemon_Team_Builder_Frame.teams_available:
            #Moves handy database counter to the next iteration
            database_counter = database_counter + 1                
            #Reads current team entry
            this_Team = read_team(Pokemon_Team_Builder_Frame.data_path,i)
            #Adds current team entry to the Database Python dictionary under a newly created sub-dictionary
            self.Database['All_Teams']['Team_'+str(database_counter)] = this_Team       

        #Reinserts Columns with their respective Headers previously defined
        #Creates/resets helpful counter
        team_header_counter = 0
        #Uses a loop to create one column per header
        for header in self.team_headers:
            #Inserts Column in Team List
            self.team_list_ctrl.InsertColumn(team_header_counter, header, width=self.team_headers_widths[team_header_counter])
            #moves counter to netx iteration
            team_header_counter = team_header_counter + 1 
            
        #Creates/Resets dictionary to handle sorted team entries
        sorted_database = {}
        
        #When sorting was used
        if sorting != 'None':
            sorted_database = team_list_ctrl_dictionary_sorting(self.Database['All_Teams'],sorting)
        #When sorting was not used
        elif sorting == 'None':
            sorted_database = dict(self.Database['All_Teams'])      
        
        #Updates the Team_list on the Panel entry by entry
        index = 0   #resets helpful counter
        for team in sorted_database:
            self.team_list_ctrl.InsertItem(index, sorted_database[team]['Team_Name'][0])
            self.team_list_ctrl.SetItem(index, 1, sorted_database[team]['Members']['Pokemon_1']['Name'][0])
            self.team_list_ctrl.SetItem(index, 2, sorted_database[team]['Members']['Pokemon_2']['Name'][0])
            self.team_list_ctrl.SetItem(index, 3, sorted_database[team]['Members']['Pokemon_3']['Name'][0])
            self.team_list_ctrl.SetItem(index, 4, sorted_database[team]['Members']['Pokemon_4']['Name'][0])
            self.team_list_ctrl.SetItem(index, 5, sorted_database[team]['Members']['Pokemon_5']['Name'][0])
            self.team_list_ctrl.SetItem(index, 6, sorted_database[team]['Members']['Pokemon_6']['Name'][0])
            #Alternating background color per line
            if index % 2:
                self.pokemon_list_ctrl.SetItemBackgroundColour(index, "white")
            else:
                self.pokemon_list_ctrl.SetItemBackgroundColour(index, (240,255,255))
            #moves index to next iteration
            index = index + 1
            
            
    #Defines function that updates the Pokemon List with database information
    def update_pokemon_listing(self, sorting='None'):
        
        #Resets Team List in the Panel
        self.pokemon_list_ctrl.ClearAll()
        
        #Resets useful variables
        self.files = []  #Keeps track of all files in the database
        Pokemon_Team_Builder_Frame.pokemon_available = []  #Keeps track of all pokemon in the database

        #Resets dictionary to handle all pokemon entries and all team entries in the database
        self.Database['All_Pokemon'].clear()
        self.Database['All_Pokemon'] = {}

        #Compiles list of files within the provided database directory
        self.files = next(os.walk(Pokemon_Team_Builder_Frame.data_path))[2]
        #print(self.files)

        #Compiles a python list of available pokemon and available teams within the database
        for f in self.files:
            if f.endswith('.poke'):  #pokemon files
                Pokemon_Team_Builder_Frame.pokemon_available.append(f.replace('.poke',''))

        #Uses lists to load database entries into subdictionary for pokemon entries
        #Loads all pokemon entries from database into Database Python dictionary, entry by entry
        database_counter = 0 # defines helpful counter
        for i in Pokemon_Team_Builder_Frame.pokemon_available:
            #Moves handy database counter to the next iteration
            database_counter = database_counter + 1                
            #Reads current pokemon entry
            this_Pokemon = read_pokemon(Pokemon_Team_Builder_Frame.data_path,i)
            #Adds current pokemon entry to the Database Python dictionary under a newly created sub-dictionary
            self.Database['All_Pokemon']['Pokemon_'+str(database_counter)] = this_Pokemon

        #Reinserts Columns with their respective Headers previously defined
        #Creates/resets helpful counter
        pokemon_header_counter = 0
        #Uses a loop to create one column per header
        for header in self.pokemon_headers:
            #Inserts Column in Pokemon List
            self.pokemon_list_ctrl.InsertColumn(pokemon_header_counter, header, width=self.pokemon_headers_widths[pokemon_header_counter])
            #moves counter to netx iteration
            pokemon_header_counter = pokemon_header_counter + 1  
        
        #Creates/Resets dictionary to handle sorted pokemon entries
        sorted_database = {}
        
        #When sorting was used
        if sorting != 'None':
            sorted_database = pokemon_list_ctrl_dictionary_sorting(self.Database['All_Pokemon'],sorting)
        #When sorting was not used
        elif sorting == 'None':
            sorted_database = dict(self.Database['All_Pokemon'])        
        
        #Updates the Pokemon_list on the Panel entry by entry
        index = 0   #resets helpful counter
        for pokemon in sorted_database:
            self.pokemon_list_ctrl.InsertItem(index, sorted_database[pokemon]['Name'][0])
            self.pokemon_list_ctrl.SetItem(index, 1, sorted_database[pokemon]['Types'][0])
            self.pokemon_list_ctrl.SetItem(index, 2, sorted_database[pokemon]['Types'][1])
            self.pokemon_list_ctrl.SetItem(index, 3, sorted_database[pokemon]['Ability']['Ability_Name'][0])
            self.pokemon_list_ctrl.SetItem(index, 4, sorted_database[pokemon]['Item'][0])
            self.pokemon_list_ctrl.SetItem(index, 5, sorted_database[pokemon]['Moves']['Move_Name'][0])
            #self.pokemon_list_ctrl.SetItem(index, 6, sorted_database[pokemon]['Moves']['Move_Type'][0])
            #self.pokemon_list_ctrl.SetItem(index, 7, sorted_database[pokemon]['Moves']['Move_Mode'][0])
            self.pokemon_list_ctrl.SetItem(index, 6, sorted_database[pokemon]['Moves']['Move_Name'][1])
            #self.pokemon_list_ctrl.SetItem(index, 9, sorted_database[pokemon]['Moves']['Move_Type'][1])
            #self.pokemon_list_ctrl.SetItem(index, 10, sorted_database[pokemon]['Moves']['Move_Mode'][1])
            self.pokemon_list_ctrl.SetItem(index, 7, sorted_database[pokemon]['Moves']['Move_Name'][2])
            #self.pokemon_list_ctrl.SetItem(index, 12, sorted_database[pokemon]['Moves']['Move_Type'][2])
            #self.pokemon_list_ctrl.SetItem(index, 13, sorted_database[pokemon]['Moves']['Move_Mode'][2])
            self.pokemon_list_ctrl.SetItem(index, 8, sorted_database[pokemon]['Moves']['Move_Name'][3])
            #self.pokemon_list_ctrl.SetItem(index, 15, sorted_database[pokemon]['Moves']['Move_Type'][3])
            #self.pokemon_list_ctrl.SetItem(index, 16, sorted_database[pokemon]['Moves']['Move_Mode'][3])
            #Alternating background color per line
            if index % 2:
                self.pokemon_list_ctrl.SetItemBackgroundColour(index, "white")
            else:
                self.pokemon_list_ctrl.SetItemBackgroundColour(index, (240,255,255))
            #moves index to next iteration
            index = index + 1


# ======================================================================================= # Add Pokemon Entry
        
#Defines Add Pokemon Entry as a Dialog Box
class AddPokemonDialog(wx.Dialog):
            
    def __init__(self):
        #When Dex was not loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) <1:
            super().__init__(parent=None, title="Add Pokemon Entry", style = wx.DEFAULT_DIALOG_STYLE | wx.RESIZE_BORDER, size = (1140,935))        
        #When Dex was loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) >1:
            super().__init__(parent=None, title="Add Pokemon Entry - Dex Assisted", style = wx.DEFAULT_DIALOG_STYLE | wx.RESIZE_BORDER, size = (1140,990)) 
                   
        #Defines Helpful Variables
        self.all_types = ['Normal', 'Fighting', 'Flying', 'Poison', 'Ground', 'Rock', 'Bug', 'Ghost', 'Steel', 'Fire', 'Water', 'Grass', 'Electric', 'Psychic', 'Ice', 'Dragon', 'Dark', 'Fairy']
        self.all_types_2 = ['', 'Normal', 'Fighting', 'Flying', 'Poison', 'Ground', 'Rock', 'Bug', 'Ghost', 'Steel', 'Fire', 'Water', 'Grass', 'Electric', 'Psychic', 'Ice', 'Dragon', 'Dark', 'Fairy']
        self.all_modes = ['Physical', 'Special', 'Status']
        
        #Defines Main Sizer
        self.main_sizer = wx.BoxSizer(wx.VERTICAL)
        
        #Defines font style for the statict text that labels the Moves and Pokemon
        font_moves = wx.Font(12, wx.DECORATIVE, wx.BOLD, wx.NORMAL)  
        
        #Sets up a Static Text to be used as empty space and adds it to the main sizer
        empty_space_label = wx.StaticText(self, label="")
        empty_space_label.SetFont(font_moves)
        self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
        
        #Sets up a Static Text for Pokemon and adds it to the main sizer
        pokemon_label = wx.StaticText(self, label="New Pokemon Entry")
        pokemon_label.SetFont(font_moves)
        self.main_sizer.Add(pokemon_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

        #When Dex was not loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) <1:
            
            #Defines Top_left_right Sizer
            self.Top_left_right_sizer = wx.BoxSizer(wx.HORIZONTAL)

            #Defines Top_left Sizer
            self.Top_left_sizer = wx.BoxSizer(wx.VERTICAL)

            #Defines Top_right Sizer
            self.Top_right_sizer = wx.BoxSizer(wx.VERTICAL)
            
            #Sets up a Static Text to be used as empty space and adds it to Top_left Sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.Top_left_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            
            #Sets up a Static Text to be used as empty space and adds it to Top_left Sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.Top_left_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            
            #Sets up Box for Name, starting empty
            self.Name_box = wx.TextCtrl(self, value='')
            self.add_widgets('Name:', self.Name_box, self.Top_left_sizer)
            
            #Creates bitmap image box, starting empty
            self.png = wx.StaticBitmap(self, -1)
            #Adds bitmap Image box to Top_right Sizer
            self.Top_right_sizer.Add(self.png, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            
            #Sets up Boxes for Species and Form
            #Sets up Box for Species, starting empty
            self.Species_box = wx.TextCtrl(self, value='')
            self.add_widgets('Species:', self.Species_box, self.Top_left_sizer)        
            #Sets up Box for Form, starting empty
            self.Form_box = wx.TextCtrl(self, value='')
            self.add_widgets('Form:', self.Form_box, self.Top_left_sizer)
            #self.add_widgets_4_boxes_horizontal('Species:', self.Species_box, 'Form:', self.Form_box, self.main_sizer)

            #Sets up Combo Boxes for Types
            #Sets up Combo Box for Type_1, starting empty
            self.Type_1_box = wx.ComboBox(self, size=(95, -1), choices=self.all_types, style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
            #self.add_widgets('Type 1:', self.Type_1_box)        
            #Sets up Combo Box for Type_2, starting empty
            self.Type_2_box = wx.ComboBox(self, size=(95, -1), choices=self.all_types_2, style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
            #self.add_widgets('Type 2:', self.Type_2_box)        
            self.add_widgets_4_boxes_horizontal('Type 1:', self.Type_1_box, 'Type 2:', self.Type_2_box, self.Top_left_sizer)

            #Sets up Box for Item, starting empty
            self.Item_box = wx.TextCtrl(self, value='')
            self.add_widgets('Item:', self.Item_box, self.Top_left_sizer)
            
            #Sets up Box for Image, starting empty
            self.Image_box = wx.TextCtrl(self, value='_', style=wx.TE_READONLY)
            self.add_widgets('Image:', self.Image_box, self.Top_left_sizer)

            #Sets up a Static Text to be used as empty space and adds it to Top_left Sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.Top_left_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            
            #Sets up a Static Text to be used as empty space and adds it to Top_left Sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.Top_left_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Adds Top_left Sizer to Top_left_right Sizer
            self.Top_left_right_sizer.Add(self.Top_left_sizer, 1, wx.EXPAND)
            
            #Sets up a Static Text to be used as empty space and adds it to Top_left_right Sizer
            empty_space_label = wx.StaticText(self, label="    ")
            empty_space_label.SetFont(font_moves)
            self.Top_left_right_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_VERTICAL)
            
            #Adds Top_right Sizer to Top_left_right Sizer
            self.Top_left_right_sizer.Add(self.Top_right_sizer, 1, wx.EXPAND)

            #Adds self.Top_left_right to Main Sizer
            self.main_sizer.Add(self.Top_left_right_sizer, 0, wx.EXPAND)

            #Sets up Boxes for Ability_Name and Ability_Info
            #Sets up a Static Text for Ability and adds it to the main sizer
            Ability_label = wx.StaticText(self, label="Ability")
            Ability_label.SetFont(font_moves)
            self.main_sizer.Add(Ability_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            #Sets up Box for Ability_Name, starting empty
            self.Ability_Name_box = wx.TextCtrl(self, value='')
            #self.add_widgets('Ability Name:', self.Ability_Name_box)        
            #Sets up Box for Ability_Info, starting empty
            self.Ability_Info_box = wx.TextCtrl(self, value='')
            #self.add_widgets('Ability Info:', self.Ability_Info_box)
            self.add_widgets_4_boxes_horizontal('Name:', self.Ability_Name_box, 'Info:', self.Ability_Info_box, self.main_sizer)         

            # ============================ #  Moves 1 and 2

            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Defines Moves 1 and 2 Sizer
            self.moves_1_2_sizer = wx.BoxSizer(wx.HORIZONTAL)

            #Defines Move 1 Sizer
            self.move_1_sizer = wx.BoxSizer(wx.VERTICAL)

            #Defines Move 2 Sizer
            self.move_2_sizer = wx.BoxSizer(wx.VERTICAL)

            # ============== #  Move 1

            #Sets up a Static Text for Move 1 and adds it to the move 1 sizer
            Move_1_label = wx.StaticText(self, label="Move 1")
            Move_1_label.SetFont(font_moves)
            self.move_1_sizer.Add(Move_1_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Sets up Boxes for Name and Mode
            #Sets up Box for Move 1 Name, starting empty
            self.Move_1_Name_box = wx.TextCtrl(self, value='')
            #Sets up Radio Box for Move 1 Mode, starting empty
            self.Move_1_Mode_box = wx.RadioBox(self, choices=self.all_modes,  majorDimension=3, style=wx.RA_SPECIFY_COLS)
            self.add_widgets_4_boxes_horizontal('Name:', self.Move_1_Name_box, 'Mode:', self.Move_1_Mode_box, self.move_1_sizer)

            #Sets up Boxes for Type and Method        
            #Sets up Combo Box for Move 1 Type, starting empty
            self.Move_1_Type_box = wx.ComboBox(self, size=(95, -1), choices=self.all_types, style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
            #Sets up Static Text for Move 1 Method, starting empty
            self.Move_1_Method_box = wx.TextCtrl(self, value='')
            self.add_widgets_4_boxes_horizontal('Type:', self.Move_1_Type_box, 'Method:', self.Move_1_Method_box, self.move_1_sizer)

            #Sets up Boxes for Power and Accuracy
            #Sets up Static Text for Move 1 Power, starting empty
            self.Move_1_Power_box = wx.TextCtrl(self, value='')       
            #Sets up Static Text for Move 1 Accuracy, starting empty
            self.Move_1_Accuracy_box = wx.TextCtrl(self, value='')
            self.add_widgets_4_boxes_horizontal('Power:', self.Move_1_Power_box, 'Acc.:', self.Move_1_Accuracy_box, self.move_1_sizer)

            #Sets up Boxes for PP and Effect
            #Sets up Static Text for Move 1 PP, starting empty
            self.Move_1_PP_box = wx.TextCtrl(self, value='')      
            #Sets up Static Text for Move 1 Effect, starting empty
            self.Move_1_Effect_box = wx.TextCtrl(self, value='')
            self.add_widgets_4_boxes_horizontal('PP:', self.Move_1_PP_box, 'Effect:', self.Move_1_Effect_box, self.move_1_sizer)

            #Sets up Static Text for Move 1 Info, starting empty
            self.Move_1_Info_box = wx.TextCtrl(self, value='')
            self.add_widgets('Info:', self.Move_1_Info_box, self.move_1_sizer)

            # ============== #  Move 2

            #Sets up a Static Text for Move 2 and adds it to the move 2 sizer
            Move_2_label = wx.StaticText(self, label="Move 2")
            Move_2_label.SetFont(font_moves)
            self.move_2_sizer.Add(Move_2_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Sets up Boxes for Name and Mode
            #Sets up Box for Move 2 Name, starting empty
            self.Move_2_Name_box = wx.TextCtrl(self, value='')
            #Sets up Radio Box for Move 2 Mode, starting empty
            self.Move_2_Mode_box = wx.RadioBox(self, choices=self.all_modes,  majorDimension=3, style=wx.RA_SPECIFY_COLS)
            self.add_widgets_4_boxes_horizontal('Name:', self.Move_2_Name_box, 'Mode:', self.Move_2_Mode_box, self.move_2_sizer)

            #Sets up Boxes for Type and Method        
            #Sets up Combo Box for Move 2 Type, starting empty
            self.Move_2_Type_box = wx.ComboBox(self, size=(95, -1), choices=self.all_types, style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
            #Sets up Static Text for Move 2 Method, starting empty
            self.Move_2_Method_box = wx.TextCtrl(self, value='')
            self.add_widgets_4_boxes_horizontal('Type:', self.Move_2_Type_box, 'Method:', self.Move_2_Method_box, self.move_2_sizer)

            #Sets up Boxes for Power and Accuracy
            #Sets up Static Text for Move 2 Power, starting empty
            self.Move_2_Power_box = wx.TextCtrl(self, value='')       
            #Sets up Static Text for Move 2 Accuracy, starting empty
            self.Move_2_Accuracy_box = wx.TextCtrl(self, value='')
            self.add_widgets_4_boxes_horizontal('Power:', self.Move_2_Power_box, 'Acc.:', self.Move_2_Accuracy_box, self.move_2_sizer)

            #Sets up Boxes for PP and Effect
            #Sets up Static Text for Move 2 PP, starting empty
            self.Move_2_PP_box = wx.TextCtrl(self, value='')      
            #Sets up Static Text for Move 2 Effect, starting empty
            self.Move_2_Effect_box = wx.TextCtrl(self, value='')
            self.add_widgets_4_boxes_horizontal('PP:', self.Move_2_PP_box, 'Effect:', self.Move_2_Effect_box, self.move_2_sizer)

            #Sets up Static Text for Move 2 Info, starting empty
            self.Move_2_Info_box = wx.TextCtrl(self, value='')
            self.add_widgets('Info:', self.Move_2_Info_box, self.move_2_sizer)

            # ============== # 

            #Adds Move 1 Sizer to Moves 1 and 2 Sizer
            self.moves_1_2_sizer.Add(self.move_1_sizer, 1, wx.EXPAND)

            #Sets up a Static Text to be used as empty space and adds it to  Moves 1 and 2 Sizer
            empty_space_label = wx.StaticText(self, label="    ")
            empty_space_label.SetFont(font_moves)
            self.moves_1_2_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_VERTICAL)

            #Adds Move 2 Sizer to Moves 1 and 2 Sizer
            self.moves_1_2_sizer.Add(self.move_2_sizer, 1, wx.EXPAND)

            #Adds Moves 1 and 2 Sizer to Main Sizer
            self.main_sizer.Add(self.moves_1_2_sizer, 0, wx.EXPAND)

            # ============================ #  Moves 3 and 4

            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Defines Moves 3 and 4 Sizer
            self.moves_3_4_sizer = wx.BoxSizer(wx.HORIZONTAL)

            #Defines Move 3 Sizer
            self.move_3_sizer = wx.BoxSizer(wx.VERTICAL)

            #Defines Move 4 Sizer
            self.move_4_sizer = wx.BoxSizer(wx.VERTICAL)

            # ============== #  Move 3

            #Sets up a Static Text for Move 3 and adds it to the move 3 sizer
            Move_3_label = wx.StaticText(self, label="Move 3")
            Move_3_label.SetFont(font_moves)
            self.move_3_sizer.Add(Move_3_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Sets up Boxes for Name and Mode
            #Sets up Box for Move 3 Name, starting empty
            self.Move_3_Name_box = wx.TextCtrl(self, value='')
            #Sets up Radio Box for Move 3 Mode, starting empty
            self.Move_3_Mode_box = wx.RadioBox(self, choices=self.all_modes,  majorDimension=3, style=wx.RA_SPECIFY_COLS)
            self.add_widgets_4_boxes_horizontal('Name:', self.Move_3_Name_box, 'Mode:', self.Move_3_Mode_box, self.move_3_sizer)

            #Sets up Boxes for Type and Method        
            #Sets up Combo Box for Move 3 Type, starting empty
            self.Move_3_Type_box = wx.ComboBox(self, size=(95, -1), choices=self.all_types, style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
            #Sets up Static Text for Move 3 Method, starting empty
            self.Move_3_Method_box = wx.TextCtrl(self, value='')
            self.add_widgets_4_boxes_horizontal('Type:', self.Move_3_Type_box, 'Method:', self.Move_3_Method_box, self.move_3_sizer)

            #Sets up Boxes for Power and Accuracy
            #Sets up Static Text for Move 3 Power, starting empty
            self.Move_3_Power_box = wx.TextCtrl(self, value='')       
            #Sets up Static Text for Move 3 Accuracy, starting empty
            self.Move_3_Accuracy_box = wx.TextCtrl(self, value='')
            self.add_widgets_4_boxes_horizontal('Power:', self.Move_3_Power_box, 'Acc.:', self.Move_3_Accuracy_box, self.move_3_sizer)

            #Sets up Boxes for PP and Effect
            #Sets up Static Text for Move 3 PP, starting empty
            self.Move_3_PP_box = wx.TextCtrl(self, value='')      
            #Sets up Static Text for Move 3 Effect, starting empty
            self.Move_3_Effect_box = wx.TextCtrl(self, value='')
            self.add_widgets_4_boxes_horizontal('PP:', self.Move_3_PP_box, 'Effect:', self.Move_3_Effect_box, self.move_3_sizer)

            #Sets up Static Text for Move 3 Info, starting empty
            self.Move_3_Info_box = wx.TextCtrl(self, value='')
            self.add_widgets('Info:', self.Move_3_Info_box, self.move_3_sizer)

            # ============== #  Move 4

            #Sets up a Static Text for Move 4 and adds it to the move 4 sizer
            Move_4_label = wx.StaticText(self, label="Move 4")
            Move_4_label.SetFont(font_moves)
            self.move_4_sizer.Add(Move_4_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Sets up Boxes for Name and Mode
            #Sets up Box for Move 4 Name, starting empty
            self.Move_4_Name_box = wx.TextCtrl(self, value='')
            #Sets up Radio Box for Move 4 Mode, starting empty
            self.Move_4_Mode_box = wx.RadioBox(self, choices=self.all_modes,  majorDimension=3, style=wx.RA_SPECIFY_COLS)
            self.add_widgets_4_boxes_horizontal('Name:', self.Move_4_Name_box, 'Mode:', self.Move_4_Mode_box, self.move_4_sizer)

            #Sets up Boxes for Type and Method        
            #Sets up Combo Box for Move 4 Type, starting empty
            self.Move_4_Type_box = wx.ComboBox(self, size=(95, -1), choices=self.all_types, style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
            #Sets up Static Text for Move 4 Method, starting empty
            self.Move_4_Method_box = wx.TextCtrl(self, value='')
            self.add_widgets_4_boxes_horizontal('Type:', self.Move_4_Type_box, 'Method:', self.Move_4_Method_box, self.move_4_sizer)

            #Sets up Boxes for Power and Accuracy
            #Sets up Static Text for Move 4 Power, starting empty
            self.Move_4_Power_box = wx.TextCtrl(self, value='')       
            #Sets up Static Text for Move 4 Accuracy, starting empty
            self.Move_4_Accuracy_box = wx.TextCtrl(self, value='')
            self.add_widgets_4_boxes_horizontal('Power:', self.Move_4_Power_box, 'Acc.:', self.Move_4_Accuracy_box, self.move_4_sizer)

            #Sets up Boxes for PP and Effect
            #Sets up Static Text for Move 4 PP, starting empty
            self.Move_4_PP_box = wx.TextCtrl(self, value='')      
            #Sets up Static Text for Move 4 Effect, starting empty
            self.Move_4_Effect_box = wx.TextCtrl(self, value='')
            self.add_widgets_4_boxes_horizontal('PP:', self.Move_4_PP_box, 'Effect:', self.Move_4_Effect_box, self.move_4_sizer)

            #Sets up Static Text for Move 4 Info, starting empty
            self.Move_4_Info_box = wx.TextCtrl(self, value='')
            self.add_widgets('Info:', self.Move_4_Info_box, self.move_4_sizer)

            # ============== # 

            #Adds Move 3 Sizer to Moves 3 and 4 Sizer
            self.moves_3_4_sizer.Add(self.move_3_sizer, 1, wx.EXPAND)

            #Sets up a Static Text to be used as empty space and adds it to  Moves 3 and 4 Sizer
            empty_space_label = wx.StaticText(self, label="    ")
            empty_space_label.SetFont(font_moves)
            self.moves_3_4_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_VERTICAL)

            #Adds Move 4 Sizer to Moves 3 and 4 Sizer
            self.moves_3_4_sizer.Add(self.move_4_sizer, 1, wx.EXPAND)

            #Adds Moves 3 and 4 Sizer to Main Sizer
            self.main_sizer.Add(self.moves_3_4_sizer, 0, wx.EXPAND)

            # ============== #
            
        #When Dex was loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) >1:
            
            #Creates/resets useful dictionary to handle selected pokemon from Dex
            self.this_dex_pokemon = {}
            
            #Creates a complimentary list to Pokemon_Team_Builder_Frame.dex_list without pokemon number and .pokedexswsh in the pokemon name
            self.dex_list_comp = []
            for poke in Pokemon_Team_Builder_Frame.dex_list:
                #print(poke)
                self.dex_list_comp.append(poke.split('_')[1].replace('.pokedexswsh',''))
            
            #Defines Top_left_right Sizer
            self.Top_left_right_sizer = wx.BoxSizer(wx.HORIZONTAL)

            #Defines Top_left Sizer
            self.Top_left_sizer = wx.BoxSizer(wx.VERTICAL)

            #Defines Top_right Sizer
            self.Top_right_sizer = wx.BoxSizer(wx.VERTICAL)
            
            #Sets up a Static Text to be used as empty space and adds it to Top_left Sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.Top_left_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            
            #Sets up a Static Text to be used as empty space and adds it to Top_left Sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.Top_left_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            
            #Sets up Box for Name, starting empty
            self.Name_box = wx.TextCtrl(self, value='')
            self.add_widgets('Name:', self.Name_box, self.Top_left_sizer)
            #self.add_widgets('Name:', self.Name_box, self.main_sizer)
            
            #Creates bitmap image box, starting empty
            self.png = wx.StaticBitmap(self, -1)
            #Adds bitmap Image box to Top_right Sizer
            self.Top_right_sizer.Add(self.png, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Sets up Boxes for Species and Form  self.combo = wx.ComboBox(panel,choices = languages) 
            #Sets up Combo Box for Species, starting empty but using pokemon from the selected Dex
            self.Species_box = wx.ComboBox(self, size=(95, -1), choices=self.dex_list_comp, style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
            #Binds event to selecting a pokemon in the Species Combo Box OnSelectForm
            self.Species_box.Bind(wx.EVT_COMBOBOX, self.OnSelectSpecies)
            
            #self.add_widgets('Species:', self.Species_box)        
            #Sets up Box for Form, starting empty
            self.Form_box = wx.ComboBox(self, size=(95, -1), choices=[], style=wx.CB_DROPDOWN | wx.CB_READONLY)
            #Binds event to selecting a form in the Form Combo Box
            self.Form_box.Bind(wx.EVT_COMBOBOX, self.OnSelectForm)
            self.add_widgets('Species:', self.Species_box, self.Top_left_sizer)
            self.add_widgets('Form:', self.Form_box, self.Top_left_sizer)            
            #self.add_widgets_4_boxes_horizontal('Species:', self.Species_box, 'Form:', self.Form_box, self.main_sizer)

            #Sets up Combo Boxes for Types
            #Sets up Combo Box for Type_1, starting empty
            self.Type_1_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY)
            #self.add_widgets('Type 1:', self.Type_1_box)        
            #Sets up Combo Box for Type_2, starting empty
            self.Type_2_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY)
            #self.add_widgets('Type 2:', self.Type_2_box)        
            self.add_widgets_4_boxes_horizontal('Type 1:', self.Type_1_box, 'Type 2:', self.Type_2_box, self.Top_left_sizer)
            #self.add_widgets_4_boxes_horizontal('Type 1:', self.Type_1_box, 'Type 2:', self.Type_2_box, self.main_sizer)

            #Sets up Box for Item, starting empty
            self.Item_box = wx.TextCtrl(self, value='')
            self.add_widgets('Item:', self.Item_box, self.Top_left_sizer)
            #self.add_widgets('Item:', self.Item_box, self.main_sizer)
            
            #Sets up Combo Box for Image, starting empty
            self.Image_box = wx.ComboBox(self, size=(95, -1), choices=[], style=wx.CB_DROPDOWN | wx.CB_READONLY)
            self.add_widgets('Image:', self.Image_box, self.Top_left_sizer)
            #self.add_widgets('Image:', self.Image_box, self.main_sizer)
            #Binds event to selecting an image name in the Images Combo Box to OnSelectImage
            self.Image_box.Bind(wx.EVT_COMBOBOX, self.OnSelectImage)
            
            #Sets up a Static Text to be used as empty space and adds it to Top_left Sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.Top_left_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            
            #Sets up a Static Text to be used as empty space and adds it to Top_left Sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.Top_left_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Adds Top_left Sizer to Top_left_right Sizer
            self.Top_left_right_sizer.Add(self.Top_left_sizer, 1, wx.EXPAND)
            
            #Sets up a Static Text to be used as empty space and adds it to Top_left_right Sizer
            empty_space_label = wx.StaticText(self, label="    ")
            empty_space_label.SetFont(font_moves)
            self.Top_left_right_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_VERTICAL)
            
            #Adds Top_right Sizer to Top_left_right Sizer
            self.Top_left_right_sizer.Add(self.Top_right_sizer, 1, wx.EXPAND)

            #Adds self.Top_left_right to Main Sizer
            self.main_sizer.Add(self.Top_left_right_sizer, 0, wx.EXPAND)

            #Sets up Boxes for Ability_Name and Ability_Info
            #Sets up a Static Text for Ability and adds it to the main sizer
            Ability_label = wx.StaticText(self, label="Ability")
            Ability_label.SetFont(font_moves)
            self.main_sizer.Add(Ability_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            #Sets up Box for Ability_Name, starting empty
            self.Ability_Name_box = wx.ComboBox(self, size=(95, -1), choices=[], style=wx.CB_DROPDOWN | wx.CB_READONLY)
            #Binds event to selecting an ability name in the Form Combo Box
            self.Ability_Name_box.Bind(wx.EVT_COMBOBOX, self.OnSelectAbility)            
       
            #Sets up Box for Ability_Info, starting empty
            self.Ability_Info_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY | wx.TE_MULTILINE | wx.HSCROLL)
            #self.add_widgets('Ability Info:', self.Ability_Info_box)
            self.add_widgets_4_boxes_horizontal('Name:', self.Ability_Name_box, 'Info:', self.Ability_Info_box, self.main_sizer)         

            # ============================ #  Moves 1 and 2

            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Defines Moves 1 and 2 Sizer
            self.moves_1_2_sizer = wx.BoxSizer(wx.HORIZONTAL)

            #Defines Move 1 Sizer
            self.move_1_sizer = wx.BoxSizer(wx.VERTICAL)

            #Defines Move 2 Sizer
            self.move_2_sizer = wx.BoxSizer(wx.VERTICAL)

            # ============== #  Move 1

            #Sets up a Static Text for Move 1 and adds it to the move 1 sizer
            Move_1_label = wx.StaticText(self, label="Move 1")
            Move_1_label.SetFont(font_moves)
            self.move_1_sizer.Add(Move_1_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Sets up Boxes for Name and Mode
            #Sets up Box for Move 1 Name, starting empty
            self.Move_1_Name_box = wx.ComboBox(self, size=(95, -1), choices=[], style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
            #Binds event to selecting a move name in the Form Combo Box
            self.Move_1_Name_box.Bind(wx.EVT_COMBOBOX, self.OnSelectMove_1) 
            
            #Sets up Box for Move 1 Mode, starting empty
            self.Move_1_Mode_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Name:', self.Move_1_Name_box, 'Mode:', self.Move_1_Mode_box, self.move_1_sizer)

            #Sets up Boxes for Type and Method        
            #Sets up Combo Box for Move 1 Type, starting empty
            self.Move_1_Type_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY)
            #Sets up Static Text for Move 1 Method, starting empty
            self.Move_1_Method_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Type:', self.Move_1_Type_box, 'Method:', self.Move_1_Method_box, self.move_1_sizer)

            #Sets up Boxes for Power and Accuracy
            #Sets up Static Text for Move 1 Power, starting empty
            self.Move_1_Power_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY)       
            #Sets up Static Text for Move 1 Accuracy, starting empty
            self.Move_1_Accuracy_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Power:', self.Move_1_Power_box, 'Acc.:', self.Move_1_Accuracy_box, self.move_1_sizer)

            #Sets up Boxes for PP and Effect
            #Sets up Static Text for Move 1 PP, starting empty
            self.Move_1_PP_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY)      
            #Sets up Static Text for Move 1 Effect, starting empty
            self.Move_1_Effect_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('PP:', self.Move_1_PP_box, 'Effect:', self.Move_1_Effect_box, self.move_1_sizer)

            #Sets up Static Text for Move 1 Info, starting empty
            self.Move_1_Info_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY | wx.TE_MULTILINE | wx.HSCROLL)
            self.add_widgets('Info:', self.Move_1_Info_box, self.move_1_sizer)

            # ============== #  Move 2

            #Sets up a Static Text for Move 2 and adds it to the move 2 sizer
            Move_2_label = wx.StaticText(self, label="Move 2")
            Move_2_label.SetFont(font_moves)
            self.move_2_sizer.Add(Move_2_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Sets up Boxes for Name and Mode
            #Sets up Box for Move 2 Name, starting empty
            self.Move_2_Name_box = wx.ComboBox(self, size=(95, -1), choices=[], style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
            #Binds event to selecting a move name in the Form Combo Box
            self.Move_2_Name_box.Bind(wx.EVT_COMBOBOX, self.OnSelectMove_2) 
            
            #Sets up Box for Move 2 Mode, starting empty
            self.Move_2_Mode_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Name:', self.Move_2_Name_box, 'Mode:', self.Move_2_Mode_box, self.move_2_sizer)

            #Sets up Boxes for Type and Method        
            #Sets up Combo Box for Move 2 Type, starting empty
            self.Move_2_Type_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY)
            #Sets up Static Text for Move 2 Method, starting empty
            self.Move_2_Method_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Type:', self.Move_2_Type_box, 'Method:', self.Move_2_Method_box, self.move_2_sizer)

            #Sets up Boxes for Power and Accuracy
            #Sets up Static Text for Move 2 Power, starting empty
            self.Move_2_Power_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY)       
            #Sets up Static Text for Move 2 Accuracy, starting empty
            self.Move_2_Accuracy_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Power:', self.Move_2_Power_box, 'Acc.:', self.Move_2_Accuracy_box, self.move_2_sizer)

            #Sets up Boxes for PP and Effect
            #Sets up Static Text for Move 2 PP, starting empty
            self.Move_2_PP_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY)      
            #Sets up Static Text for Move 2 Effect, starting empty
            self.Move_2_Effect_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('PP:', self.Move_2_PP_box, 'Effect:', self.Move_2_Effect_box, self.move_2_sizer)

            #Sets up Static Text for Move 2 Info, starting empty
            self.Move_2_Info_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY | wx.TE_MULTILINE | wx.HSCROLL)
            self.add_widgets('Info:', self.Move_2_Info_box, self.move_2_sizer)

            # ============== # 

            #Adds Move 1 Sizer to Moves 1 and 2 Sizer
            self.moves_1_2_sizer.Add(self.move_1_sizer, 1, wx.EXPAND)

            #Sets up a Static Text to be used as empty space and adds it to  Moves 1 and 2 Sizer
            empty_space_label = wx.StaticText(self, label="    ")
            empty_space_label.SetFont(font_moves)
            self.moves_1_2_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_VERTICAL)

            #Adds Move 2 Sizer to Moves 1 and 2 Sizer
            self.moves_1_2_sizer.Add(self.move_2_sizer, 1, wx.EXPAND)

            #Adds Moves 1 and 2 Sizer to Main Sizer
            self.main_sizer.Add(self.moves_1_2_sizer, 0, wx.EXPAND)   
            
            # ============================ #  Moves 3 and 4

            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Defines Moves 3 and 4 Sizer
            self.moves_3_4_sizer = wx.BoxSizer(wx.HORIZONTAL)

            #Defines Move 3 Sizer
            self.move_3_sizer = wx.BoxSizer(wx.VERTICAL)

            #Defines Move 4 Sizer
            self.move_4_sizer = wx.BoxSizer(wx.VERTICAL)

            # ============== #  Move 3

            #Sets up a Static Text for Move 3 and adds it to the move 3 sizer
            Move_3_label = wx.StaticText(self, label="Move 3")
            Move_3_label.SetFont(font_moves)
            self.move_3_sizer.Add(Move_3_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Sets up Boxes for Name and Mode
            #Sets up Box for Move 3 Name, starting empty
            self.Move_3_Name_box = wx.ComboBox(self, size=(95, -1), choices=[], style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
            #Binds event to selecting a move name in the Form Combo Box
            self.Move_3_Name_box.Bind(wx.EVT_COMBOBOX, self.OnSelectMove_3) 
            
            #Sets up Box for Move 3 Mode, starting empty
            self.Move_3_Mode_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Name:', self.Move_3_Name_box, 'Mode:', self.Move_3_Mode_box, self.move_3_sizer)

            #Sets up Boxes for Type and Method        
            #Sets up Combo Box for Move 3 Type, starting empty
            self.Move_3_Type_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY) 
            #Sets up Static Text for Move 3 Method, starting empty
            self.Move_3_Method_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Type:', self.Move_3_Type_box, 'Method:', self.Move_3_Method_box, self.move_3_sizer)

            #Sets up Boxes for Power and Accuracy
            #Sets up Static Text for Move 3 Power, starting empty
            self.Move_3_Power_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY)       
            #Sets up Static Text for Move 3 Accuracy, starting empty
            self.Move_3_Accuracy_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Power:', self.Move_3_Power_box, 'Acc.:', self.Move_3_Accuracy_box, self.move_3_sizer)

            #Sets up Boxes for PP and Effect
            #Sets up Static Text for Move 3 PP, starting empty
            self.Move_3_PP_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY)      
            #Sets up Static Text for Move 3 Effect, starting empty
            self.Move_3_Effect_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('PP:', self.Move_3_PP_box, 'Effect:', self.Move_3_Effect_box, self.move_3_sizer)

            #Sets up Static Text for Move 3 Info, starting empty
            self.Move_3_Info_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY | wx.TE_MULTILINE | wx.HSCROLL)
            self.add_widgets('Info:', self.Move_3_Info_box, self.move_3_sizer)

            # ============== #  Move 4

            #Sets up a Static Text for Move 4 and adds it to the move 4 sizer
            Move_4_label = wx.StaticText(self, label="Move 4")
            Move_4_label.SetFont(font_moves)
            self.move_4_sizer.Add(Move_4_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Sets up Boxes for Name and Mode
            #Sets up Box for Move 4 Name, starting empty
            self.Move_4_Name_box = wx.ComboBox(self, size=(95, -1), choices=[], style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
            #Binds event to selecting a move name in the Form Combo Box
            self.Move_4_Name_box.Bind(wx.EVT_COMBOBOX, self.OnSelectMove_4) 
            
            #Sets up Box for Move 4 Mode, starting empty
            self.Move_4_Mode_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Name:', self.Move_4_Name_box, 'Mode:', self.Move_4_Mode_box, self.move_4_sizer)

            #Sets up Boxes for Type and Method        
            #Sets up Combo Box for Move 4 Type, starting empty
            self.Move_4_Type_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY) 
            #Sets up Static Text for Move 4 Method, starting empty
            self.Move_4_Method_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Type:', self.Move_4_Type_box, 'Method:', self.Move_4_Method_box, self.move_4_sizer)

            #Sets up Boxes for Power and Accuracy
            #Sets up Static Text for Move 4 Power, starting empty
            self.Move_4_Power_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY)       
            #Sets up Static Text for Move 4 Accuracy, starting empty
            self.Move_4_Accuracy_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Power:', self.Move_4_Power_box, 'Acc.:', self.Move_4_Accuracy_box, self.move_4_sizer)

            #Sets up Boxes for PP and Effect
            #Sets up Static Text for Move 4 PP, starting empty
            self.Move_4_PP_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY)      
            #Sets up Static Text for Move 4 Effect, starting empty
            self.Move_4_Effect_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('PP:', self.Move_4_PP_box, 'Effect:', self.Move_4_Effect_box, self.move_4_sizer)

            #Sets up Static Text for Move 4 Info, starting empty
            self.Move_4_Info_box = wx.TextCtrl(self, value='', style=wx.TE_READONLY | wx.TE_MULTILINE | wx.HSCROLL)
            self.add_widgets('Info:', self.Move_4_Info_box, self.move_4_sizer)

            # ============== # 

            #Adds Move 3 Sizer to Moves 3 and 4 Sizer
            self.moves_3_4_sizer.Add(self.move_3_sizer, 1, wx.EXPAND)

            #Sets up a Static Text to be used as empty space and adds it to  Moves 3 and 4 Sizer
            empty_space_label = wx.StaticText(self, label="    ")
            empty_space_label.SetFont(font_moves)
            self.moves_3_4_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_VERTICAL)

            #Adds Move 4 Sizer to Moves 3 and 4 Sizer
            self.moves_3_4_sizer.Add(self.move_4_sizer, 1, wx.EXPAND)

            #Adds Moves 3 and 4 Sizer to Main Sizer
            self.main_sizer.Add(self.moves_3_4_sizer, 0, wx.EXPAND)

            # ============== #          
                    
        #Sets up a Static Text to be used as empty space and adds it to the main sizer
        empty_space_label = wx.StaticText(self, label="")
        empty_space_label.SetFont(font_moves)
        self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
        
        #Sets up a Static Text to be used as empty space and adds it to the main sizer
        empty_space_label = wx.StaticText(self, label="")
        empty_space_label.SetFont(font_moves)
        self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
        
        #Defines Button Sizer
        btn_sizer = wx.BoxSizer()
        #Creates Save Button
        save_btn = wx.Button(self, label='Save')
        #Binds Button event to Save Button
        save_btn.Bind(wx.EVT_BUTTON, self.on_save_pokemon)
        #Adds Save Button to Button Sizer
        btn_sizer.Add(save_btn, 0, wx.ALL, 5)
        #Creates Preview Button
        preview_btn = wx.Button(self, label='Preview')
        #Binds Button event to Preview Button
        preview_btn.Bind(wx.EVT_BUTTON, self.on_preview_pokemon)
        #Adds Preview Button to Button Sizer
        btn_sizer.Add(preview_btn, 0, wx.ALL, 5)        
        #Adds Cancel Button (pre defined) to Button Sizer
        btn_sizer.Add(wx.Button(self, id=wx.ID_CANCEL), 0, wx.ALL, 5)
        #Adds Button Sizer to Main Sizer
        self.main_sizer.Add(btn_sizer, 0, wx.CENTER)
        
        #Sets Main Sizer for user to see
        self.SetSizer(self.main_sizer)
              
     # ====================================== # Organizing layout   
    
    #This function helps position each box (Text Control, Combo Box, Radio Box) to entry pokemon information
    def add_widgets(self, label_text, entry_box, this_sizer):
        #Defines font style for widgets statict text
        font_labels = wx.Font(12, wx.DECORATIVE, wx.BOLD, wx.NORMAL)   
        #Defines Row Sizer for each "Line"
        row_sizer = wx.BoxSizer(wx.HORIZONTAL)
        #Uses Static Text to Label the boxes
        label = wx.StaticText(self, label=label_text, size=(60, -1))
        #Sets font style to label
        label.SetFont(font_labels)        
        #Adds Static Text Label to Row Sizer
        row_sizer.Add(label, 0, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Adds Box to add information to row sizer
        row_sizer.Add(entry_box, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Adds Row Sizer to Provided Sizer
        this_sizer.Add(row_sizer, 0, wx.EXPAND)
        #self.main_sizer.Add(row_sizer, 0, wx.EXPAND)
        
    #This function helps position each box (Text Control, Combo Box, Radio Box) to entry pokemon information
    def add_widgets_4_boxes_horizontal(self, text_1, box_1, text_2, box_2, this_sizer):
        #Defines font style for widgets statict text
        font_labels = wx.Font(12, wx.DECORATIVE, wx.BOLD, wx.NORMAL)
        #Uses Static Text to Label using text_1
        label_1 = wx.StaticText(self, label=text_1, size=(60, -1))
        #Sets font style to text_1
        label_1.SetFont(font_labels) 
        #Uses Static Text to Label using text_2
        label_2 = wx.StaticText(self, label=text_2, size=(60, -1))
        #Sets font style to label
        label_2.SetFont(font_labels) 
        #Defines Row Sizer for each "Line"
        row_sizer = wx.BoxSizer(wx.HORIZONTAL)        
        #Adds text_1 to Row Sizer
        row_sizer.Add(label_1, 0, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)        
        #Adds box_1 to Row Sizer
        row_sizer.Add(box_1, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Sets up and adds empty space
        empty_space_label = wx.StaticText(self, label="    ")
        row_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_VERTICAL)
        #Adds text_2 to Row Sizer
        row_sizer.Add(label_2, 0, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5) 
        #Adds box_2 to row sizer
        row_sizer.Add(box_2, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Adds Row Sizer to Provided Sizer
        this_sizer.Add(row_sizer, 0, wx.EXPAND)
        #self.main_sizer.Add(row_sizer, 0, wx.EXPAND)
        
    # ====================================== # When a pokemon choice is made from a combo box (Using Dex ONLY)     
        
    def OnSelectSpecies(self, event):
        #Loads pokemon info on self.this_dex_pokemon dictionary using selected species
        self.this_dex_pokemon = load_dex_entry(str(Pokemon_Team_Builder_Frame.dex_path)+str(Pokemon_Team_Builder_Frame.dex_list[self.dex_list_comp.index(str(self.Species_box.GetValue()))]))
        #self.label.SetLabel("selected "+ self.combo.GetValue() +" from Combobox")
        #Resets choices list for forms and images
        self.this_form = []    
        self.this_images = []
        
        #Updates choices list for images
        #Compiles list of files within the provided database directory
        self.files = next(os.walk(Pokemon_Team_Builder_Frame.dex_path))[2]
        #print(self.files)
        #Compiles the list of available images for selected pokemon species
        for f in self.files:
            if f.endswith('.png') and f.startswith(str(Pokemon_Team_Builder_Frame.dex_list[self.dex_list_comp.index(str(self.Species_box.GetValue()))]).replace('.pokedexswsh','')):  #image files for selected pokemon
                self.this_images.append(f)
        #Updates Images Combo Box
        self.Image_box.Clear()
        self.Image_box.Append(self.this_images)
        #Selects last image for Images Combo Box and bitmap image box
        self.Image_box.SetValue(self.this_images[-1])
        #Loads selected image file into bitmap image box
        #Finds size of Top_right_sizer
        W, H = self.Top_right_sizer.Size
        #Loads selected image
        img = wx.Image(Pokemon_Team_Builder_Frame.dex_path+self.Image_box.GetValue(), wx.BITMAP_TYPE_ANY)
        #Places image inside sizer without resizing it, but with cutting/adding edges (making transparent) if they are too long/short
        img = img.Size([int(W),int(H)], pos=[0,0], red=-1, green=-1, blue=-1) #red=-1, green=-1, blue=-1 makes edges transparent
        #Finally sets image into bitmap image box
        self.png.SetBitmap(wx.Bitmap(img))        
            
        #Updates choices list for forms
        for forms in self.this_dex_pokemon['Forms']:
            self.this_form.append(self.this_dex_pokemon['Forms'][forms]['Name'])
        #Updates Forms Combo Box
        self.Form_box.Clear()
        self.Form_box.Append(self.this_form)
        
        #Selects first form as a preset
        self.Form_box.SetValue(self.this_form[0])        
        #Resets relevant choice lists
        self.this_type_1 = []
        self.this_type_2 = []
        self.this_ability_name = []
        self.this_move_name = []
        #Updates relevant choice lists
        self.this_type_1.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Types'][0]) 
        self.this_type_2.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Types'][1])
        self.this_ability_name.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Ability']['Name'])
        #print(self.this_ability_name)
        for moves in self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']:
            self.this_move_name.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves'][moves]['Name'][0])
        #print(self.this_move_name)        
        #Updates Relevant Boxes
        self.Type_1_box.Clear()
        self.Type_1_box.SetValue(self.this_type_1[0])
        #Calls function to color background according to type from pokemon_functions.py
        background_type(self.Type_1_box, str(self.this_type_1[0]))
        self.Type_2_box.Clear()
        self.Type_2_box.SetValue(self.this_type_2[0])
        #Calls function to color background according to type from pokemon_functions.py
        background_type(self.Type_2_box, str(self.this_type_2[0]))
        #Ability Name Box
        self.Ability_Name_box.Clear()
        self.Ability_Name_box.Append(self.this_ability_name[0])
        #Move Name Boxes
        self.Move_1_Name_box.Clear()
        self.Move_1_Name_box.Append(self.this_move_name)        
        self.Move_2_Name_box.Clear()
        self.Move_2_Name_box.Append(self.this_move_name)        
        self.Move_3_Name_box.Clear()
        self.Move_3_Name_box.Append(self.this_move_name)        
        self.Move_4_Name_box.Clear()
        self.Move_4_Name_box.Append(self.this_move_name)        
        
        #Clears all other boxes
        #self.Type_1_box.Clear()
        #background_type(self.Type_1_box, '')
        #self.Type_2_box.Clear()
        #background_type(self.Type_2_box, '')
        #self.Ability_Name_box.Clear()
        self.Ability_Info_box.Clear()
        #self.Move_1_Name_box.Clear()
        self.Move_1_Type_box.Clear()
        background_type(self.Move_1_Type_box, '')
        self.Move_1_Mode_box.Clear()
        self.Move_1_Method_box.Clear()
        self.Move_1_Power_box.Clear()
        self.Move_1_Accuracy_box.Clear()
        self.Move_1_Effect_box.Clear()
        self.Move_1_PP_box.Clear()
        self.Move_1_Info_box.Clear()
        #self.Move_2_Name_box.Clear()
        self.Move_2_Type_box.Clear()
        background_type(self.Move_2_Type_box, '')
        self.Move_2_Mode_box.Clear()
        self.Move_2_Method_box.Clear()
        self.Move_2_Power_box.Clear()
        self.Move_2_Accuracy_box.Clear()
        self.Move_2_Effect_box.Clear()
        self.Move_2_PP_box.Clear()
        self.Move_2_Info_box.Clear()
        #self.Move_3_Name_box.Clear()
        self.Move_3_Type_box.Clear()
        background_type(self.Move_3_Type_box, '')
        self.Move_3_Mode_box.Clear()
        self.Move_3_Method_box.Clear()
        self.Move_3_Power_box.Clear()
        self.Move_3_Accuracy_box.Clear()
        self.Move_3_Effect_box.Clear()
        self.Move_3_PP_box.Clear()
        self.Move_3_Info_box.Clear()
        #self.Move_4_Name_box.Clear()
        self.Move_4_Type_box.Clear()
        background_type(self.Move_4_Type_box, '')
        self.Move_4_Mode_box.Clear()
        self.Move_4_Method_box.Clear()
        self.Move_4_Power_box.Clear()
        self.Move_4_Accuracy_box.Clear()
        self.Move_4_Effect_box.Clear()
        self.Move_4_PP_box.Clear()
        self.Move_4_Info_box.Clear()
        
        
    def OnSelectImage(self, event): 
        #Loads selected image file into bitmap image box
        #Finds size of Top_right_sizer
        W, H = self.Top_right_sizer.Size
        #Loads selected image
        img = wx.Image(Pokemon_Team_Builder_Frame.dex_path+self.Image_box.GetValue(), wx.BITMAP_TYPE_ANY)
        #Gets size of selected image after transforming into bitmap
        #W_img, H_img = wx.Bitmap(Pokemon_Team_Builder_Frame.dex_path+self.Image_box.GetValue(), wx.BITMAP_TYPE_ANY).Size
        #Resizes image according to  Top_right_sizer size
        #NewH = int(H)
        #NewW = int(W_img * H / H_img)
        #Changes size of the image to fit  Top_right_sizer
        #img = img.Scale(NewW,NewH)
        #Places image inside sizer without resizing it, but with cutting/adding edges (making transparent) if they are too long/short
        img = img.Size([int(W),int(H)], pos=[0,0], red=-1, green=-1, blue=-1) #red=-1, green=-1, blue=-1 makes edges transparent
        #Finally sets image into bitmap image box
        self.png.SetBitmap(wx.Bitmap(img))
        #self.png.SetBitmap(wx.Bitmap(Pokemon_Team_Builder_Frame.dex_path+self.Image_box.GetValue(), wx.BITMAP_TYPE_ANY))
   
    def OnSelectForm(self, event):
        #Resets relevant choice lists
        self.this_type_1 = []
        self.this_type_2 = []
        self.this_ability_name = []
        self.this_move_name = []
        #Updates relevant choice lists
        self.this_type_1.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Types'][0]) 
        self.this_type_2.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Types'][1])
        self.this_ability_name.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Ability']['Name'])
        #print(self.this_ability_name)
        for moves in self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']:
            self.this_move_name.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves'][moves]['Name'][0])
        #print(self.this_move_name)
        
        #Updates Relevant Boxes
        self.Type_1_box.Clear()
        self.Type_1_box.SetValue(self.this_type_1[0])
        #Calls function to color background according to type from pokemon_functions.py
        background_type(self.Type_1_box, str(self.this_type_1[0]))
        self.Type_2_box.Clear()
        self.Type_2_box.SetValue(self.this_type_2[0])
        #Calls function to color background according to type from pokemon_functions.py
        background_type(self.Type_2_box, str(self.this_type_2[0]))
        #Ability Name Box
        self.Ability_Name_box.Clear()
        self.Ability_Name_box.Append(self.this_ability_name[0])
        #Move Name Boxes
        self.Move_1_Name_box.Clear()
        self.Move_1_Name_box.Append(self.this_move_name)        
        self.Move_2_Name_box.Clear()
        self.Move_2_Name_box.Append(self.this_move_name)        
        self.Move_3_Name_box.Clear()
        self.Move_3_Name_box.Append(self.this_move_name)        
        self.Move_4_Name_box.Clear()
        self.Move_4_Name_box.Append(self.this_move_name)        
        #Clears all other relevant boxes
        self.Ability_Info_box.Clear()
        self.Move_1_Type_box.Clear()
        background_type(self.Move_1_Type_box, '')
        self.Move_1_Mode_box.Clear()
        self.Move_1_Method_box.Clear()
        self.Move_1_Power_box.Clear()
        self.Move_1_Accuracy_box.Clear()
        self.Move_1_Effect_box.Clear()
        self.Move_1_PP_box.Clear()
        self.Move_1_Info_box.Clear()
        self.Move_2_Type_box.Clear()
        background_type(self.Move_2_Type_box, '')
        self.Move_2_Mode_box.Clear()
        self.Move_2_Method_box.Clear()
        self.Move_2_Power_box.Clear()
        self.Move_2_Accuracy_box.Clear()
        self.Move_2_Effect_box.Clear()
        self.Move_2_PP_box.Clear()
        self.Move_2_Info_box.Clear()
        self.Move_3_Type_box.Clear()
        background_type(self.Move_3_Type_box, '')
        self.Move_3_Mode_box.Clear()
        self.Move_3_Method_box.Clear()
        self.Move_3_Power_box.Clear()
        self.Move_3_Accuracy_box.Clear()
        self.Move_3_Effect_box.Clear()
        self.Move_3_PP_box.Clear()
        self.Move_3_Info_box.Clear()
        self.Move_4_Type_box.Clear()
        background_type(self.Move_4_Type_box, '')
        self.Move_4_Mode_box.Clear()
        self.Move_4_Method_box.Clear()
        self.Move_4_Power_box.Clear()
        self.Move_4_Accuracy_box.Clear()
        self.Move_4_Effect_box.Clear()
        self.Move_4_PP_box.Clear()
        self.Move_4_Info_box.Clear()
        
    def OnSelectAbility(self, event):
        #Resets relevant choice lists
        self.this_ability_info = []
        #Updates relevant choice lists
        self.this_ability_info.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Ability']['Info'])
        #Ability Info Box
        self.Ability_Info_box.Clear()
        self.Ability_Info_box.SetValue(self.this_ability_info[0][self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Ability']['Name'].index(str(self.Ability_Name_box.GetValue()))])
        
        
    def OnSelectMove_1(self, event):
        #Resets relevant choice lists
        self.this_move_1_type = []
        self.this_move_1_mode = []
        self.this_move_1_method = []
        self.this_move_1_power = []
        self.this_move_1_accuracy = []
        self.this_move_1_PP = []
        self.this_move_1_effect = []
        self.this_move_1_info = []
        #Updates relevant choice lists
        self.this_move_1_type.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_1_Name_box.GetValue())))]['Type'][0])
        self.this_move_1_mode.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_1_Name_box.GetValue())))]['Mode'][0])
        self.this_move_1_method.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_1_Name_box.GetValue())))]['Method'][0])
        self.this_move_1_power.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_1_Name_box.GetValue())))]['Power'][0])
        self.this_move_1_accuracy.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_1_Name_box.GetValue())))]['Accuracy'][0])
        self.this_move_1_PP.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_1_Name_box.GetValue())))]['PP'][0])
        self.this_move_1_effect.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_1_Name_box.GetValue())))]['Effect'][0])
        self.this_move_1_info.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_1_Name_box.GetValue())))]['Info'][0])
        #Updates Boxes
        self.Move_1_Type_box.SetValue(self.this_move_1_type[0])
        background_type(self.Move_1_Type_box, str(self.this_move_1_type[0]))
        self.Move_1_Mode_box.SetValue(self.this_move_1_mode[0])
        self.Move_1_Method_box.SetValue(self.this_move_1_method[0])
        self.Move_1_Power_box.SetValue(self.this_move_1_power[0])
        self.Move_1_Accuracy_box.SetValue(self.this_move_1_accuracy[0])
        self.Move_1_PP_box.SetValue(self.this_move_1_PP[0])
        self.Move_1_Effect_box.SetValue(self.this_move_1_effect[0])
        self.Move_1_Info_box.SetValue(self.this_move_1_info[0])
        
        
    def OnSelectMove_2(self, event):
        #Resets relevant choice lists
        self.this_move_2_type = []
        self.this_move_2_mode = []
        self.this_move_2_method = []
        self.this_move_2_power = []
        self.this_move_2_accuracy = []
        self.this_move_2_PP = []
        self.this_move_2_effect = []
        self.this_move_2_info = []
        #Updates relevant choice lists
        self.this_move_2_type.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_2_Name_box.GetValue())))]['Type'][0])
        self.this_move_2_mode.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_2_Name_box.GetValue())))]['Mode'][0])
        self.this_move_2_method.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_2_Name_box.GetValue())))]['Method'][0])
        self.this_move_2_power.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_2_Name_box.GetValue())))]['Power'][0])
        self.this_move_2_accuracy.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_2_Name_box.GetValue())))]['Accuracy'][0])
        self.this_move_2_PP.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_2_Name_box.GetValue())))]['PP'][0])
        self.this_move_2_effect.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_2_Name_box.GetValue())))]['Effect'][0])
        self.this_move_2_info.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_2_Name_box.GetValue())))]['Info'][0])
        #Updates Boxes
        self.Move_2_Type_box.SetValue(self.this_move_2_type[0])
        background_type(self.Move_2_Type_box, str(self.this_move_2_type[0]))
        self.Move_2_Mode_box.SetValue(self.this_move_2_mode[0])
        self.Move_2_Method_box.SetValue(self.this_move_2_method[0])
        self.Move_2_Power_box.SetValue(self.this_move_2_power[0])
        self.Move_2_Accuracy_box.SetValue(self.this_move_2_accuracy[0])
        self.Move_2_PP_box.SetValue(self.this_move_2_PP[0])
        self.Move_2_Effect_box.SetValue(self.this_move_2_effect[0])
        self.Move_2_Info_box.SetValue(self.this_move_2_info[0])
        
    def OnSelectMove_3(self, event):
        #Resets relevant choice lists
        self.this_move_3_type = []
        self.this_move_3_mode = []
        self.this_move_3_method = []
        self.this_move_3_power = []
        self.this_move_3_accuracy = []
        self.this_move_3_PP = []
        self.this_move_3_effect = []
        self.this_move_3_info = []
        #Updates relevant choice lists
        self.this_move_3_type.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_3_Name_box.GetValue())))]['Type'][0])
        self.this_move_3_mode.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_3_Name_box.GetValue())))]['Mode'][0])
        self.this_move_3_method.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_3_Name_box.GetValue())))]['Method'][0])
        self.this_move_3_power.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_3_Name_box.GetValue())))]['Power'][0])
        self.this_move_3_accuracy.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_3_Name_box.GetValue())))]['Accuracy'][0])
        self.this_move_3_PP.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_3_Name_box.GetValue())))]['PP'][0])
        self.this_move_3_effect.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_3_Name_box.GetValue())))]['Effect'][0])
        self.this_move_3_info.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_3_Name_box.GetValue())))]['Info'][0])
        #Updates Boxes
        self.Move_3_Type_box.SetValue(self.this_move_3_type[0])
        background_type(self.Move_3_Type_box, str(self.this_move_3_type[0]))
        self.Move_3_Mode_box.SetValue(self.this_move_3_mode[0])
        self.Move_3_Method_box.SetValue(self.this_move_3_method[0])
        self.Move_3_Power_box.SetValue(self.this_move_3_power[0])
        self.Move_3_Accuracy_box.SetValue(self.this_move_3_accuracy[0])
        self.Move_3_PP_box.SetValue(self.this_move_3_PP[0])
        self.Move_3_Effect_box.SetValue(self.this_move_3_effect[0])
        self.Move_3_Info_box.SetValue(self.this_move_3_info[0])
        
    def OnSelectMove_4(self, event):
        #Resets relevant choice lists
        self.this_move_4_type = []
        self.this_move_4_mode = []
        self.this_move_4_method = []
        self.this_move_4_power = []
        self.this_move_4_accuracy = []
        self.this_move_4_PP = []
        self.this_move_4_effect = []
        self.this_move_4_info = []
        #Updates relevant choice lists
        self.this_move_4_type.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_4_Name_box.GetValue())))]['Type'][0])
        self.this_move_4_mode.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_4_Name_box.GetValue())))]['Mode'][0])
        self.this_move_4_method.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_4_Name_box.GetValue())))]['Method'][0])
        self.this_move_4_power.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_4_Name_box.GetValue())))]['Power'][0])
        self.this_move_4_accuracy.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_4_Name_box.GetValue())))]['Accuracy'][0])
        self.this_move_4_PP.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_4_Name_box.GetValue())))]['PP'][0])
        self.this_move_4_effect.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_4_Name_box.GetValue())))]['Effect'][0])
        self.this_move_4_info.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_4_Name_box.GetValue())))]['Info'][0])
        #Updates Boxes
        self.Move_4_Type_box.SetValue(self.this_move_4_type[0])
        background_type(self.Move_4_Type_box, str(self.this_move_4_type[0]))
        self.Move_4_Mode_box.SetValue(self.this_move_4_mode[0])
        self.Move_4_Method_box.SetValue(self.this_move_4_method[0])
        self.Move_4_Power_box.SetValue(self.this_move_4_power[0])
        self.Move_4_Accuracy_box.SetValue(self.this_move_4_accuracy[0])
        self.Move_4_PP_box.SetValue(self.this_move_4_PP[0])
        self.Move_4_Effect_box.SetValue(self.this_move_4_effect[0])
        self.Move_4_Info_box.SetValue(self.this_move_4_info[0])
        
        
    # ====================================== # When save button is pressed
        
    #This function is called when the save button is pressed. It gets the data from the entry boxes and saves it into a .poke file
    def on_save_pokemon(self, event):
        
        #For initial pokemon info
        self.Name = self.Name_box.GetValue()
        self.Species = self.Species_box.GetValue()
        self.Form = self.Form_box.GetValue()
        self.Image = self.Image_box.GetValue()
        self.Type_1 = self.Type_1_box.GetValue()
        self.Type_2 = self.Type_2_box.GetValue()
        self.Ability_name = self.Ability_Name_box.GetValue()
        self.Ability_info = self.Ability_Info_box.GetValue()
        self.Item = self.Item_box.GetValue()
        
        #When Dex was loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) >1:
            #Copies image file from dex folder to database folder
            shutil.copyfile(Pokemon_Team_Builder_Frame.dex_path+self.Image_box.GetValue(), Pokemon_Team_Builder_Frame.data_path+self.Image_box.GetValue())
        
        #For pokemon move info
        self.Move_1 = self.Move_1_Name_box.GetValue()
        self.Move_type_1 = self.Move_1_Type_box.GetValue()
        #When Dex was not loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) <1:
            self.Move_mode_1 = self.Move_1_Mode_box.GetItemLabel(self.Move_1_Mode_box.GetSelection())
        #When Dex was loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) >1:
            self.Move_mode_1 = self.Move_1_Mode_box.GetValue()
        
        self.Move_info_1 = self.Move_1_Info_box.GetValue()
        self.Move_method_1 = self.Move_1_Method_box.GetValue()
        self.Move_power_1 = self.Move_1_Power_box.GetValue()
        self.Move_accuracy_1 = self.Move_1_Accuracy_box.GetValue()
        self.Move_PP_1 = self.Move_1_PP_box.GetValue()
        self.Move_Effect_1 = self.Move_1_Effect_box.GetValue()
        
        self.Move_2 = self.Move_2_Name_box.GetValue()
        self.Move_type_2 = self.Move_2_Type_box.GetValue()
        #When Dex was not loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) <1:
            self.Move_mode_2 = self.Move_2_Mode_box.GetItemLabel(self.Move_2_Mode_box.GetSelection())
        #When Dex was loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) >1:
            self.Move_mode_2 = self.Move_2_Mode_box.GetValue()
        
        self.Move_info_2 = self.Move_2_Info_box.GetValue()
        self.Move_method_2 = self.Move_2_Method_box.GetValue()
        self.Move_power_2 = self.Move_2_Power_box.GetValue()
        self.Move_accuracy_2 = self.Move_2_Accuracy_box.GetValue()
        self.Move_PP_2 = self.Move_2_PP_box.GetValue()
        self.Move_Effect_2 = self.Move_2_Effect_box.GetValue()
        
        self.Move_3 = self.Move_3_Name_box.GetValue()
        self.Move_type_3 = self.Move_3_Type_box.GetValue()
        #When Dex was not loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) <1:
            self.Move_mode_3 = self.Move_3_Mode_box.GetItemLabel(self.Move_3_Mode_box.GetSelection())
        #When Dex was loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) >1:
            self.Move_mode_3 = self.Move_3_Mode_box.GetValue()
        
        self.Move_info_3 = self.Move_3_Info_box.GetValue()
        self.Move_method_3 = self.Move_3_Method_box.GetValue()
        self.Move_power_3 = self.Move_3_Power_box.GetValue()
        self.Move_accuracy_3 = self.Move_3_Accuracy_box.GetValue()
        self.Move_PP_3 = self.Move_3_PP_box.GetValue()
        self.Move_Effect_3 = self.Move_3_Effect_box.GetValue()
        
        self.Move_4 = self.Move_4_Name_box.GetValue()
        self.Move_type_4 = self.Move_4_Type_box.GetValue()
        #When Dex was not loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) <1:
            self.Move_mode_4 = self.Move_4_Mode_box.GetItemLabel(self.Move_4_Mode_box.GetSelection())
        #When Dex was loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) >1:
            self.Move_mode_4 = self.Move_4_Mode_box.GetValue()
        
        self.Move_info_4 = self.Move_4_Info_box.GetValue()
        self.Move_method_4 = self.Move_4_Method_box.GetValue()
        self.Move_power_4 = self.Move_4_Power_box.GetValue()
        self.Move_accuracy_4 = self.Move_4_Accuracy_box.GetValue()
        self.Move_PP_4 = self.Move_4_PP_box.GetValue()
        self.Move_Effect_4 = self.Move_4_Effect_box.GetValue()
        
        #If any of the variables is empty, except for Pokemon Type 2, the program tells the user to fill up the boxes
        if self.Name == '' or self.Species == '' or self.Form == '' or self.Type_1 == '' or self.Ability_name == '' or self.Ability_info == '' or self.Item == '' or self.Move_1 == '' or self.Move_type_1 == '' or self.Move_mode_1 == '' or self.Move_info_1 == '' or self.Move_method_1 == '' or self.Move_power_1 == '' or self.Move_accuracy_1 == '' or self.Move_PP_1 == '' or self.Move_Effect_1 == '' or self.Move_2 == '' or self.Move_type_2 == '' or self.Move_mode_2 == '' or self.Move_info_2 == '' or self.Move_method_2 == '' or self.Move_power_2 == '' or self.Move_accuracy_2 == '' or self.Move_PP_2 == '' or self.Move_Effect_2 == '' or self.Move_3 == '' or self.Move_type_3 == '' or self.Move_mode_3 == '' or self.Move_info_3 == '' or self.Move_method_3 == '' or self.Move_power_3 == '' or self.Move_accuracy_3 == '' or self.Move_PP_3 == '' or self.Move_Effect_3 == '' or self.Move_4 == '' or self.Move_type_4 == '' or self.Move_mode_4 == '' or self.Move_info_4 == '' or self.Move_method_4 == '' or self.Move_power_4 == '' or self.Move_accuracy_4 == '' or self.Move_PP_4 == '' or self.Move_Effect_4 == '':
            #Displays a default error message dialog telling the user to fill up the fields/boxes
            #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
            dlg = wx.MessageDialog(self, caption='Error!', message = 'One or more fields are emtpy! Please fill up all the fields with Pokemon information.\nPokemon type 2 should be left empty in case the pokemon has only one type.', style= wx.OK | wx.ICON_ERROR)
            dlg.ShowModal()
            #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
            #Closes the message dialog box    
            dlg.Destroy()
        
        #when all fields/boxes (not counting Pokemon type 2) are filled, the program proceeds normally
        else:        
            #When types 1 and 2 are equal, provides error message and allows user to select types again:
            if self.Type_1 == self.Type_2:
                #Calls a default Message Dialog
                #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
                dlg = wx.MessageDialog(self, caption='Error!', message = 'Pokemon types 1 and 2 are equal! Please change one of the two types.', style= wx.OK | wx.ICON_ERROR)
                dlg.ShowModal()
                #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
                #Closes the message dialog box    
                dlg.Destroy()

            #When types 1 and 2 are not equal, the program proceeds to create Pokemon Entry
            else:
                #When the move names are equal, provides error message and allows user to select move names again:
                if self.Move_1 == self.Move_2 or self.Move_1 == self.Move_3 or self.Move_1 == self.Move_4 or self.Move_2 == self.Move_3 or self.Move_2 == self.Move_4 or self.Move_3 == self.Move_4:
                    #Calls a default Message Dialog
                    #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
                    dlg = wx.MessageDialog(self, caption='Error!', message = 'One or more Pokemon move names are equal! Please make all move names unique.', style= wx.OK | wx.ICON_ERROR)
                    dlg.ShowModal()
                    #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
                    #Closes the message dialog box    
                    dlg.Destroy()
                    
                #When move names are not equal, the program proceeds to preview Pokemon Entry    
                else: 
                    #When the pokemon entry already was created, provide the ok or cancel option in an attention dialog
                    if self.Name in Pokemon_Team_Builder_Frame.pokemon_available:
                        #Calls a default attention Message Dialog
                        #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
                        dlg_attention = wx.MessageDialog(self, caption='Attention!', message = 'A Pokemon Entry with this name already exists! Would you like to overwrite it?', style= wx.OK | wx.CANCEL | wx.ICON_WARNING)
                        #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown
                        #If the user presses the attention dialog box’s OK button, the program can get the user’s path choice via the dialog’s .GetPath() function
                        if dlg_attention.ShowModal() == wx.ID_OK:
                            #Overwrites pokemon entry in the specified database
                            register_pokemon(Pokemon_Team_Builder_Frame.data_path,self.Species,self.Form,self.Image,self.Name,self.Type_1,self.Type_2,self.Ability_name,self.Ability_info,self.Item,self.Move_1,self.Move_type_1,self.Move_mode_1,self.Move_info_1,self.Move_method_1,self.Move_power_1,self.Move_accuracy_1,self.Move_PP_1,self.Move_Effect_1,self.Move_2,self.Move_type_2,self.Move_mode_2,self.Move_info_2,self.Move_method_2,self.Move_power_2,self.Move_accuracy_2,self.Move_PP_2,self.Move_Effect_2,self.Move_3,self.Move_type_3,self.Move_mode_3,self.Move_info_3,self.Move_method_3,self.Move_power_3,self.Move_accuracy_3,self.Move_PP_3,self.Move_Effect_3,self.Move_4,self.Move_type_4,self.Move_mode_4,self.Move_info_4,self.Move_method_4,self.Move_power_4,self.Move_accuracy_4,self.Move_PP_4,self.Move_Effect_4)
                            #Displays a default message to say that the Entry was Successfully Overwritten in the Database
                            #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
                            dlg = wx.MessageDialog(self, caption='Success!', message = 'Pokemon entry was successfully overwritten in the database!', style= wx.OK | wx.ICON_INFORMATION)
                            dlg.ShowModal()
                            #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
                            #Closes the message dialog box    
                            dlg.Destroy()
                            #Closes the attention message dialog box    
                            dlg_attention.Destroy()
                            #Closes AddPokemonDialog Box
                            self.Destroy()
                        #When the user doesn't want to overwrite data, the message box closes and the use retuns to the AddPokemon Dialog Box
                        else:
                            #Closes the attention message dialog box    
                            dlg_attention.Destroy()

                    #When the pokemon entry is not in the database, the program can proceed to create it normally
                    elif self.Name not in Pokemon_Team_Builder_Frame.pokemon_available:
                        #Creates pokemon entry in the specified database
                        register_pokemon(Pokemon_Team_Builder_Frame.data_path,self.Species,self.Form,self.Image,self.Name,self.Type_1,self.Type_2,self.Ability_name,self.Ability_info,self.Item,self.Move_1,self.Move_type_1,self.Move_mode_1,self.Move_info_1,self.Move_method_1,self.Move_power_1,self.Move_accuracy_1,self.Move_PP_1,self.Move_Effect_1,self.Move_2,self.Move_type_2,self.Move_mode_2,self.Move_info_2,self.Move_method_2,self.Move_power_2,self.Move_accuracy_2,self.Move_PP_2,self.Move_Effect_2,self.Move_3,self.Move_type_3,self.Move_mode_3,self.Move_info_3,self.Move_method_3,self.Move_power_3,self.Move_accuracy_3,self.Move_PP_3,self.Move_Effect_3,self.Move_4,self.Move_type_4,self.Move_mode_4,self.Move_info_4,self.Move_method_4,self.Move_power_4,self.Move_accuracy_4,self.Move_PP_4,self.Move_Effect_4)
                        #Displays a default message to say that the Entry was Successfully added
                        #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
                        dlg = wx.MessageDialog(self, caption='Success!', message = 'Pokemon entry was successfully added to the database!', style= wx.OK | wx.ICON_INFORMATION)
                        dlg.ShowModal()
                        #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
                        #Closes the message dialog box    
                        dlg.Destroy()
                        #Closes AddPokemonDialog Box
                        self.Destroy()
                    
    # ====================================== # When preview button is pressed
        
    #This function is called when the preview button is pressed. It gets the data from the entry boxes and assembles a pokemon dictionary for previewing the pokemon entry
    #It does not create a .poke file
    def on_preview_pokemon(self, event):
        
        self.Name = self.Name_box.GetValue()
        self.Species = self.Species_box.GetValue()
        self.Form = self.Form_box.GetValue()
        self.Image = self.Image_box.GetValue()
        self.Type_1 = self.Type_1_box.GetValue()
        self.Type_2 = self.Type_2_box.GetValue()
        self.Ability_name = self.Ability_Name_box.GetValue()
        self.Ability_info = self.Ability_Info_box.GetValue()
        self.Item = self.Item_box.GetValue()
        
        self.Move_1 = self.Move_1_Name_box.GetValue()
        self.Move_type_1 = self.Move_1_Type_box.GetValue()
        #When Dex was not loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) <1:
            self.Move_mode_1 = self.Move_1_Mode_box.GetItemLabel(self.Move_1_Mode_box.GetSelection())
        #When Dex was loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) >1:
            self.Move_mode_1 = self.Move_1_Mode_box.GetValue()
        
        self.Move_info_1 = self.Move_1_Info_box.GetValue()
        self.Move_method_1 = self.Move_1_Method_box.GetValue()
        self.Move_power_1 = self.Move_1_Power_box.GetValue()
        self.Move_accuracy_1 = self.Move_1_Accuracy_box.GetValue()
        self.Move_PP_1 = self.Move_1_PP_box.GetValue()
        self.Move_Effect_1 = self.Move_1_Effect_box.GetValue()
        
        self.Move_2 = self.Move_2_Name_box.GetValue()
        self.Move_type_2 = self.Move_2_Type_box.GetValue()
        #When Dex was not loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) <1:
            self.Move_mode_2 = self.Move_2_Mode_box.GetItemLabel(self.Move_2_Mode_box.GetSelection())
        #When Dex was loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) >1:
            self.Move_mode_2 = self.Move_2_Mode_box.GetValue()
        
        self.Move_info_2 = self.Move_2_Info_box.GetValue()
        self.Move_method_2 = self.Move_2_Method_box.GetValue()
        self.Move_power_2 = self.Move_2_Power_box.GetValue()
        self.Move_accuracy_2 = self.Move_2_Accuracy_box.GetValue()
        self.Move_PP_2 = self.Move_2_PP_box.GetValue()
        self.Move_Effect_2 = self.Move_2_Effect_box.GetValue()
        
        self.Move_3 = self.Move_3_Name_box.GetValue()
        self.Move_type_3 = self.Move_3_Type_box.GetValue()
        #When Dex was not loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) <1:
            self.Move_mode_3 = self.Move_3_Mode_box.GetItemLabel(self.Move_3_Mode_box.GetSelection())
        #When Dex was loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) >1:
            self.Move_mode_3 = self.Move_3_Mode_box.GetValue()
        
        self.Move_info_3 = self.Move_3_Info_box.GetValue()
        self.Move_method_3 = self.Move_3_Method_box.GetValue()
        self.Move_power_3 = self.Move_3_Power_box.GetValue()
        self.Move_accuracy_3 = self.Move_3_Accuracy_box.GetValue()
        self.Move_PP_3 = self.Move_3_PP_box.GetValue()
        self.Move_Effect_3 = self.Move_3_Effect_box.GetValue()
        
        self.Move_4 = self.Move_4_Name_box.GetValue()
        self.Move_type_4 = self.Move_4_Type_box.GetValue()
        #When Dex was not loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) <1:
            self.Move_mode_4 = self.Move_4_Mode_box.GetItemLabel(self.Move_4_Mode_box.GetSelection())
        #When Dex was loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) >1:
            self.Move_mode_4 = self.Move_4_Mode_box.GetValue()
        
        self.Move_info_4 = self.Move_4_Info_box.GetValue()
        self.Move_method_4 = self.Move_4_Method_box.GetValue()
        self.Move_power_4 = self.Move_4_Power_box.GetValue()
        self.Move_accuracy_4 = self.Move_4_Accuracy_box.GetValue()
        self.Move_PP_4 = self.Move_4_PP_box.GetValue()
        self.Move_Effect_4 = self.Move_4_Effect_box.GetValue()
        
        #If any of the variables is empty, except for Pokemon Type 2, the program tells the user to fill up the boxes
        if self.Name == '' or self.Species == '' or self.Form == '' or self.Type_1 == '' or self.Ability_name == '' or self.Ability_info == '' or self.Item == '' or self.Move_1 == '' or self.Move_type_1 == '' or self.Move_mode_1 == '' or self.Move_info_1 == '' or self.Move_method_1 == '' or self.Move_power_1 == '' or self.Move_accuracy_1 == '' or self.Move_PP_1 == '' or self.Move_Effect_1 == '' or self.Move_2 == '' or self.Move_type_2 == '' or self.Move_mode_2 == '' or self.Move_info_2 == '' or self.Move_method_2 == '' or self.Move_power_2 == '' or self.Move_accuracy_2 == '' or self.Move_PP_2 == '' or self.Move_Effect_2 == '' or self.Move_3 == '' or self.Move_type_3 == '' or self.Move_mode_3 == '' or self.Move_info_3 == '' or self.Move_method_3 == '' or self.Move_power_3 == '' or self.Move_accuracy_3 == '' or self.Move_PP_3 == '' or self.Move_Effect_3 == '' or self.Move_4 == '' or self.Move_type_4 == '' or self.Move_mode_4 == '' or self.Move_info_4 == '' or self.Move_method_4 == '' or self.Move_power_4 == '' or self.Move_accuracy_4 == '' or self.Move_PP_4 == '' or self.Move_Effect_4 == '':
            #Displays a default error message dialog telling the user to fill up the fields/boxes
            #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
            dlg = wx.MessageDialog(self, caption='Error!', message = 'One or more fields are emtpy! Please fill up all the fields with Pokemon information.\nPokemon type 2 should be left empty in case the pokemon has only one type.', style= wx.OK | wx.ICON_ERROR)
            dlg.ShowModal()
            #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
            #Closes the message dialog box    
            dlg.Destroy()
        
        #when all fields/boxes (not counting Pokemon type 2) are filled, the program proceeds normally
        else:        
            #When types 1 and 2 are equal, provides error message and allows user to select types again:
            if self.Type_1 == self.Type_2:
                #Calls a default Message Dialog
                #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
                dlg = wx.MessageDialog(self, caption='Error!', message = 'Pokemon types 1 and 2 are equal! Please change one of the two types.', style= wx.OK | wx.ICON_ERROR)
                dlg.ShowModal()
                #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
                #Closes the message dialog box    
                dlg.Destroy()

            #When types 1 and 2 are not equal, the program proceeds to preview Pokemon Entry
            else:
                #When the move names are equal, provides error message and allows user to select move names again:
                if self.Move_1 == self.Move_2 or self.Move_1 == self.Move_3 or self.Move_1 == self.Move_4 or self.Move_2 == self.Move_3 or self.Move_2 == self.Move_4 or self.Move_3 == self.Move_4:
                    #Calls a default Message Dialog
                    #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
                    dlg = wx.MessageDialog(self, caption='Error!', message = 'One or more Pokemon move names are equal! Please make all move names unique.', style= wx.OK | wx.ICON_ERROR)
                    dlg.ShowModal()
                    #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
                    #Closes the message dialog box    
                    dlg.Destroy()
                    
                #When move names are not equal, the program proceeds to preview Pokemon Entry    
                else:                    
                    #When the pokemon entry already was created, provide the ok or cancel option in an attention dialog
                    if self.Name in Pokemon_Team_Builder_Frame.pokemon_available:
                        #Calls a default attention Message Dialog
                        #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
                        dlg_attention = wx.MessageDialog(self, caption='Attention!', message = 'A Pokemon Entry with this name already exists! Would you still like to preview it?', style= wx.OK | wx.CANCEL | wx.ICON_WARNING)
                        #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown
                        #If the user presses the attention dialog box’s OK button, the program can get the user’s path choice via the dialog’s .GetPath() function
                        if dlg_attention.ShowModal() == wx.ID_OK:
                            #Uses function to assemble pokemon entry into local dictionary
                            self.this_pokemon_preview = preview_pokemon(self.Species,self.Form,self.Image,self.Name,self.Type_1,self.Type_2,self.Ability_name,self.Ability_info,self.Item,self.Move_1,self.Move_type_1,self.Move_mode_1,self.Move_info_1,self.Move_method_1,self.Move_power_1,self.Move_accuracy_1,self.Move_PP_1,self.Move_Effect_1,self.Move_2,self.Move_type_2,self.Move_mode_2,self.Move_info_2,self.Move_method_2,self.Move_power_2,self.Move_accuracy_2,self.Move_PP_2,self.Move_Effect_2,self.Move_3,self.Move_type_3,self.Move_mode_3,self.Move_info_3,self.Move_method_3,self.Move_power_3,self.Move_accuracy_3,self.Move_PP_3,self.Move_Effect_3,self.Move_4,self.Move_type_4,self.Move_mode_4,self.Move_info_4,self.Move_method_4,self.Move_power_4,self.Move_accuracy_4,self.Move_PP_4,self.Move_Effect_4)
                            #Passes previewed pokemon dictionary to the ViewPokemonDialog_from_dict Box
                            #Sets up ViewPokemonDialog_from_dict Box
                            dlg = ViewPokemonDialog_from_dict(self.this_pokemon_preview, preview='yes')
                            #Displays ViewPokemonDialog_from_dict Box
                            dlg.Show()
                        #When the user doesn't want to preview pokemon entry, the message box closes and the use retuns to the AddPokemon Dialog Box
                        else:
                            #Closes the attention message dialog box    
                            dlg_attention.Destroy()

                    #When the pokemon entry is not in the database, the program can proceed to create it normally
                    elif self.Name not in Pokemon_Team_Builder_Frame.pokemon_available:
                        #Uses function to assemble pokemon entry into local dictionary
                        self.this_pokemon_preview = preview_pokemon(self.Species,self.Form,self.Image,self.Name,self.Type_1,self.Type_2,self.Ability_name,self.Ability_info,self.Item,self.Move_1,self.Move_type_1,self.Move_mode_1,self.Move_info_1,self.Move_method_1,self.Move_power_1,self.Move_accuracy_1,self.Move_PP_1,self.Move_Effect_1,self.Move_2,self.Move_type_2,self.Move_mode_2,self.Move_info_2,self.Move_method_2,self.Move_power_2,self.Move_accuracy_2,self.Move_PP_2,self.Move_Effect_2,self.Move_3,self.Move_type_3,self.Move_mode_3,self.Move_info_3,self.Move_method_3,self.Move_power_3,self.Move_accuracy_3,self.Move_PP_3,self.Move_Effect_3,self.Move_4,self.Move_type_4,self.Move_mode_4,self.Move_info_4,self.Move_method_4,self.Move_power_4,self.Move_accuracy_4,self.Move_PP_4,self.Move_Effect_4)
                        #Passes previewed pokemon dictionary to the ViewPokemonDialog_from_dict Box
                        #Sets up ViewPokemonDialog_from_dict Box
                        dlg = ViewPokemonDialog_from_dict(self.this_pokemon_preview, preview='yes')
                        #Displays ViewPokemonDialog_from_dict Box
                        dlg.Show()

# ======================================================================================= # Edit Pokemon Entry
        
#Defines Edit Pokemon Entry as a Dialog Box
class EditPokemonDialog(wx.Dialog):
            
    def __init__(self, pokemon_name):
        
        #Reads selected pokemon data into a python dictionary named this_Pokemon
        this_Pokemon = read_pokemon(Pokemon_Team_Builder_Frame.data_path,pokemon_name)
        
        #When Dex was loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) >1:
            #Creates a complimentary list to Pokemon_Team_Builder_Frame.dex_list without pokemon number and .pokedexswsh in the pokemon name
            self.dex_list_comp = []
            for poke in Pokemon_Team_Builder_Frame.dex_list:
                #print(poke)
                self.dex_list_comp.append(poke.split('_')[1].replace('.pokedexswsh',''))
                
        #When Dex was not loaded
        else:
            #Creates a complimentary list to Pokemon_Team_Builder_Frame.dex_list without pokemon number and .pokedexswsh in the pokemon name
            self.dex_list_comp = [] #stays empty because Dex was not loaded, but still useful
            
                
        #When Dex was loaded and the pokemon entry's species exists in the dex, information from dex will be loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) >1 and this_Pokemon['Species'][0] in self.dex_list_comp:
            
            #Creates/resets useful dictionary to handle selected pokemon from Dex
            self.this_dex_pokemon = {}
            
            #Loads pokemon info on self.this_dex_pokemon dictionary using selected species
            self.this_dex_pokemon = load_dex_entry(str(Pokemon_Team_Builder_Frame.dex_path)+str(Pokemon_Team_Builder_Frame.dex_list[self.dex_list_comp.index(this_Pokemon['Species'][0])]))
            
            #Resets choices list for forms and images
            self.this_form = []    
            self.this_images = []
            #Updates choices list for images using pokemon entry's species
            #Compiles list of files within the provided database directory
            self.files = next(os.walk(Pokemon_Team_Builder_Frame.dex_path))[2]
            #print(self.files)
            #Compiles the list of available images for pokemon entry's species
            for f in self.files:
                if f.endswith('.png') and f.startswith(str(Pokemon_Team_Builder_Frame.dex_list[self.dex_list_comp.index(this_Pokemon['Species'][0])]).replace('.pokedexswsh','')):  #image files for selected pokemon
                    self.this_images.append(f)                    
            #Updates choices list for forms using pokemon entry's species
            for forms in self.this_dex_pokemon['Forms']:
                self.this_form.append(self.this_dex_pokemon['Forms'][forms]['Name'])
                
            
        #Initiating dialog box and choosing its size
        #When Dex was not loaded; or Dex was loaded but pokemon entry's species does not exists in dex;
        # or Dex was loaded but pokemon entry's form does not exists in dex
        if len(Pokemon_Team_Builder_Frame.dex_list) <1 or this_Pokemon['Species'][0] not in self.dex_list_comp or this_Pokemon['Form'][0] not in self.this_form:
            super().__init__(parent=None, title="Edit Pokemon Entry", style = wx.DEFAULT_DIALOG_STYLE | wx.RESIZE_BORDER, size = (1140,935))               
        #When Dex was loaded and the pokemon entry's species exists in the dex and pokemon entry's form exists in the dex
        if len(Pokemon_Team_Builder_Frame.dex_list) >1 and this_Pokemon['Species'][0] in self.dex_list_comp and this_Pokemon['Form'][0] in self.this_form:     
            super().__init__(parent=None, title="Edit Pokemon Entry - Dex Assisted", style = wx.DEFAULT_DIALOG_STYLE | wx.RESIZE_BORDER, size = (1140,990))        
      
                
        #Defines Helpful Variables
        self.all_types = ['Normal', 'Fighting', 'Flying', 'Poison', 'Ground', 'Rock', 'Bug', 'Ghost', 'Steel', 'Fire', 'Water', 'Grass', 'Electric', 'Psychic', 'Ice', 'Dragon', 'Dark', 'Fairy']
        self.all_types_2 = ['', 'Normal', 'Fighting', 'Flying', 'Poison', 'Ground', 'Rock', 'Bug', 'Ghost', 'Steel', 'Fire', 'Water', 'Grass', 'Electric', 'Psychic', 'Ice', 'Dragon', 'Dark', 'Fairy']
        self.all_modes = ['Physical', 'Special', 'Status']
        
        #Defines Main Sizer
        self.main_sizer = wx.BoxSizer(wx.VERTICAL)
        
        #Defines font style for the statict text that labels the Moves and Pokemon
        font_moves = wx.Font(12, wx.DECORATIVE, wx.BOLD, wx.NORMAL)
        
        #Sets up a Static Text to be used as empty space and adds it to the main sizer
        empty_space_label = wx.StaticText(self, label="")
        empty_space_label.SetFont(font_moves)
        self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
        
        #Sets up a Static Text for Pokemon and adds it to the main sizer
        pokemon_label = wx.StaticText(self, label="Pokemon Entry")
        pokemon_label.SetFont(font_moves)
        self.main_sizer.Add(pokemon_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
        
        #When Dex was not loaded; or Dex was loaded but pokemon entry's species does not exists in dex;
        # or Dex was loaded but pokemon entry's form does not exists in dex
        if len(Pokemon_Team_Builder_Frame.dex_list) <1 or this_Pokemon['Species'][0] not in self.dex_list_comp or this_Pokemon['Form'][0] not in self.this_form:
            
            #Defines Top_left_right Sizer
            self.Top_left_right_sizer = wx.BoxSizer(wx.HORIZONTAL)

            #Defines Top_left Sizer
            self.Top_left_sizer = wx.BoxSizer(wx.VERTICAL)

            #Defines Top_right Sizer
            self.Top_right_sizer = wx.BoxSizer(wx.VERTICAL)
            
            #Sets up a Static Text to be used as empty space and adds it to Top_left Sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.Top_left_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            
            #Sets up a Static Text to be used as empty space and adds it to Top_left Sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.Top_left_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            
            #Sets up Box for Name
            self.Name_box = wx.TextCtrl(self, value=this_Pokemon['Name'][0], style=wx.TE_READONLY)
            self.add_widgets('Name:', self.Name_box, self.Top_left_sizer)
            
            #Creates bitmap image box, starting empty
            self.png = wx.StaticBitmap(self, -1)
            #Adds bitmap Image box to Top_right Sizer
            self.Top_right_sizer.Add(self.png, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            #Loads pokemon entry's image file into bitmap image box
            #Loads selected image
            img = wx.Image(Pokemon_Team_Builder_Frame.data_path+this_Pokemon['Image'][0], wx.BITMAP_TYPE_ANY)
            #Finally sets image into bitmap image box, if there was an image to begin with
            try:
                self.png.SetBitmap(wx.Bitmap(img))
            except:
                pass
            
            #Sets up Boxes for Species and Form
            #Sets up Box for Species
            self.Species_box = wx.TextCtrl(self, value=this_Pokemon['Species'][0], style=wx.TE_READONLY)
            self.add_widgets('Species:', self.Species_box, self.Top_left_sizer)        
            #Sets up Box for Form
            self.Form_box = wx.TextCtrl(self, value=this_Pokemon['Form'][0])
            self.add_widgets('Form:', self.Form_box, self.Top_left_sizer)
            #self.add_widgets_4_boxes_horizontal('Species:', self.Species_box, 'Form:', self.Form_box, self.main_sizer)

            #Sets up Combo Boxes for Types
            #Sets up Combo Box for Type_1
            self.Type_1_box = wx.ComboBox(self, size=(95, -1), value=this_Pokemon['Types'][0], choices=self.all_types, style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
            #self.add_widgets('Type 1:', self.Type_1_box)        
            #Sets up Combo Box for Type_2
            self.Type_2_box = wx.ComboBox(self, size=(95, -1), value=this_Pokemon['Types'][1], choices=self.all_types_2, style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
            #self.add_widgets('Type 2:', self.Type_2_box)        
            self.add_widgets_4_boxes_horizontal('Type 1:', self.Type_1_box, 'Type 2:', self.Type_2_box, self.Top_left_sizer)

            #Sets up Box for Item
            self.Item_box = wx.TextCtrl(self, value=this_Pokemon['Item'][0])
            self.add_widgets('Item:', self.Item_box, self.Top_left_sizer)
            
            #Sets up Box for Image
            self.Image_box = wx.TextCtrl(self, value=this_Pokemon['Image'][0], style=wx.TE_READONLY)
            self.add_widgets('Image:', self.Image_box, self.Top_left_sizer)

            #Sets up a Static Text to be used as empty space and adds it to Top_left Sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.Top_left_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            
            #Sets up a Static Text to be used as empty space and adds it to Top_left Sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.Top_left_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Adds Top_left Sizer to Top_left_right Sizer
            self.Top_left_right_sizer.Add(self.Top_left_sizer, 1, wx.EXPAND)
            
            #Sets up a Static Text to be used as empty space and adds it to Top_left_right Sizer
            empty_space_label = wx.StaticText(self, label="    ")
            empty_space_label.SetFont(font_moves)
            self.Top_left_right_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_VERTICAL)
            
            #Adds Top_right Sizer to Top_left_right Sizer
            self.Top_left_right_sizer.Add(self.Top_right_sizer, 1, wx.EXPAND)

            #Adds self.Top_left_right to Main Sizer
            self.main_sizer.Add(self.Top_left_right_sizer, 0, wx.EXPAND)

            #Sets up Boxes for Ability_Name and Ability_Info
            #Sets up a Static Text for Ability and adds it to the main sizer
            Ability_label = wx.StaticText(self, label="Ability")
            Ability_label.SetFont(font_moves)
            self.main_sizer.Add(Ability_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            #Sets up Box for Ability_Name
            self.Ability_Name_box = wx.TextCtrl(self, value=this_Pokemon['Ability']['Ability_Name'][0])
            #self.add_widgets('Ability Name:', self.Ability_Name_box)        
            #Sets up Box for Ability_Info
            self.Ability_Info_box = wx.TextCtrl(self, value=this_Pokemon['Ability']['Ability_Info'][0])
            #self.add_widgets('Ability Info:', self.Ability_Info_box)
            self.add_widgets_4_boxes_horizontal('Name:', self.Ability_Name_box, 'Info:', self.Ability_Info_box, self.main_sizer)         

            # ============================ #  Moves 1 and 2

            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Defines Moves 1 and 2 Sizer
            self.moves_1_2_sizer = wx.BoxSizer(wx.HORIZONTAL)

            #Defines Move 1 Sizer
            self.move_1_sizer = wx.BoxSizer(wx.VERTICAL)

            #Defines Move 2 Sizer
            self.move_2_sizer = wx.BoxSizer(wx.VERTICAL)

            # ============== #  Move 1

            #Sets up a Static Text for Move 1 and adds it to the move 1 sizer
            Move_1_label = wx.StaticText(self, label="Move 1")
            Move_1_label.SetFont(font_moves)
            self.move_1_sizer.Add(Move_1_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Sets up Boxes for Name and Mode
            #Sets up Box for Move 1 Name
            self.Move_1_Name_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Name'][0])
            #Sets up Radio Box for Move 1 Mode
            self.Move_1_Mode_box = wx.RadioBox(self, choices=self.all_modes,  majorDimension=3, style=wx.RA_SPECIFY_COLS)
            self.Move_1_Mode_box.SetSelection(self.Move_1_Mode_box.FindString(this_Pokemon['Moves']['Move_Mode'][0]))
            self.add_widgets_4_boxes_horizontal('Name:', self.Move_1_Name_box, 'Mode:', self.Move_1_Mode_box, self.move_1_sizer)

            #Sets up Boxes for Type and Method        
            #Sets up Combo Box for Move 1 Type
            self.Move_1_Type_box = wx.ComboBox(self, size=(95, -1), value=this_Pokemon['Moves']['Move_Type'][0], choices=self.all_types, style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
            #Sets up Static Text for Move 1 Method
            self.Move_1_Method_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Method'][0])
            self.add_widgets_4_boxes_horizontal('Type:', self.Move_1_Type_box, 'Method:', self.Move_1_Method_box, self.move_1_sizer)

            #Sets up Boxes for Power and Accuracy
            #Sets up Static Text for Move 1 Power
            self.Move_1_Power_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Power'][0])       
            #Sets up Static Text for Move 1 Accuracy
            self.Move_1_Accuracy_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Accuracy'][0])
            self.add_widgets_4_boxes_horizontal('Power:', self.Move_1_Power_box, 'Acc.:', self.Move_1_Accuracy_box, self.move_1_sizer)

            #Sets up Boxes for PP and Effect
            #Sets up Static Text for Move 1 PP
            self.Move_1_PP_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_PP'][0])      
            #Sets up Static Text for Move 1 Effect
            self.Move_1_Effect_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Effect'][0])
            self.add_widgets_4_boxes_horizontal('PP:', self.Move_1_PP_box, 'Effect:', self.Move_1_Effect_box, self.move_1_sizer)

            #Sets up Static Text for Move 1 Info
            self.Move_1_Info_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Info'][0])
            self.add_widgets('Info:', self.Move_1_Info_box, self.move_1_sizer)

            # ============== #  Move 2

            #Sets up a Static Text for Move 2 and adds it to the move 2 sizer
            Move_2_label = wx.StaticText(self, label="Move 2")
            Move_2_label.SetFont(font_moves)
            self.move_2_sizer.Add(Move_2_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Sets up Boxes for Name and Mode
            #Sets up Box for Move 2 Name
            self.Move_2_Name_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Name'][1])
            #Sets up Radio Box for Move 2 Mode
            self.Move_2_Mode_box = wx.RadioBox(self, choices=self.all_modes,  majorDimension=3, style=wx.RA_SPECIFY_COLS)
            self.Move_2_Mode_box.SetSelection(self.Move_2_Mode_box.FindString(this_Pokemon['Moves']['Move_Mode'][1]))
            self.add_widgets_4_boxes_horizontal('Name:', self.Move_2_Name_box, 'Mode:', self.Move_2_Mode_box, self.move_2_sizer)

            #Sets up Boxes for Type and Method        
            #Sets up Combo Box for Move 2 Type
            self.Move_2_Type_box = wx.ComboBox(self, size=(95, -1), value=this_Pokemon['Moves']['Move_Type'][1], choices=self.all_types, style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
            #Sets up Static Text for Move 2 Method
            self.Move_2_Method_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Method'][1])
            self.add_widgets_4_boxes_horizontal('Type:', self.Move_2_Type_box, 'Method:', self.Move_2_Method_box, self.move_2_sizer)

            #Sets up Boxes for Power and Accuracy
            #Sets up Static Text for Move 2 Power
            self.Move_2_Power_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Power'][1])       
            #Sets up Static Text for Move 2 Accuracy
            self.Move_2_Accuracy_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Accuracy'][1])
            self.add_widgets_4_boxes_horizontal('Power:', self.Move_2_Power_box, 'Acc.:', self.Move_2_Accuracy_box, self.move_2_sizer)

            #Sets up Boxes for PP and Effect
            #Sets up Static Text for Move 2 PP
            self.Move_2_PP_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_PP'][1])      
            #Sets up Static Text for Move 2 Effect
            self.Move_2_Effect_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Effect'][1])
            self.add_widgets_4_boxes_horizontal('PP:', self.Move_2_PP_box, 'Effect:', self.Move_2_Effect_box, self.move_2_sizer)

            #Sets up Static Text for Move 2 Info
            self.Move_2_Info_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_PP'][1])
            self.add_widgets('Info:', self.Move_2_Info_box, self.move_2_sizer)

            # ============== # 

            #Adds Move 1 Sizer to Moves 1 and 2 Sizer
            self.moves_1_2_sizer.Add(self.move_1_sizer, 1, wx.EXPAND)

            #Sets up a Static Text to be used as empty space and adds it to  Moves 1 and 2 Sizer
            empty_space_label = wx.StaticText(self, label="    ")
            empty_space_label.SetFont(font_moves)
            self.moves_1_2_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_VERTICAL)

            #Adds Move 2 Sizer to Moves 1 and 2 Sizer
            self.moves_1_2_sizer.Add(self.move_2_sizer, 1, wx.EXPAND)

            #Adds Moves 1 and 2 Sizer to Main Sizer
            self.main_sizer.Add(self.moves_1_2_sizer, 0, wx.EXPAND)

            # ============================ #  Moves 3 and 4

            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Defines Moves 3 and 4 Sizer
            self.moves_3_4_sizer = wx.BoxSizer(wx.HORIZONTAL)

            #Defines Move 3 Sizer
            self.move_3_sizer = wx.BoxSizer(wx.VERTICAL)

            #Defines Move 4 Sizer
            self.move_4_sizer = wx.BoxSizer(wx.VERTICAL)

            # ============== #  Move 3

            #Sets up a Static Text for Move 3 and adds it to the move 3 sizer
            Move_3_label = wx.StaticText(self, label="Move 3")
            Move_3_label.SetFont(font_moves)
            self.move_3_sizer.Add(Move_3_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Sets up Boxes for Name and Mode
            #Sets up Box for Move 3 Name
            self.Move_3_Name_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Name'][2])
            #Sets up Radio Box for Move 3 Mode
            self.Move_3_Mode_box = wx.RadioBox(self, choices=self.all_modes,  majorDimension=3, style=wx.RA_SPECIFY_COLS)
            self.Move_3_Mode_box.SetSelection(self.Move_3_Mode_box.FindString(this_Pokemon['Moves']['Move_Mode'][2]))
            self.add_widgets_4_boxes_horizontal('Name:', self.Move_3_Name_box, 'Mode:', self.Move_3_Mode_box, self.move_3_sizer)

            #Sets up Boxes for Type and Method        
            #Sets up Combo Box for Move 3 Type
            self.Move_3_Type_box = wx.ComboBox(self, value=this_Pokemon['Moves']['Move_Type'][2], size=(95, -1), choices=self.all_types, style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
            #Sets up Static Text for Move 3 Method
            self.Move_3_Method_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Method'][2])
            self.add_widgets_4_boxes_horizontal('Type:', self.Move_3_Type_box, 'Method:', self.Move_3_Method_box, self.move_3_sizer)

            #Sets up Boxes for Power and Accuracy
            #Sets up Static Text for Move 3 Power
            self.Move_3_Power_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Power'][2])       
            #Sets up Static Text for Move 3 Accuracy
            self.Move_3_Accuracy_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Accuracy'][2])
            self.add_widgets_4_boxes_horizontal('Power:', self.Move_3_Power_box, 'Acc.:', self.Move_3_Accuracy_box, self.move_3_sizer)

            #Sets up Boxes for PP and Effect
            #Sets up Static Text for Move 3 PP
            self.Move_3_PP_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_PP'][2])      
            #Sets up Static Text for Move 3 Effect
            self.Move_3_Effect_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Effect'][2])
            self.add_widgets_4_boxes_horizontal('PP:', self.Move_3_PP_box, 'Effect:', self.Move_3_Effect_box, self.move_3_sizer)

            #Sets up Static Text for Move 3 Info
            self.Move_3_Info_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Info'][2])
            self.add_widgets('Info:', self.Move_3_Info_box, self.move_3_sizer)

            # ============== #  Move 4

            #Sets up a Static Text for Move 4 and adds it to the move 4 sizer
            Move_4_label = wx.StaticText(self, label="Move 4")
            Move_4_label.SetFont(font_moves)
            self.move_4_sizer.Add(Move_4_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Sets up Boxes for Name and Mode
            #Sets up Box for Move 4 Name
            self.Move_4_Name_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Name'][3])
            #Sets up Radio Box for Move 4 Mode
            self.Move_4_Mode_box = wx.RadioBox(self, choices=self.all_modes,  majorDimension=3, style=wx.RA_SPECIFY_COLS)
            self.Move_4_Mode_box.SetSelection(self.Move_4_Mode_box.FindString(this_Pokemon['Moves']['Move_Mode'][3]))
            self.add_widgets_4_boxes_horizontal('Name:', self.Move_4_Name_box, 'Mode:', self.Move_4_Mode_box, self.move_4_sizer)

            #Sets up Boxes for Type and Method        
            #Sets up Combo Box for Move 4 Type
            self.Move_4_Type_box = wx.ComboBox(self, value=this_Pokemon['Moves']['Move_Type'][3], size=(95, -1), choices=self.all_types, style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
            #Sets up Static Text for Move 4 Method
            self.Move_4_Method_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Method'][3])
            self.add_widgets_4_boxes_horizontal('Type:', self.Move_4_Type_box, 'Method:', self.Move_4_Method_box, self.move_4_sizer)

            #Sets up Boxes for Power and Accuracy
            #Sets up Static Text for Move 4 Power
            self.Move_4_Power_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Power'][3])       
            #Sets up Static Text for Move 4 Accuracy
            self.Move_4_Accuracy_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Accuracy'][3])
            self.add_widgets_4_boxes_horizontal('Power:', self.Move_4_Power_box, 'Acc.:', self.Move_4_Accuracy_box, self.move_4_sizer)

            #Sets up Boxes for PP and Effect
            #Sets up Static Text for Move 4 PP
            self.Move_4_PP_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_PP'][3])      
            #Sets up Static Text for Move 4 Effect
            self.Move_4_Effect_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Effect'][3])
            self.add_widgets_4_boxes_horizontal('PP:', self.Move_4_PP_box, 'Effect:', self.Move_4_Effect_box, self.move_4_sizer)

            #Sets up Static Text for Move 4 Info
            self.Move_4_Info_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Info'][3])
            self.add_widgets('Info:', self.Move_4_Info_box, self.move_4_sizer)

            # ============== # 

            #Adds Move 3 Sizer to Moves 3 and 4 Sizer
            self.moves_3_4_sizer.Add(self.move_3_sizer, 1, wx.EXPAND)

            #Sets up a Static Text to be used as empty space and adds it to  Moves 3 and 4 Sizer
            empty_space_label = wx.StaticText(self, label="    ")
            empty_space_label.SetFont(font_moves)
            self.moves_3_4_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_VERTICAL)

            #Adds Move 4 Sizer to Moves 3 and 4 Sizer
            self.moves_3_4_sizer.Add(self.move_4_sizer, 1, wx.EXPAND)

            #Adds Moves 3 and 4 Sizer to Main Sizer
            self.main_sizer.Add(self.moves_3_4_sizer, 0, wx.EXPAND)

            # ============== #
            
        #When Dex was loaded and the pokemon entry's species exists in the dex and pokemon entry's form exists in the dex
        if len(Pokemon_Team_Builder_Frame.dex_list) >1 and this_Pokemon['Species'][0] in self.dex_list_comp and this_Pokemon['Form'][0] in self.this_form:     
            
            #Resets relevant choice lists as blank
            self.this_type_1 = []
            self.this_type_2 = []
            self.this_ability_name = []
            self.this_move_name = []
            #Updates relevant choice lists using pokemon entry's form
            self.this_type_1.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(this_Pokemon['Form'][0]))]['Types'][0]) 
            self.this_type_2.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(this_Pokemon['Form'][0]))]['Types'][1])
            self.this_ability_name.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(this_Pokemon['Form'][0]))]['Ability']['Name'])
            #print(self.this_ability_name)
            for moves in self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(this_Pokemon['Form'][0]))]['Moves']:
                self.this_move_name.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(this_Pokemon['Form'][0]))]['Moves'][moves]['Name'][0])

            #Defines Top_left_right Sizer
            self.Top_left_right_sizer = wx.BoxSizer(wx.HORIZONTAL)

            #Defines Top_left Sizer
            self.Top_left_sizer = wx.BoxSizer(wx.VERTICAL)

            #Defines Top_right Sizer
            self.Top_right_sizer = wx.BoxSizer(wx.VERTICAL)
            
            #Sets up a Static Text to be used as empty space and adds it to Top_left Sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.Top_left_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            
            #Sets up a Static Text to be used as empty space and adds it to Top_left Sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.Top_left_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            
            #Sets up Box for Name
            self.Name_box = wx.TextCtrl(self, value=this_Pokemon['Name'][0], style=wx.TE_READONLY)
            self.add_widgets('Name:', self.Name_box, self.Top_left_sizer)
            #self.add_widgets('Name:', self.Name_box, self.main_sizer)
            
            #Creates bitmap image box, starting empty
            self.png = wx.StaticBitmap(self, -1)
            #Adds bitmap Image box to Top_right Sizer
            self.Top_right_sizer.Add(self.png, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            #Loads pokemon entry's image file into bitmap image box
            #Loads selected image
            img = wx.Image(Pokemon_Team_Builder_Frame.dex_path+this_Pokemon['Image'][0], wx.BITMAP_TYPE_ANY)
            #Finally sets image into bitmap image box, if there was an image to begin with
            try:
                self.png.SetBitmap(wx.Bitmap(img))
            except:
                pass

            #Sets up Boxes for Species and Form 
            #Sets up Combo Box for Species
            self.Species_box = wx.TextCtrl(self, value=this_Pokemon['Species'][0], style=wx.TE_READONLY)
            #self.Species_box = wx.ComboBox(self, size=(95, -1), choices=self.dex_list_comp, style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
            #Binds event to selecting a pokemon in the Species Combo Box OnSelectForm
            #self.Species_box.Bind(wx.EVT_COMBOBOX, self.OnSelectSpecies)
            
            #self.add_widgets('Species:', self.Species_box)        
            #Sets up Box for Form
            self.Form_box = wx.ComboBox(self, size=(95, -1), value=this_Pokemon['Form'][0], choices=self.this_form, style=wx.CB_DROPDOWN | wx.CB_READONLY)
            #Binds event to selecting a form in the Form Combo Box
            self.Form_box.Bind(wx.EVT_COMBOBOX, self.OnSelectForm)
            self.add_widgets('Species:', self.Species_box, self.Top_left_sizer)
            self.add_widgets('Form:', self.Form_box, self.Top_left_sizer)            
            #self.add_widgets_4_boxes_horizontal('Species:', self.Species_box, 'Form:', self.Form_box, self.main_sizer)

            #Sets up Combo Boxes for Types
            #Sets up Combo Box for Type_1
            self.Type_1_box = wx.TextCtrl(self, value=this_Pokemon['Types'][0], style=wx.TE_READONLY)
            background_type(self.Type_1_box, this_Pokemon['Types'][0])
            #self.add_widgets('Type 1:', self.Type_1_box)        
            #Sets up Combo Box for Type_2
            self.Type_2_box = wx.TextCtrl(self, value=this_Pokemon['Types'][1], style=wx.TE_READONLY)
            background_type(self.Type_2_box, this_Pokemon['Types'][1])
            #self.add_widgets('Type 2:', self.Type_2_box)        
            self.add_widgets_4_boxes_horizontal('Type 1:', self.Type_1_box, 'Type 2:', self.Type_2_box, self.Top_left_sizer)
            #self.add_widgets_4_boxes_horizontal('Type 1:', self.Type_1_box, 'Type 2:', self.Type_2_box, self.main_sizer)

            #Sets up Box for Item
            self.Item_box = wx.TextCtrl(self, value=this_Pokemon['Item'][0])
            self.add_widgets('Item:', self.Item_box, self.Top_left_sizer)
            #self.add_widgets('Item:', self.Item_box, self.main_sizer)
            
            #Sets up Combo Box for Image
            self.Image_box = wx.ComboBox(self, size=(95, -1),value=this_Pokemon['Image'][0] , choices=self.this_images, style=wx.CB_DROPDOWN | wx.CB_READONLY)
            self.add_widgets('Image:', self.Image_box, self.Top_left_sizer)
            #self.add_widgets('Image:', self.Image_box, self.main_sizer)
            #Binds event to selecting an image name in the Images Combo Box to OnSelectImage
            self.Image_box.Bind(wx.EVT_COMBOBOX, self.OnSelectImage)
            
            #Sets up a Static Text to be used as empty space and adds it to Top_left Sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.Top_left_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            
            #Sets up a Static Text to be used as empty space and adds it to Top_left Sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.Top_left_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Adds Top_left Sizer to Top_left_right Sizer
            self.Top_left_right_sizer.Add(self.Top_left_sizer, 1, wx.EXPAND)
            
            #Sets up a Static Text to be used as empty space and adds it to Top_left_right Sizer
            empty_space_label = wx.StaticText(self, label="    ")
            empty_space_label.SetFont(font_moves)
            self.Top_left_right_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_VERTICAL)
            
            #Adds Top_right Sizer to Top_left_right Sizer
            self.Top_left_right_sizer.Add(self.Top_right_sizer, 1, wx.EXPAND)

            #Adds self.Top_left_right to Main Sizer
            self.main_sizer.Add(self.Top_left_right_sizer, 0, wx.EXPAND)

            #Sets up Boxes for Ability_Name and Ability_Info
            #Sets up a Static Text for Ability and adds it to the main sizer
            Ability_label = wx.StaticText(self, label="Ability")
            Ability_label.SetFont(font_moves)
            self.main_sizer.Add(Ability_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            
            #Sets up Box for Ability_Name
            self.Ability_Name_box = wx.ComboBox(self, size=(95, -1), value=this_Pokemon['Ability']['Ability_Name'][0], choices=self.this_ability_name[0], style=wx.CB_DROPDOWN | wx.CB_READONLY)
            #Binds event to selecting an ability name in the Form Combo Box
            self.Ability_Name_box.Bind(wx.EVT_COMBOBOX, self.OnSelectAbility) 
            #Sets up Box for Ability_Info
            self.Ability_Info_box = wx.TextCtrl(self, value=this_Pokemon['Ability']['Ability_Info'][0], style=wx.TE_READONLY | wx.TE_MULTILINE | wx.HSCROLL)

            #self.add_widgets('Ability Info:', self.Ability_Info_box)
            self.add_widgets_4_boxes_horizontal('Name:', self.Ability_Name_box, 'Info:', self.Ability_Info_box, self.main_sizer)         

            # ============================ #  Moves 1 and 2

            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Defines Moves 1 and 2 Sizer
            self.moves_1_2_sizer = wx.BoxSizer(wx.HORIZONTAL)

            #Defines Move 1 Sizer
            self.move_1_sizer = wx.BoxSizer(wx.VERTICAL)

            #Defines Move 2 Sizer
            self.move_2_sizer = wx.BoxSizer(wx.VERTICAL)

            # ============== #  Move 1

            #Sets up a Static Text for Move 1 and adds it to the move 1 sizer
            Move_1_label = wx.StaticText(self, label="Move 1")
            Move_1_label.SetFont(font_moves)
            self.move_1_sizer.Add(Move_1_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Sets up Boxes for Name and Mode
            #Sets up Box for Move 1 Name
            self.Move_1_Name_box = wx.ComboBox(self, size=(95, -1), value=this_Pokemon['Moves']['Move_Name'][0], choices=self.this_move_name, style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
            #Binds event to selecting a move name in the Form Combo Box
            self.Move_1_Name_box.Bind(wx.EVT_COMBOBOX, self.OnSelectMove_1) 
            
            #Sets up Box for Move 1 Mode
            self.Move_1_Mode_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Mode'][0], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Name:', self.Move_1_Name_box, 'Mode:', self.Move_1_Mode_box, self.move_1_sizer)

            #Sets up Boxes for Type and Method        
            #Sets up Combo Box for Move 1 Type
            self.Move_1_Type_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Type'][0], style=wx.TE_READONLY)
            background_type(self.Move_1_Type_box, this_Pokemon['Moves']['Move_Type'][0])
            #Sets up Static Text for Move 1 Method
            self.Move_1_Method_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Method'][0], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Type:', self.Move_1_Type_box, 'Method:', self.Move_1_Method_box, self.move_1_sizer)

            #Sets up Boxes for Power and Accuracy
            #Sets up Static Text for Move 1 Power
            self.Move_1_Power_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Power'][0], style=wx.TE_READONLY)       
            #Sets up Static Text for Move 1 Accuracy
            self.Move_1_Accuracy_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Accuracy'][0], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Power:', self.Move_1_Power_box, 'Acc.:', self.Move_1_Accuracy_box, self.move_1_sizer)

            #Sets up Boxes for PP and Effect
            #Sets up Static Text for Move 1 PP
            self.Move_1_PP_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_PP'][0], style=wx.TE_READONLY)      
            #Sets up Static Text for Move 1 Effect
            self.Move_1_Effect_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Effect'][0], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('PP:', self.Move_1_PP_box, 'Effect:', self.Move_1_Effect_box, self.move_1_sizer)

            #Sets up Static Text for Move 1 Info
            self.Move_1_Info_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Info'][0], style=wx.TE_READONLY | wx.TE_MULTILINE | wx.HSCROLL)
            self.add_widgets('Info:', self.Move_1_Info_box, self.move_1_sizer)

            # ============== #  Move 2

            #Sets up a Static Text for Move 2 and adds it to the move 2 sizer
            Move_2_label = wx.StaticText(self, label="Move 2")
            Move_2_label.SetFont(font_moves)
            self.move_2_sizer.Add(Move_2_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Sets up Boxes for Name and Mode
            #Sets up Box for Move 2 Name
            self.Move_2_Name_box = wx.ComboBox(self, size=(95, -1), value=this_Pokemon['Moves']['Move_Name'][1], choices=self.this_move_name, style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
            #Binds event to selecting a move name in the Form Combo Box
            self.Move_2_Name_box.Bind(wx.EVT_COMBOBOX, self.OnSelectMove_2) 
            
            #Sets up Box for Move 2 Mode
            self.Move_2_Mode_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Mode'][1], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Name:', self.Move_2_Name_box, 'Mode:', self.Move_2_Mode_box, self.move_2_sizer)

            #Sets up Boxes for Type and Method        
            #Sets up Combo Box for Move 2 Type
            self.Move_2_Type_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Type'][1], style=wx.TE_READONLY)
            background_type(self.Move_2_Type_box, this_Pokemon['Moves']['Move_Type'][1])
            #Sets up Static Text for Move 2 Method
            self.Move_2_Method_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Method'][1], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Type:', self.Move_2_Type_box, 'Method:', self.Move_2_Method_box, self.move_2_sizer)

            #Sets up Boxes for Power and Accuracy
            #Sets up Static Text for Move 2 Power
            self.Move_2_Power_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Power'][1], style=wx.TE_READONLY)       
            #Sets up Static Text for Move 2 Accuracy
            self.Move_2_Accuracy_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Accuracy'][1], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Power:', self.Move_2_Power_box, 'Acc.:', self.Move_2_Accuracy_box, self.move_2_sizer)

            #Sets up Boxes for PP and Effect
            #Sets up Static Text for Move 2 PP
            self.Move_2_PP_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_PP'][1], style=wx.TE_READONLY)      
            #Sets up Static Text for Move 2 Effect
            self.Move_2_Effect_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Effect'][1], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('PP:', self.Move_2_PP_box, 'Effect:', self.Move_2_Effect_box, self.move_2_sizer)

            #Sets up Static Text for Move 2 Info
            self.Move_2_Info_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Info'][1], style=wx.TE_READONLY | wx.TE_MULTILINE | wx.HSCROLL)
            self.add_widgets('Info:', self.Move_2_Info_box, self.move_2_sizer)

            # ============== # 

            #Adds Move 1 Sizer to Moves 1 and 2 Sizer
            self.moves_1_2_sizer.Add(self.move_1_sizer, 1, wx.EXPAND)

            #Sets up a Static Text to be used as empty space and adds it to  Moves 1 and 2 Sizer
            empty_space_label = wx.StaticText(self, label="    ")
            empty_space_label.SetFont(font_moves)
            self.moves_1_2_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_VERTICAL)

            #Adds Move 2 Sizer to Moves 1 and 2 Sizer
            self.moves_1_2_sizer.Add(self.move_2_sizer, 1, wx.EXPAND)

            #Adds Moves 1 and 2 Sizer to Main Sizer
            self.main_sizer.Add(self.moves_1_2_sizer, 0, wx.EXPAND)   
            
            # ============================ #  Moves 3 and 4

            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Defines Moves 3 and 4 Sizer
            self.moves_3_4_sizer = wx.BoxSizer(wx.HORIZONTAL)

            #Defines Move 3 Sizer
            self.move_3_sizer = wx.BoxSizer(wx.VERTICAL)

            #Defines Move 4 Sizer
            self.move_4_sizer = wx.BoxSizer(wx.VERTICAL)

            # ============== #  Move 3

            #Sets up a Static Text for Move 3 and adds it to the move 3 sizer
            Move_3_label = wx.StaticText(self, label="Move 3")
            Move_3_label.SetFont(font_moves)
            self.move_3_sizer.Add(Move_3_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Sets up Boxes for Name and Mode
            #Sets up Box for Move 3 Name
            self.Move_3_Name_box = wx.ComboBox(self, size=(95, -1), value=this_Pokemon['Moves']['Move_Name'][2], choices=self.this_move_name, style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
            #Binds event to selecting a move name in the Form Combo Box
            self.Move_3_Name_box.Bind(wx.EVT_COMBOBOX, self.OnSelectMove_3) 
            
            #Sets up Box for Move 3 Mode
            self.Move_3_Mode_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Mode'][2], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Name:', self.Move_3_Name_box, 'Mode:', self.Move_3_Mode_box, self.move_3_sizer)

            #Sets up Boxes for Type and Method        
            #Sets up Combo Box for Move 3 Type
            self.Move_3_Type_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Type'][2], style=wx.TE_READONLY) 
            background_type(self.Move_3_Type_box, this_Pokemon['Moves']['Move_Type'][2])
            #Sets up Static Text for Move 3 Method
            self.Move_3_Method_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Method'][2], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Type:', self.Move_3_Type_box, 'Method:', self.Move_3_Method_box, self.move_3_sizer)

            #Sets up Boxes for Power and Accuracy
            #Sets up Static Text for Move 3 Power
            self.Move_3_Power_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Power'][2], style=wx.TE_READONLY)       
            #Sets up Static Text for Move 3 Accuracy
            self.Move_3_Accuracy_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Accuracy'][2], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Power:', self.Move_3_Power_box, 'Acc.:', self.Move_3_Accuracy_box, self.move_3_sizer)

            #Sets up Boxes for PP and Effect
            #Sets up Static Text for Move 3 PP
            self.Move_3_PP_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_PP'][2], style=wx.TE_READONLY)      
            #Sets up Static Text for Move 3 Effect
            self.Move_3_Effect_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Effect'][2], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('PP:', self.Move_3_PP_box, 'Effect:', self.Move_3_Effect_box, self.move_3_sizer)

            #Sets up Static Text for Move 3 Info
            self.Move_3_Info_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Info'][2], style=wx.TE_READONLY | wx.TE_MULTILINE | wx.HSCROLL)
            self.add_widgets('Info:', self.Move_3_Info_box, self.move_3_sizer)

            # ============== #  Move 4

            #Sets up a Static Text for Move 4 and adds it to the move 4 sizer
            Move_4_label = wx.StaticText(self, label="Move 4")
            Move_4_label.SetFont(font_moves)
            self.move_4_sizer.Add(Move_4_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Sets up Boxes for Name and Mode
            #Sets up Box for Move 4 Name
            self.Move_4_Name_box = wx.ComboBox(self, size=(95, -1), value=this_Pokemon['Moves']['Move_Name'][3], choices=self.this_move_name, style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
            #Binds event to selecting a move name in the Form Combo Box
            self.Move_4_Name_box.Bind(wx.EVT_COMBOBOX, self.OnSelectMove_4) 
            
            #Sets up Box for Move 4 Mode
            self.Move_4_Mode_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Mode'][3], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Name:', self.Move_4_Name_box, 'Mode:', self.Move_4_Mode_box, self.move_4_sizer)

            #Sets up Boxes for Type and Method        
            #Sets up Combo Box for Move 4 Type
            self.Move_4_Type_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Type'][3], style=wx.TE_READONLY) 
            background_type(self.Move_4_Type_box, this_Pokemon['Moves']['Move_Type'][3])
            #Sets up Static Text for Move 4 Method
            self.Move_4_Method_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Method'][3], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Type:', self.Move_4_Type_box, 'Method:', self.Move_4_Method_box, self.move_4_sizer)

            #Sets up Boxes for Power and Accuracy
            #Sets up Static Text for Move 4 Power
            self.Move_4_Power_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Power'][3], style=wx.TE_READONLY)       
            #Sets up Static Text for Move 4 Accuracy
            self.Move_4_Accuracy_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Accuracy'][3], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Power:', self.Move_4_Power_box, 'Acc.:', self.Move_4_Accuracy_box, self.move_4_sizer)

            #Sets up Boxes for PP and Effect
            #Sets up Static Text for Move 4 PP
            self.Move_4_PP_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_PP'][3], style=wx.TE_READONLY)      
            #Sets up Static Text for Move 4 Effect
            self.Move_4_Effect_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Effect'][3], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('PP:', self.Move_4_PP_box, 'Effect:', self.Move_4_Effect_box, self.move_4_sizer)

            #Sets up Static Text for Move 4 Info
            self.Move_4_Info_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Info'][3], style=wx.TE_READONLY | wx.TE_MULTILINE | wx.HSCROLL)
            self.add_widgets('Info:', self.Move_4_Info_box, self.move_4_sizer)

            # ============== # 

            #Adds Move 3 Sizer to Moves 3 and 4 Sizer
            self.moves_3_4_sizer.Add(self.move_3_sizer, 1, wx.EXPAND)

            #Sets up a Static Text to be used as empty space and adds it to  Moves 3 and 4 Sizer
            empty_space_label = wx.StaticText(self, label="    ")
            empty_space_label.SetFont(font_moves)
            self.moves_3_4_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_VERTICAL)

            #Adds Move 4 Sizer to Moves 3 and 4 Sizer
            self.moves_3_4_sizer.Add(self.move_4_sizer, 1, wx.EXPAND)

            #Adds Moves 3 and 4 Sizer to Main Sizer
            self.main_sizer.Add(self.moves_3_4_sizer, 0, wx.EXPAND)

            # ============== #          
                    
        #Sets up a Static Text to be used as empty space and adds it to the main sizer
        empty_space_label = wx.StaticText(self, label="")
        empty_space_label.SetFont(font_moves)
        self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
        
        #Sets up a Static Text to be used as empty space and adds it to the main sizer
        empty_space_label = wx.StaticText(self, label="")
        empty_space_label.SetFont(font_moves)
        self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
        
        #Defines Button Sizer
        btn_sizer = wx.BoxSizer()
        #Creates Save Button
        edit_btn = wx.Button(self, label='Edit')
        #Binds Button event to Edit Button
        edit_btn.Bind(wx.EVT_BUTTON, self.on_edit_pokemon)
        #Adds Edit Button to Button Sizer
        btn_sizer.Add(edit_btn, 0, wx.ALL, 5)
        #Creates Preview Button
        preview_btn = wx.Button(self, label='Preview')
        #Binds Button event to Preview Button
        preview_btn.Bind(wx.EVT_BUTTON, self.on_preview_pokemon)
        #Adds Preview Button to Button Sizer
        btn_sizer.Add(preview_btn, 0, wx.ALL, 5)          
        #Adds Cancel Button (pre defined) to Button Sizer
        btn_sizer.Add(wx.Button(self, id=wx.ID_CANCEL), 0, wx.ALL, 5)
        #Adds Button Sizer to Main Sizer
        self.main_sizer.Add(btn_sizer, 0, wx.CENTER)
        
        #Sets Main Sizer for user to see
        self.SetSizer(self.main_sizer)
              
     # ====================================== # Organizing layout   
    
    #This function helps position each box (Text Control, Combo Box, Radio Box) to entry pokemon information
    def add_widgets(self, label_text, entry_box, this_sizer):
        #Defines font style for widgets statict text
        font_labels = wx.Font(12, wx.DECORATIVE, wx.BOLD, wx.NORMAL)   
        #Defines Row Sizer for each "Line"
        row_sizer = wx.BoxSizer(wx.HORIZONTAL)
        #Uses Static Text to Label the boxes
        label = wx.StaticText(self, label=label_text, size=(60, -1))
        #Sets font style to label
        label.SetFont(font_labels)        
        #Adds Static Text Label to Row Sizer
        row_sizer.Add(label, 0, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Adds Box to add information to row sizer
        row_sizer.Add(entry_box, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Adds Row Sizer to Provided Sizer
        this_sizer.Add(row_sizer, 0, wx.EXPAND)
        #self.main_sizer.Add(row_sizer, 0, wx.EXPAND)
        
    #This function helps position each box (Text Control, Combo Box, Radio Box) to entry pokemon information
    def add_widgets_4_boxes_horizontal(self, text_1, box_1, text_2, box_2, this_sizer):
        #Defines font style for widgets statict text
        font_labels = wx.Font(12, wx.DECORATIVE, wx.BOLD, wx.NORMAL)
        #Uses Static Text to Label using text_1
        label_1 = wx.StaticText(self, label=text_1, size=(60, -1))
        #Sets font style to text_1
        label_1.SetFont(font_labels) 
        #Uses Static Text to Label using text_2
        label_2 = wx.StaticText(self, label=text_2, size=(60, -1))
        #Sets font style to label
        label_2.SetFont(font_labels) 
        #Defines Row Sizer for each "Line"
        row_sizer = wx.BoxSizer(wx.HORIZONTAL)        
        #Adds text_1 to Row Sizer
        row_sizer.Add(label_1, 0, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)        
        #Adds box_1 to Row Sizer
        row_sizer.Add(box_1, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Sets up and adds empty space
        empty_space_label = wx.StaticText(self, label="    ")
        row_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_VERTICAL)
        #Adds text_2 to Row Sizer
        row_sizer.Add(label_2, 0, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5) 
        #Adds box_2 to row sizer
        row_sizer.Add(box_2, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Adds Row Sizer to Provided Sizer
        this_sizer.Add(row_sizer, 0, wx.EXPAND)
        #self.main_sizer.Add(row_sizer, 0, wx.EXPAND)
        
    # ====================================== # When choices are made from a combo box (Using Dex ONLY) 

    def OnSelectImage(self, event): 
        #Loads selected image file into bitmap image box
        #Finds size of Top_right_sizer
        W, H = self.Top_right_sizer.Size
        #Loads selected image
        img = wx.Image(Pokemon_Team_Builder_Frame.dex_path+self.Image_box.GetValue(), wx.BITMAP_TYPE_ANY)
        #Gets size of selected image after transforming into bitmap
        #W_img, H_img = wx.Bitmap(Pokemon_Team_Builder_Frame.dex_path+self.Image_box.GetValue(), wx.BITMAP_TYPE_ANY).Size
        #Resizes image according to  Top_right_sizer size
        #NewH = int(H)
        #NewW = int(W_img * H / H_img)
        #Changes size of the image to fit  Top_right_sizer
        #img = img.Scale(NewW,NewH)
        #Places image inside sizer without resizing it, but with cutting/adding edges (making transparent) if they are too long/short
        img = img.Size([int(W),int(H)], pos=[0,0], red=-1, green=-1, blue=-1) #red=-1, green=-1, blue=-1 makes edges transparent
        #Finally sets image into bitmap image box
        self.png.SetBitmap(wx.Bitmap(img))
        #self.png.SetBitmap(wx.Bitmap(Pokemon_Team_Builder_Frame.dex_path+self.Image_box.GetValue(), wx.BITMAP_TYPE_ANY))
   
    def OnSelectForm(self, event):
        #Resets relevant choice lists
        self.this_type_1 = []
        self.this_type_2 = []
        self.this_ability_name = []
        self.this_move_name = []
        #Updates relevant choice lists
        self.this_type_1.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Types'][0]) 
        self.this_type_2.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Types'][1])
        self.this_ability_name.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Ability']['Name'])
        #print(self.this_ability_name)
        for moves in self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']:
            self.this_move_name.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves'][moves]['Name'][0])
        #print(self.this_move_name)
        
        #Updates Relevant Boxes
        self.Type_1_box.Clear()
        self.Type_1_box.SetValue(self.this_type_1[0])
        #Calls function to color background according to type from pokemon_functions.py
        background_type(self.Type_1_box, str(self.this_type_1[0]))
        self.Type_2_box.Clear()
        self.Type_2_box.SetValue(self.this_type_2[0])
        #Calls function to color background according to type from pokemon_functions.py
        background_type(self.Type_2_box, str(self.this_type_2[0]))
        #Ability Name Box
        self.Ability_Name_box.Clear()
        self.Ability_Name_box.Append(self.this_ability_name[0])
        #Move Name Boxes
        self.Move_1_Name_box.Clear()
        self.Move_1_Name_box.Append(self.this_move_name)        
        self.Move_2_Name_box.Clear()
        self.Move_2_Name_box.Append(self.this_move_name)        
        self.Move_3_Name_box.Clear()
        self.Move_3_Name_box.Append(self.this_move_name)        
        self.Move_4_Name_box.Clear()
        self.Move_4_Name_box.Append(self.this_move_name)        
        #Clears all other relevant boxes
        self.Ability_Info_box.Clear()
        self.Move_1_Type_box.Clear()
        background_type(self.Move_1_Type_box, '')
        self.Move_1_Mode_box.Clear()
        self.Move_1_Method_box.Clear()
        self.Move_1_Power_box.Clear()
        self.Move_1_Accuracy_box.Clear()
        self.Move_1_Effect_box.Clear()
        self.Move_1_PP_box.Clear()
        self.Move_1_Info_box.Clear()
        self.Move_2_Type_box.Clear()
        background_type(self.Move_2_Type_box, '')
        self.Move_2_Mode_box.Clear()
        self.Move_2_Method_box.Clear()
        self.Move_2_Power_box.Clear()
        self.Move_2_Accuracy_box.Clear()
        self.Move_2_Effect_box.Clear()
        self.Move_2_PP_box.Clear()
        self.Move_2_Info_box.Clear()
        self.Move_3_Type_box.Clear()
        background_type(self.Move_3_Type_box, '')
        self.Move_3_Mode_box.Clear()
        self.Move_3_Method_box.Clear()
        self.Move_3_Power_box.Clear()
        self.Move_3_Accuracy_box.Clear()
        self.Move_3_Effect_box.Clear()
        self.Move_3_PP_box.Clear()
        self.Move_3_Info_box.Clear()
        self.Move_4_Type_box.Clear()
        background_type(self.Move_4_Type_box, '')
        self.Move_4_Mode_box.Clear()
        self.Move_4_Method_box.Clear()
        self.Move_4_Power_box.Clear()
        self.Move_4_Accuracy_box.Clear()
        self.Move_4_Effect_box.Clear()
        self.Move_4_PP_box.Clear()
        self.Move_4_Info_box.Clear()
        
    def OnSelectAbility(self, event):
        #Resets relevant choice lists
        self.this_ability_info = []
        #Updates relevant choice lists
        self.this_ability_info.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Ability']['Info'])
        #Ability Info Box
        self.Ability_Info_box.Clear()
        self.Ability_Info_box.SetValue(self.this_ability_info[0][self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Ability']['Name'].index(str(self.Ability_Name_box.GetValue()))])
        
        
    def OnSelectMove_1(self, event):
        #Resets relevant choice lists
        self.this_move_1_type = []
        self.this_move_1_mode = []
        self.this_move_1_method = []
        self.this_move_1_power = []
        self.this_move_1_accuracy = []
        self.this_move_1_PP = []
        self.this_move_1_effect = []
        self.this_move_1_info = []
        #Updates relevant choice lists
        self.this_move_1_type.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_1_Name_box.GetValue())))]['Type'][0])
        self.this_move_1_mode.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_1_Name_box.GetValue())))]['Mode'][0])
        self.this_move_1_method.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_1_Name_box.GetValue())))]['Method'][0])
        self.this_move_1_power.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_1_Name_box.GetValue())))]['Power'][0])
        self.this_move_1_accuracy.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_1_Name_box.GetValue())))]['Accuracy'][0])
        self.this_move_1_PP.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_1_Name_box.GetValue())))]['PP'][0])
        self.this_move_1_effect.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_1_Name_box.GetValue())))]['Effect'][0])
        self.this_move_1_info.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_1_Name_box.GetValue())))]['Info'][0])
        #Updates Boxes
        self.Move_1_Type_box.SetValue(self.this_move_1_type[0])
        background_type(self.Move_1_Type_box, str(self.this_move_1_type[0]))
        self.Move_1_Mode_box.SetValue(self.this_move_1_mode[0])
        self.Move_1_Method_box.SetValue(self.this_move_1_method[0])
        self.Move_1_Power_box.SetValue(self.this_move_1_power[0])
        self.Move_1_Accuracy_box.SetValue(self.this_move_1_accuracy[0])
        self.Move_1_PP_box.SetValue(self.this_move_1_PP[0])
        self.Move_1_Effect_box.SetValue(self.this_move_1_effect[0])
        self.Move_1_Info_box.SetValue(self.this_move_1_info[0])
        
        
    def OnSelectMove_2(self, event):
        #Resets relevant choice lists
        self.this_move_2_type = []
        self.this_move_2_mode = []
        self.this_move_2_method = []
        self.this_move_2_power = []
        self.this_move_2_accuracy = []
        self.this_move_2_PP = []
        self.this_move_2_effect = []
        self.this_move_2_info = []
        #Updates relevant choice lists
        self.this_move_2_type.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_2_Name_box.GetValue())))]['Type'][0])
        self.this_move_2_mode.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_2_Name_box.GetValue())))]['Mode'][0])
        self.this_move_2_method.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_2_Name_box.GetValue())))]['Method'][0])
        self.this_move_2_power.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_2_Name_box.GetValue())))]['Power'][0])
        self.this_move_2_accuracy.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_2_Name_box.GetValue())))]['Accuracy'][0])
        self.this_move_2_PP.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_2_Name_box.GetValue())))]['PP'][0])
        self.this_move_2_effect.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_2_Name_box.GetValue())))]['Effect'][0])
        self.this_move_2_info.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_2_Name_box.GetValue())))]['Info'][0])
        #Updates Boxes
        self.Move_2_Type_box.SetValue(self.this_move_2_type[0])
        background_type(self.Move_2_Type_box, str(self.this_move_2_type[0]))
        self.Move_2_Mode_box.SetValue(self.this_move_2_mode[0])
        self.Move_2_Method_box.SetValue(self.this_move_2_method[0])
        self.Move_2_Power_box.SetValue(self.this_move_2_power[0])
        self.Move_2_Accuracy_box.SetValue(self.this_move_2_accuracy[0])
        self.Move_2_PP_box.SetValue(self.this_move_2_PP[0])
        self.Move_2_Effect_box.SetValue(self.this_move_2_effect[0])
        self.Move_2_Info_box.SetValue(self.this_move_2_info[0])
        
    def OnSelectMove_3(self, event):
        #Resets relevant choice lists
        self.this_move_3_type = []
        self.this_move_3_mode = []
        self.this_move_3_method = []
        self.this_move_3_power = []
        self.this_move_3_accuracy = []
        self.this_move_3_PP = []
        self.this_move_3_effect = []
        self.this_move_3_info = []
        #Updates relevant choice lists
        self.this_move_3_type.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_3_Name_box.GetValue())))]['Type'][0])
        self.this_move_3_mode.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_3_Name_box.GetValue())))]['Mode'][0])
        self.this_move_3_method.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_3_Name_box.GetValue())))]['Method'][0])
        self.this_move_3_power.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_3_Name_box.GetValue())))]['Power'][0])
        self.this_move_3_accuracy.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_3_Name_box.GetValue())))]['Accuracy'][0])
        self.this_move_3_PP.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_3_Name_box.GetValue())))]['PP'][0])
        self.this_move_3_effect.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_3_Name_box.GetValue())))]['Effect'][0])
        self.this_move_3_info.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_3_Name_box.GetValue())))]['Info'][0])
        #Updates Boxes
        self.Move_3_Type_box.SetValue(self.this_move_3_type[0])
        background_type(self.Move_3_Type_box, str(self.this_move_3_type[0]))
        self.Move_3_Mode_box.SetValue(self.this_move_3_mode[0])
        self.Move_3_Method_box.SetValue(self.this_move_3_method[0])
        self.Move_3_Power_box.SetValue(self.this_move_3_power[0])
        self.Move_3_Accuracy_box.SetValue(self.this_move_3_accuracy[0])
        self.Move_3_PP_box.SetValue(self.this_move_3_PP[0])
        self.Move_3_Effect_box.SetValue(self.this_move_3_effect[0])
        self.Move_3_Info_box.SetValue(self.this_move_3_info[0])
        
    def OnSelectMove_4(self, event):
        #Resets relevant choice lists
        self.this_move_4_type = []
        self.this_move_4_mode = []
        self.this_move_4_method = []
        self.this_move_4_power = []
        self.this_move_4_accuracy = []
        self.this_move_4_PP = []
        self.this_move_4_effect = []
        self.this_move_4_info = []
        #Updates relevant choice lists
        self.this_move_4_type.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_4_Name_box.GetValue())))]['Type'][0])
        self.this_move_4_mode.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_4_Name_box.GetValue())))]['Mode'][0])
        self.this_move_4_method.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_4_Name_box.GetValue())))]['Method'][0])
        self.this_move_4_power.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_4_Name_box.GetValue())))]['Power'][0])
        self.this_move_4_accuracy.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_4_Name_box.GetValue())))]['Accuracy'][0])
        self.this_move_4_PP.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_4_Name_box.GetValue())))]['PP'][0])
        self.this_move_4_effect.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_4_Name_box.GetValue())))]['Effect'][0])
        self.this_move_4_info.append(self.this_dex_pokemon['Forms']['Form'+str(1+self.this_form.index(str(self.Form_box.GetValue())))]['Moves']['Move'+str(1+self.this_move_name.index(str(self.Move_4_Name_box.GetValue())))]['Info'][0])
        #Updates Boxes
        self.Move_4_Type_box.SetValue(self.this_move_4_type[0])
        background_type(self.Move_4_Type_box, str(self.this_move_4_type[0]))
        self.Move_4_Mode_box.SetValue(self.this_move_4_mode[0])
        self.Move_4_Method_box.SetValue(self.this_move_4_method[0])
        self.Move_4_Power_box.SetValue(self.this_move_4_power[0])
        self.Move_4_Accuracy_box.SetValue(self.this_move_4_accuracy[0])
        self.Move_4_PP_box.SetValue(self.this_move_4_PP[0])
        self.Move_4_Effect_box.SetValue(self.this_move_4_effect[0])
        self.Move_4_Info_box.SetValue(self.this_move_4_info[0])
        
        
    # ====================================== # When edit button is pressed
        
    #This function is called when the edit button is pressed. It gets the data from the entry boxes and saves it into a .poke file
    def on_edit_pokemon(self, event):
        
        #For initial pokemon info
        self.Name = self.Name_box.GetValue()
        self.Species = self.Species_box.GetValue()
        self.Form = self.Form_box.GetValue()
        self.Image = self.Image_box.GetValue()
        self.Type_1 = self.Type_1_box.GetValue()
        self.Type_2 = self.Type_2_box.GetValue()
        self.Ability_name = self.Ability_Name_box.GetValue()
        self.Ability_info = self.Ability_Info_box.GetValue()
        self.Item = self.Item_box.GetValue()
        
        #When Dex was loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) >1:
            #Copies image file from dex folder to database folder
            shutil.copyfile(Pokemon_Team_Builder_Frame.dex_path+self.Image_box.GetValue(), Pokemon_Team_Builder_Frame.data_path+self.Image_box.GetValue())
        
        #For pokemon move info
        self.Move_1 = self.Move_1_Name_box.GetValue()
        self.Move_type_1 = self.Move_1_Type_box.GetValue()
        #When Dex was not loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) <1:
            self.Move_mode_1 = self.Move_1_Mode_box.GetItemLabel(self.Move_1_Mode_box.GetSelection())
        #When Dex was loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) >1:
            self.Move_mode_1 = self.Move_1_Mode_box.GetValue()
        
        self.Move_info_1 = self.Move_1_Info_box.GetValue()
        self.Move_method_1 = self.Move_1_Method_box.GetValue()
        self.Move_power_1 = self.Move_1_Power_box.GetValue()
        self.Move_accuracy_1 = self.Move_1_Accuracy_box.GetValue()
        self.Move_PP_1 = self.Move_1_PP_box.GetValue()
        self.Move_Effect_1 = self.Move_1_Effect_box.GetValue()
        
        self.Move_2 = self.Move_2_Name_box.GetValue()
        self.Move_type_2 = self.Move_2_Type_box.GetValue()
        #When Dex was not loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) <1:
            self.Move_mode_2 = self.Move_2_Mode_box.GetItemLabel(self.Move_2_Mode_box.GetSelection())
        #When Dex was loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) >1:
            self.Move_mode_2 = self.Move_2_Mode_box.GetValue()
        
        self.Move_info_2 = self.Move_2_Info_box.GetValue()
        self.Move_method_2 = self.Move_2_Method_box.GetValue()
        self.Move_power_2 = self.Move_2_Power_box.GetValue()
        self.Move_accuracy_2 = self.Move_2_Accuracy_box.GetValue()
        self.Move_PP_2 = self.Move_2_PP_box.GetValue()
        self.Move_Effect_2 = self.Move_2_Effect_box.GetValue()
        
        self.Move_3 = self.Move_3_Name_box.GetValue()
        self.Move_type_3 = self.Move_3_Type_box.GetValue()
        #When Dex was not loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) <1:
            self.Move_mode_3 = self.Move_3_Mode_box.GetItemLabel(self.Move_3_Mode_box.GetSelection())
        #When Dex was loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) >1:
            self.Move_mode_3 = self.Move_3_Mode_box.GetValue()
        
        self.Move_info_3 = self.Move_3_Info_box.GetValue()
        self.Move_method_3 = self.Move_3_Method_box.GetValue()
        self.Move_power_3 = self.Move_3_Power_box.GetValue()
        self.Move_accuracy_3 = self.Move_3_Accuracy_box.GetValue()
        self.Move_PP_3 = self.Move_3_PP_box.GetValue()
        self.Move_Effect_3 = self.Move_3_Effect_box.GetValue()
        
        self.Move_4 = self.Move_4_Name_box.GetValue()
        self.Move_type_4 = self.Move_4_Type_box.GetValue()
        #When Dex was not loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) <1:
            self.Move_mode_4 = self.Move_4_Mode_box.GetItemLabel(self.Move_4_Mode_box.GetSelection())
        #When Dex was loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) >1:
            self.Move_mode_4 = self.Move_4_Mode_box.GetValue()
        
        self.Move_info_4 = self.Move_4_Info_box.GetValue()
        self.Move_method_4 = self.Move_4_Method_box.GetValue()
        self.Move_power_4 = self.Move_4_Power_box.GetValue()
        self.Move_accuracy_4 = self.Move_4_Accuracy_box.GetValue()
        self.Move_PP_4 = self.Move_4_PP_box.GetValue()
        self.Move_Effect_4 = self.Move_4_Effect_box.GetValue()
        
        #If any of the variables is empty, except for Pokemon Type 2, the program tells the user to fill up the boxes
        if self.Name == '' or self.Species == '' or self.Form == '' or self.Type_1 == '' or self.Ability_name == '' or self.Ability_info == '' or self.Item == '' or self.Move_1 == '' or self.Move_type_1 == '' or self.Move_mode_1 == '' or self.Move_info_1 == '' or self.Move_method_1 == '' or self.Move_power_1 == '' or self.Move_accuracy_1 == '' or self.Move_PP_1 == '' or self.Move_Effect_1 == '' or self.Move_2 == '' or self.Move_type_2 == '' or self.Move_mode_2 == '' or self.Move_info_2 == '' or self.Move_method_2 == '' or self.Move_power_2 == '' or self.Move_accuracy_2 == '' or self.Move_PP_2 == '' or self.Move_Effect_2 == '' or self.Move_3 == '' or self.Move_type_3 == '' or self.Move_mode_3 == '' or self.Move_info_3 == '' or self.Move_method_3 == '' or self.Move_power_3 == '' or self.Move_accuracy_3 == '' or self.Move_PP_3 == '' or self.Move_Effect_3 == '' or self.Move_4 == '' or self.Move_type_4 == '' or self.Move_mode_4 == '' or self.Move_info_4 == '' or self.Move_method_4 == '' or self.Move_power_4 == '' or self.Move_accuracy_4 == '' or self.Move_PP_4 == '' or self.Move_Effect_4 == '':
            #Displays a default error message dialog telling the user to fill up the fields/boxes
            #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
            dlg = wx.MessageDialog(self, caption='Error!', message = 'One or more fields are emtpy! Please fill up all the fields with Pokemon information.\nPokemon type 2 should be left empty in case the pokemon has only one type.', style= wx.OK | wx.ICON_ERROR)
            dlg.ShowModal()
            #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
            #Closes the message dialog box    
            dlg.Destroy()
        
        #when all fields/boxes (not counting Pokemon type 2) are filled, the program proceeds normally
        else:        
            #When types 1 and 2 are equal, provides error message and allows user to select types again:
            if self.Type_1 == self.Type_2:
                #Calls a default Message Dialog
                #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
                dlg = wx.MessageDialog(self, caption='Error!', message = 'Pokemon types 1 and 2 are equal! Please change one of the two types.', style= wx.OK | wx.ICON_ERROR)
                dlg.ShowModal()
                #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
                #Closes the message dialog box    
                dlg.Destroy()

            #When types 1 and 2 are not equal, the program proceeds to create Pokemon Entry
            else:  
                #When the move names are equal, provides error message and allows user to select move names again:
                if self.Move_1 == self.Move_2 or self.Move_1 == self.Move_3 or self.Move_1 == self.Move_4 or self.Move_2 == self.Move_3 or self.Move_2 == self.Move_4 or self.Move_3 == self.Move_4:
                    #Calls a default Message Dialog
                    #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
                    dlg = wx.MessageDialog(self, caption='Error!', message = 'One or more Pokemon move names are equal! Please make all move names unique.', style= wx.OK | wx.ICON_ERROR)
                    dlg.ShowModal()
                    #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
                    #Closes the message dialog box    
                    dlg.Destroy()
                    
                #When move names are not equal, the program proceeds to preview Pokemon Entry    
                else:                 
                    #Calls a default attention Message Dialog
                    #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
                    dlg_attention = wx.MessageDialog(self, caption='Attention!', message = 'Are you sure you would like to permanently edit the pokemon entry?', style= wx.OK | wx.CANCEL | wx.ICON_WARNING)
                    #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown
                    #If the user presses the attention dialog box’s OK button, the program can get the user’s path choice via the dialog’s .GetPath() function
                    if dlg_attention.ShowModal() == wx.ID_OK:
                        #Edits pokemon entry in the specified database
                        edit_pokemon(Pokemon_Team_Builder_Frame.data_path,self.Species,self.Form,self.Image,self.Name,self.Type_1,self.Type_2,self.Ability_name,self.Ability_info,self.Item,self.Move_1,self.Move_type_1,self.Move_mode_1,self.Move_info_1,self.Move_method_1,self.Move_power_1,self.Move_accuracy_1,self.Move_PP_1,self.Move_Effect_1,self.Move_2,self.Move_type_2,self.Move_mode_2,self.Move_info_2,self.Move_method_2,self.Move_power_2,self.Move_accuracy_2,self.Move_PP_2,self.Move_Effect_2,self.Move_3,self.Move_type_3,self.Move_mode_3,self.Move_info_3,self.Move_method_3,self.Move_power_3,self.Move_accuracy_3,self.Move_PP_3,self.Move_Effect_3,self.Move_4,self.Move_type_4,self.Move_mode_4,self.Move_info_4,self.Move_method_4,self.Move_power_4,self.Move_accuracy_4,self.Move_PP_4,self.Move_Effect_4)
                        #Displays a default message to say that the Entry was Successfully Edited in the Database
                        #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
                        dlg = wx.MessageDialog(self, caption='Success!', message = 'Pokemon entry was successfully edited in the database!', style= wx.OK | wx.ICON_INFORMATION)
                        dlg.ShowModal()
                        #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
                        #Closes the message dialog box    
                        dlg.Destroy()
                        #Closes the attention message dialog box    
                        dlg_attention.Destroy()
                        #Closes EditPokemonDialog Box
                        self.Destroy()
                    #When the user doesn't want to overwrite data, the message box closes and the use retuns to the EditPokemon Dialog Box
                    else:
                        #Closes the attention message dialog box    
                        dlg_attention.Destroy()

    # ====================================== # When preview button is pressed
        
    #This function is called when the preview button is pressed. It gets the data from the entry boxes and assembles a pokemon dictionary for previewing the pokemon entry
    #It does not create a .poke file
    def on_preview_pokemon(self, event):
        
        self.Name = self.Name_box.GetValue()
        self.Species = self.Species_box.GetValue()
        self.Form = self.Form_box.GetValue()
        self.Image = self.Image_box.GetValue()
        self.Type_1 = self.Type_1_box.GetValue()
        self.Type_2 = self.Type_2_box.GetValue()
        self.Ability_name = self.Ability_Name_box.GetValue()
        self.Ability_info = self.Ability_Info_box.GetValue()
        self.Item = self.Item_box.GetValue()
        
        self.Move_1 = self.Move_1_Name_box.GetValue()
        self.Move_type_1 = self.Move_1_Type_box.GetValue()
        #When Dex was not loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) <1:
            self.Move_mode_1 = self.Move_1_Mode_box.GetItemLabel(self.Move_1_Mode_box.GetSelection())
        #When Dex was loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) >1:
            self.Move_mode_1 = self.Move_1_Mode_box.GetValue()
        
        self.Move_info_1 = self.Move_1_Info_box.GetValue()
        self.Move_method_1 = self.Move_1_Method_box.GetValue()
        self.Move_power_1 = self.Move_1_Power_box.GetValue()
        self.Move_accuracy_1 = self.Move_1_Accuracy_box.GetValue()
        self.Move_PP_1 = self.Move_1_PP_box.GetValue()
        self.Move_Effect_1 = self.Move_1_Effect_box.GetValue()
        
        self.Move_2 = self.Move_2_Name_box.GetValue()
        self.Move_type_2 = self.Move_2_Type_box.GetValue()
        #When Dex was not loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) <1:
            self.Move_mode_2 = self.Move_2_Mode_box.GetItemLabel(self.Move_2_Mode_box.GetSelection())
        #When Dex was loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) >1:
            self.Move_mode_2 = self.Move_2_Mode_box.GetValue()
        
        self.Move_info_2 = self.Move_2_Info_box.GetValue()
        self.Move_method_2 = self.Move_2_Method_box.GetValue()
        self.Move_power_2 = self.Move_2_Power_box.GetValue()
        self.Move_accuracy_2 = self.Move_2_Accuracy_box.GetValue()
        self.Move_PP_2 = self.Move_2_PP_box.GetValue()
        self.Move_Effect_2 = self.Move_2_Effect_box.GetValue()
        
        self.Move_3 = self.Move_3_Name_box.GetValue()
        self.Move_type_3 = self.Move_3_Type_box.GetValue()
        #When Dex was not loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) <1:
            self.Move_mode_3 = self.Move_3_Mode_box.GetItemLabel(self.Move_3_Mode_box.GetSelection())
        #When Dex was loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) >1:
            self.Move_mode_3 = self.Move_3_Mode_box.GetValue()
        
        self.Move_info_3 = self.Move_3_Info_box.GetValue()
        self.Move_method_3 = self.Move_3_Method_box.GetValue()
        self.Move_power_3 = self.Move_3_Power_box.GetValue()
        self.Move_accuracy_3 = self.Move_3_Accuracy_box.GetValue()
        self.Move_PP_3 = self.Move_3_PP_box.GetValue()
        self.Move_Effect_3 = self.Move_3_Effect_box.GetValue()
        
        self.Move_4 = self.Move_4_Name_box.GetValue()
        self.Move_type_4 = self.Move_4_Type_box.GetValue()
        #When Dex was not loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) <1:
            self.Move_mode_4 = self.Move_4_Mode_box.GetItemLabel(self.Move_4_Mode_box.GetSelection())
        #When Dex was loaded
        if len(Pokemon_Team_Builder_Frame.dex_list) >1:
            self.Move_mode_4 = self.Move_4_Mode_box.GetValue()
        
        self.Move_info_4 = self.Move_4_Info_box.GetValue()
        self.Move_method_4 = self.Move_4_Method_box.GetValue()
        self.Move_power_4 = self.Move_4_Power_box.GetValue()
        self.Move_accuracy_4 = self.Move_4_Accuracy_box.GetValue()
        self.Move_PP_4 = self.Move_4_PP_box.GetValue()
        self.Move_Effect_4 = self.Move_4_Effect_box.GetValue()
        
        #If any of the variables is empty, except for Pokemon Type 2, the program tells the user to fill up the boxes
        if self.Name == '' or self.Species == '' or self.Form == '' or self.Type_1 == '' or self.Ability_name == '' or self.Ability_info == '' or self.Item == '' or self.Move_1 == '' or self.Move_type_1 == '' or self.Move_mode_1 == '' or self.Move_info_1 == '' or self.Move_method_1 == '' or self.Move_power_1 == '' or self.Move_accuracy_1 == '' or self.Move_PP_1 == '' or self.Move_Effect_1 == '' or self.Move_2 == '' or self.Move_type_2 == '' or self.Move_mode_2 == '' or self.Move_info_2 == '' or self.Move_method_2 == '' or self.Move_power_2 == '' or self.Move_accuracy_2 == '' or self.Move_PP_2 == '' or self.Move_Effect_2 == '' or self.Move_3 == '' or self.Move_type_3 == '' or self.Move_mode_3 == '' or self.Move_info_3 == '' or self.Move_method_3 == '' or self.Move_power_3 == '' or self.Move_accuracy_3 == '' or self.Move_PP_3 == '' or self.Move_Effect_3 == '' or self.Move_4 == '' or self.Move_type_4 == '' or self.Move_mode_4 == '' or self.Move_info_4 == '' or self.Move_method_4 == '' or self.Move_power_4 == '' or self.Move_accuracy_4 == '' or self.Move_PP_4 == '' or self.Move_Effect_4 == '':
            #Displays a default error message dialog telling the user to fill up the fields/boxes
            #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
            dlg = wx.MessageDialog(self, caption='Error!', message = 'One or more fields are emtpy! Please fill up all the fields with Pokemon information.\nPokemon type 2 should be left empty in case the pokemon has only one type.', style= wx.OK | wx.ICON_ERROR)
            dlg.ShowModal()
            #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
            #Closes the message dialog box    
            dlg.Destroy()
        
        #when all fields/boxes (not counting Pokemon type 2) are filled, the program proceeds normally
        else:        
            #When types 1 and 2 are equal, provides error message and allows user to select types again:
            if self.Type_1 == self.Type_2:
                #Calls a default Message Dialog
                #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
                dlg = wx.MessageDialog(self, caption='Error!', message = 'Pokemon types 1 and 2 are equal! Please change one of the two types.', style= wx.OK | wx.ICON_ERROR)
                dlg.ShowModal()
                #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
                #Closes the message dialog box    
                dlg.Destroy()

            #When types 1 and 2 are not equal, the program proceeds to preview Pokemon Entry
            else:
                #When the move names are equal, provides error message and allows user to select move names again:
                if self.Move_1 == self.Move_2 or self.Move_1 == self.Move_3 or self.Move_1 == self.Move_4 or self.Move_2 == self.Move_3 or self.Move_2 == self.Move_4 or self.Move_3 == self.Move_4:
                    #Calls a default Message Dialog
                    #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
                    dlg = wx.MessageDialog(self, caption='Error!', message = 'One or more Pokemon move names are equal! Please make all move names unique.', style= wx.OK | wx.ICON_ERROR)
                    dlg.ShowModal()
                    #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
                    #Closes the message dialog box    
                    dlg.Destroy()
                    
                #When move names are not equal, the program proceeds to preview Pokemon Entry    
                else:                    
                    #When the pokemon entry already was created, provide the ok or cancel option in an attention dialog
                    if self.Name in Pokemon_Team_Builder_Frame.pokemon_available:
                        #Calls a default attention Message Dialog
                        #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
                        dlg_attention = wx.MessageDialog(self, caption='Attention!', message = 'A Pokemon Entry with this name already exists! Would you still like to preview it?', style= wx.OK | wx.CANCEL | wx.ICON_WARNING)
                        #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown
                        #If the user presses the attention dialog box’s OK button, the program can get the user’s path choice via the dialog’s .GetPath() function
                        if dlg_attention.ShowModal() == wx.ID_OK:
                            #Uses function to assemble pokemon entry into local dictionary
                            self.this_pokemon_preview = preview_pokemon(self.Species,self.Form,self.Image,self.Name,self.Type_1,self.Type_2,self.Ability_name,self.Ability_info,self.Item,self.Move_1,self.Move_type_1,self.Move_mode_1,self.Move_info_1,self.Move_method_1,self.Move_power_1,self.Move_accuracy_1,self.Move_PP_1,self.Move_Effect_1,self.Move_2,self.Move_type_2,self.Move_mode_2,self.Move_info_2,self.Move_method_2,self.Move_power_2,self.Move_accuracy_2,self.Move_PP_2,self.Move_Effect_2,self.Move_3,self.Move_type_3,self.Move_mode_3,self.Move_info_3,self.Move_method_3,self.Move_power_3,self.Move_accuracy_3,self.Move_PP_3,self.Move_Effect_3,self.Move_4,self.Move_type_4,self.Move_mode_4,self.Move_info_4,self.Move_method_4,self.Move_power_4,self.Move_accuracy_4,self.Move_PP_4,self.Move_Effect_4)
                            #Passes previewed pokemon dictionary to the ViewPokemonDialog_from_dict Box
                            #Sets up ViewPokemonDialog_from_dict Box
                            dlg = ViewPokemonDialog_from_dict(self.this_pokemon_preview, preview='yes')
                            #Displays ViewPokemonDialog_from_dict Box
                            dlg.Show()
                        #When the user doesn't want to preview pokemon entry, the message box closes and the use retuns to the AddPokemon Dialog Box
                        else:
                            #Closes the attention message dialog box    
                            dlg_attention.Destroy()

                    #When the pokemon entry is not in the database, the program can proceed to create it normally
                    elif self.Name not in Pokemon_Team_Builder_Frame.pokemon_available:
                        #Uses function to assemble pokemon entry into local dictionary
                        self.this_pokemon_preview = preview_pokemon(self.Species,self.Form,self.Image,self.Name,self.Type_1,self.Type_2,self.Ability_name,self.Ability_info,self.Item,self.Move_1,self.Move_type_1,self.Move_mode_1,self.Move_info_1,self.Move_method_1,self.Move_power_1,self.Move_accuracy_1,self.Move_PP_1,self.Move_Effect_1,self.Move_2,self.Move_type_2,self.Move_mode_2,self.Move_info_2,self.Move_method_2,self.Move_power_2,self.Move_accuracy_2,self.Move_PP_2,self.Move_Effect_2,self.Move_3,self.Move_type_3,self.Move_mode_3,self.Move_info_3,self.Move_method_3,self.Move_power_3,self.Move_accuracy_3,self.Move_PP_3,self.Move_Effect_3,self.Move_4,self.Move_type_4,self.Move_mode_4,self.Move_info_4,self.Move_method_4,self.Move_power_4,self.Move_accuracy_4,self.Move_PP_4,self.Move_Effect_4)
                        #Passes previewed pokemon dictionary to the ViewPokemonDialog_from_dict Box
                        #Sets up ViewPokemonDialog_from_dict Box
                        dlg = ViewPokemonDialog_from_dict(self.this_pokemon_preview, preview='yes')
                        #Displays ViewPokemonDialog_from_dict Box
                        dlg.Show()

# ======================================================================================= # Add Team Entry
        
#Defines Add Team Entry as a Dialog Box
class AddTeamDialog(wx.Dialog):
            
    def __init__(self):
        title = "Add a Team Entry"
        super().__init__(parent=None, title="Add Team Entry", style = wx.DEFAULT_DIALOG_STYLE | wx.RESIZE_BORDER, size = (820,375))        
        
        #Defines self.this_Team dictionary that will hold the team information when it's assembled
        self.this_Team = {}
        
        #Defines Main Sizer
        self.main_sizer = wx.BoxSizer(wx.VERTICAL)
        
        #Defines font style for the statict text that labels the Team Members and Team Info titles
        font_moves = wx.Font(12, wx.DECORATIVE, wx.BOLD, wx.NORMAL)
        
        #Sets up a Static Text to be used as empty space and adds it to the main sizer
        empty_space_label = wx.StaticText(self, label="")
        empty_space_label.SetFont(font_moves)
        self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
        
        #Sets up a Static Text for Team Members and adds it to the main sizer
        Team_Members_label = wx.StaticText(self, label="Team Members")
        Team_Members_label.SetFont(font_moves)
        self.main_sizer.Add(Team_Members_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
        
        #Sets up Box for Name, starting empty
        self.Name_box = wx.TextCtrl(self, value='')
        self.add_widgets('Name:', self.Name_box)
        
        #Sets up Combo Box and View Button for Pokemon_1, starting empty
        self.Pokemon_1_box = wx.ComboBox(self, size=(95, -1), choices=Pokemon_Team_Builder_Frame.pokemon_available, style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
        #Defines View Button
        self.view_btn_1 = wx.Button(self, label='View')
        #Binds Button event to View Button
        self.view_btn_1.Bind(wx.EVT_BUTTON, self.on_view_pokemon_1)        
        self.add_widgets_view('Pokemon 1:', self.Pokemon_1_box, self.view_btn_1 )
        
        #Sets up Combo Box and View Button for Pokemon_2, starting empty
        self.Pokemon_2_box = wx.ComboBox(self, size=(95, -1), choices=Pokemon_Team_Builder_Frame.pokemon_available, style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
        #Defines View Button
        self.view_btn_2 = wx.Button(self, label='View')
        #Binds Button event to View Button
        self.view_btn_2.Bind(wx.EVT_BUTTON, self.on_view_pokemon_2)
        self.add_widgets_view('Pokemon 2:', self.Pokemon_2_box, self.view_btn_2 )
        
        #Sets up Combo Box and View Button for Pokemon_3, starting empty
        self.Pokemon_3_box = wx.ComboBox(self, size=(95, -1), choices=Pokemon_Team_Builder_Frame.pokemon_available, style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
        #Defines View Button
        self.view_btn_3 = wx.Button(self, label='View')
        #Binds Button event to View Button
        self.view_btn_3.Bind(wx.EVT_BUTTON, self.on_view_pokemon_3)
        self.add_widgets_view('Pokemon 3:', self.Pokemon_3_box, self.view_btn_3 )
        
        #Sets up Combo Box and View Button for Pokemon_4, starting empty
        self.Pokemon_4_box = wx.ComboBox(self, size=(95, -1), choices=Pokemon_Team_Builder_Frame.pokemon_available, style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
        #Defines View Button
        self.view_btn_4 = wx.Button(self, label='View')
        #Binds Button event to View Button
        self.view_btn_4.Bind(wx.EVT_BUTTON, self.on_view_pokemon_4)
        self.add_widgets_view('Pokemon 4:', self.Pokemon_4_box, self.view_btn_4 )
        
        #Sets up Combo Box and View Button for Pokemon_5, starting empty
        self.Pokemon_5_box = wx.ComboBox(self, size=(95, -1), choices=Pokemon_Team_Builder_Frame.pokemon_available, style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
        #Defines View Button
        self.view_btn_5 = wx.Button(self, label='View')
        #Binds Button event to View Button
        self.view_btn_5.Bind(wx.EVT_BUTTON, self.on_view_pokemon_5)
        self.add_widgets_view('Pokemon 5:', self.Pokemon_5_box, self.view_btn_5 )
        
        #Sets up Combo Box and View Button for Pokemon_6, starting empty
        self.Pokemon_6_box = wx.ComboBox(self, size=(95, -1), choices=Pokemon_Team_Builder_Frame.pokemon_available, style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
        #Defines View Button
        self.view_btn_6 = wx.Button(self, label='View')
        #Binds Button event to View Button
        self.view_btn_6.Bind(wx.EVT_BUTTON, self.on_view_pokemon_6)
        self.add_widgets_view('Pokemon 6:', self.Pokemon_6_box, self.view_btn_6 )
        
        #Sets up a Static Text to be used as empty space and adds it to the main sizer
        empty_space_label = wx.StaticText(self, label="")
        empty_space_label.SetFont(font_moves)
        self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
        
        # ====================== #
        
        #Defines Button Sizer
        btn_sizer = wx.BoxSizer()
        #Creates Calculate Button
        calculate_btn = wx.Button(self, label='Calculate')
        #Binds Button event to Calculate Button
        calculate_btn.Bind(wx.EVT_BUTTON, self.on_calculate_team)
        #Adds Calculate Button to Button Sizer
        btn_sizer.Add(calculate_btn, 0, wx.ALL, 5)
        #Adds Cancel Button (pre defined) to Button Sizer
        btn_sizer.Add(wx.Button(self, id=wx.ID_CANCEL), 0, wx.ALL, 5)
        #Adds Button Sizer to Main Sizer
        self.main_sizer.Add(btn_sizer, 0, wx.CENTER)
        
        #Sets Main Sizer for user to see
        self.SetSizer(self.main_sizer)
              
     # ====================================== # Organizing layout   
    
    #This function helps position each box (Text Control, Combo Box, Radio Box) to entry information
    def add_widgets(self, label_text, entry_box):
        #Defines font style for widgets statict text
        font_labels = wx.Font(12, wx.DECORATIVE, wx.BOLD, wx.NORMAL)   
        #Defines Row Sizer for each "Line"
        row_sizer = wx.BoxSizer(wx.HORIZONTAL)
        #Uses Static Text to Label the boxes
        label = wx.StaticText(self, label=label_text, size=(135, -1))
        #Sets font style to label
        label.SetFont(font_labels)        
        #Adds Static Text Label to Row Sizer
        row_sizer.Add(label, 0, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Adds Box to add information to row sizer
        row_sizer.Add(entry_box, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Adds Row Sizer to Main Sizer
        self.main_sizer.Add(row_sizer, 0, wx.EXPAND)
        
    #This function helps position each box (Text Control, Combo Box, Radio Box) to entry information with the addition of a side button
    def add_widgets_view(self, label_text, entry_box, view_btn):
        #Defines font style for widgets statict text
        font_labels = wx.Font(12, wx.DECORATIVE, wx.BOLD, wx.NORMAL)   
        #Defines Row Sizer for each "Line"
        row_sizer = wx.BoxSizer(wx.HORIZONTAL)
        #Uses Static Text to Label the boxes
        label = wx.StaticText(self, label=label_text, size=(135, -1))
        #Sets font style to label
        label.SetFont(font_labels)  
        #Adds Static Text Label to Row Sizer
        row_sizer.Add(label, 0, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Adds Box to add information to row sizer
        row_sizer.Add(entry_box, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Adds Row Sizer to Main Sizer
        self.main_sizer.Add(row_sizer, 0, wx.EXPAND)
        #Adds View Button to Row Sizer
        row_sizer.Add(view_btn, 0, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        
    # ====================================== # When calculate button is pressed
        
    #This function is called when the calculate button is pressed. It gets the data from the pokemon names from the entry boxes and assembles a dictionary
    #It also displays the recently calculated team info into the bottom boxes
    def on_calculate_team(self, event):
        
        #Gets Team Name
        Team_name = self.Name_box.GetValue()
        
        #Creates/resets local pokemon_name_list variable
        pokemon_name_list = []        
        
        #Gets pokemon names from entry boxes and adds them to the pokemon_name_list
        pokemon_name_list.append(self.Pokemon_1_box.GetValue())
        pokemon_name_list.append(self.Pokemon_2_box.GetValue())
        pokemon_name_list.append(self.Pokemon_3_box.GetValue())
        pokemon_name_list.append(self.Pokemon_4_box.GetValue())
        pokemon_name_list.append(self.Pokemon_5_box.GetValue())
        pokemon_name_list.append(self.Pokemon_6_box.GetValue())
        
        #Checks if there was an empty field
        #If any of the variables is empty, the program tells the user to fill up the boxes
        if '' in pokemon_name_list or Team_name == '':
            #Displays a default error message dialog telling the user to fill up the fields/boxes
            #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
            dlg = wx.MessageDialog(self, caption='Error!', message = 'One or more fields are emtpy! Please fill up all the fields with Pokemon information.', style= wx.OK | wx.ICON_ERROR)
            dlg.ShowModal()
            #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
            #Closes the message dialog box    
            dlg.Destroy()
        
        #when all fields/boxes are filled, the program proceeds normally
        else:        
            #Checks if there is a repeated name in the pokemon_name_list
            #creates/resets helpful variable
            repeated_name = []
            for element in pokemon_name_list:
                #if a name appears more than once, the program displays a default error message telling the user to not repeat names
                if pokemon_name_list.count(element) > 1:
                    #Only adds repeated pokemon name to the helpful bariable for the first time, avoiding adding the same pokemon name twice
                    if element not in repeated_name:
                        repeated_name.append(element)
            #If there is a repeated pokemon name
            if repeated_name != []:
                #Displays the default error message dialog
                #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
                dlg = wx.MessageDialog(self, caption='Error!', message = 'One or more pokemon were selected multiple times! Please only select a pokemon name once.', style= wx.OK | wx.ICON_ERROR)
                dlg.ShowModal()
                #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
                #Closes the message dialog box    
                dlg.Destroy()

            #When there is no repeated pokemon name, the program proceeds normally
            else:
                #When the team entry already was created, provide the ok or cancel option in an attention dialog
                if Team_name in Pokemon_Team_Builder_Frame.teams_available:
                    #Calls a default attention Message Dialog
                    #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
                    dlg_attention = wx.MessageDialog(self, caption='Attention!', message = 'A Team Entry with this name already exists! Would you still like to proceed?', style= wx.OK | wx.CANCEL | wx.ICON_WARNING)
                    #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown
                    #If the user presses the attention dialog box’s OK button, the program can get the user’s path choice via the dialog’s .GetPath() function
                    if dlg_attention.ShowModal() == wx.ID_OK:
                        #Assembles a Team Dictionary and Calculates pokemon team information
                        self.this_Team = assemble_team(Pokemon_Team_Builder_Frame.data_path,Team_name,pokemon_name_list)
                        #Sets up PreViewTeamDialog Box while passing self.this_Team to it
                        dlg_preview = PreViewTeamDialog(self.this_Team, preview='yes')
                        #Displays ViewPokemonDialog Box using ShowModal(), so the user will only be able to interact  with this dialog box until it is closed
                        ret = dlg_preview.ShowModal()  #ret keeps track of the value returned by dlg_preview
                        #print(ret)
                        if ret == 1: #when save team is selected, dlg_preview returns a value (True)
                            #Closes AddTeamDialog Box
                            self.Destroy()
                        
                        #Closes the attention message dialog box    
                        dlg_attention.Destroy()

                    #When the user doesn't want to use the same team name, the message box closes and the use retuns to the AddTeam Dialog Box
                    else:
                        #Closes the attention message dialog box    
                        dlg_attention.Destroy()
                
                #When a team entry with the provided Team Name is not in the database (it is a new entry)
                else:
                    #Assembles a Team Dictionary and Calculates pokemon team information
                    self.this_Team = assemble_team(Pokemon_Team_Builder_Frame.data_path,Team_name,pokemon_name_list)
                    #Sets up PreViewTeamDialog Box while passing self.this_Team to it
                    dlg_preview = PreViewTeamDialog(self.this_Team, preview='yes')
                    #Displays ViewPokemonDialog Box using ShowModal(), so the user will only be able to interact  with this dialog box until it is closed
                    ret = dlg_preview.ShowModal()  #ret keeps track of the value returned by dlg_preview
                    #print(ret)
                    if ret == 1: #when save team is selected, dlg_preview returns a value (True)
                        #Closes AddTeamDialog Box
                        self.Destroy()
                    
                
    # ====================================== # When view pokemon 1 button is pressed
        
    #This function is called when the view pokemon 1 button is pressed. 
    #It gets the data from the pokemon name from the entry box in the same line and passes it to the ViewPokemonDialog
    def on_view_pokemon_1(self, event):
        
        #Gets Pokemon_1 Name
        Pokemon_1_name = self.Pokemon_1_box.GetValue()
        #Reads selected pokemon data into a python dictionary named this_Pokemon using given pokemon name
        this_Pokemon = read_pokemon(Pokemon_Team_Builder_Frame.data_path,Pokemon_1_name)
        
        #Passes it to the ViewPokemonDialog Box
        #Sets up ViewPokemonDialog Box
        dlg = ViewPokemonDialog_from_dict(this_Pokemon, preview='no')
        #Displays ViewPokemonDialog Box
        dlg.Show()
        
    # ====================================== # When view pokemon 2 button is pressed
        
    #This function is called when the view pokemon 2 button is pressed. 
    #It gets the data from the pokemon name from the entry box in the same line and passes it to the ViewPokemonDialog
    def on_view_pokemon_2(self, event):
        
        #Gets Pokemon_2 Name
        Pokemon_2_name = self.Pokemon_2_box.GetValue()
        #Reads selected pokemon data into a python dictionary named this_Pokemon using given pokemon name
        this_Pokemon = read_pokemon(Pokemon_Team_Builder_Frame.data_path,Pokemon_2_name)
        
        #Passes it to the ViewPokemonDialog Box
        #Sets up ViewPokemonDialog Box
        dlg = ViewPokemonDialog_from_dict(this_Pokemon, preview='no')
        #Displays ViewPokemonDialog Box
        dlg.Show()
        
# ====================================== # When view pokemon 3 button is pressed
        
    #This function is called when the view pokemon 3 button is pressed. 
    #It gets the data from the pokemon name from the entry box in the same line and passes it to the ViewPokemonDialog
    def on_view_pokemon_3(self, event):
        
        #Gets Pokemon_3 Name
        Pokemon_3_name = self.Pokemon_3_box.GetValue()
        #Reads selected pokemon data into a python dictionary named this_Pokemon using given pokemon name
        this_Pokemon = read_pokemon(Pokemon_Team_Builder_Frame.data_path,Pokemon_3_name)
        
        #Passes it to the ViewPokemonDialog Box
        #Sets up ViewPokemonDialog Box
        dlg = ViewPokemonDialog_from_dict(this_Pokemon, preview='no')
        #Displays ViewPokemonDialog Box
        dlg.Show()       

# ====================================== # When view pokemon 4 button is pressed
        
    #This function is called when the view pokemon 4 button is pressed. 
    #It gets the data from the pokemon name from the entry box in the same line and passes it to the ViewPokemonDialog
    def on_view_pokemon_4(self, event):
        
        #Gets Pokemon_4 Name
        Pokemon_4_name = self.Pokemon_4_box.GetValue()
        #Reads selected pokemon data into a python dictionary named this_Pokemon using given pokemon name
        this_Pokemon = read_pokemon(Pokemon_Team_Builder_Frame.data_path,Pokemon_4_name)
        
        #Passes it to the ViewPokemonDialog Box
        #Sets up ViewPokemonDialog Box
        dlg = ViewPokemonDialog_from_dict(this_Pokemon, preview='no')
        #Displays ViewPokemonDialog Box
        dlg.Show()
        
    # ====================================== # When view pokemon 5 button is pressed
        
    #This function is called when the view pokemon 5 button is pressed. 
    #It gets the data from the pokemon name from the entry box in the same line and passes it to the ViewPokemonDialog
    def on_view_pokemon_5(self, event):
        
        #Gets Pokemon_5 Name
        Pokemon_5_name = self.Pokemon_5_box.GetValue()
        #Reads selected pokemon data into a python dictionary named this_Pokemon using given pokemon name
        this_Pokemon = read_pokemon(Pokemon_Team_Builder_Frame.data_path,Pokemon_5_name)
        
        #Passes it to the ViewPokemonDialog Box
        #Sets up ViewPokemonDialog Box
        dlg = ViewPokemonDialog_from_dict(this_Pokemon, preview='no')
        #Displays ViewPokemonDialog Box
        dlg.Show()
        
    # ====================================== # When view pokemon 6 button is pressed
        
    #This function is called when the view pokemon 6 button is pressed. 
    #It gets the data from the pokemon name from the entry box in the same line and passes it to the ViewPokemonDialog
    def on_view_pokemon_6(self, event):
        
        #Gets Pokemon_6 Name
        Pokemon_6_name = self.Pokemon_6_box.GetValue()
        #Reads selected pokemon data into a python dictionary named this_Pokemon using given pokemon name
        this_Pokemon = read_pokemon(Pokemon_Team_Builder_Frame.data_path,Pokemon_6_name)
        
        #Passes it to the ViewPokemonDialog Box
        #Sets up ViewPokemonDialog Box
        dlg = ViewPokemonDialog_from_dict(this_Pokemon, preview='no')
        #Displays ViewPokemonDialog Box
        dlg.Show()
        
# ======================================================================================= # Edit Team Entry
        
#Defines Edit Team Entry as a Dialog Box
class EditTeamDialog(wx.Dialog):
            
    def __init__(self, team_name):
        title = "Edit a Team Entry"
        super().__init__(parent=None, title="Edit Team Entry", style = wx.DEFAULT_DIALOG_STYLE | wx.RESIZE_BORDER, size = (820,375))        
        
        #Reads selected team data into a local python dictionary named this_Team
        self.this_Team = read_team(Pokemon_Team_Builder_Frame.data_path,team_name)
        
        #Defines Main Sizer
        self.main_sizer = wx.BoxSizer(wx.VERTICAL)
        
        #Defines font style for the statict text that labels the Team Members and Team Info titles
        font_moves = wx.Font(12, wx.DECORATIVE, wx.BOLD, wx.NORMAL)
        
        #Sets up a Static Text to be used as empty space and adds it to the main sizer
        empty_space_label = wx.StaticText(self, label="")
        empty_space_label.SetFont(font_moves)
        self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
        
        #Sets up a Static Text for Team Members and adds it to the main sizer
        Team_Members_label = wx.StaticText(self, label="Team Members")
        Team_Members_label.SetFont(font_moves)
        self.main_sizer.Add(Team_Members_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
        
        #Sets up Box for Name (READONLY), starting with former information
        self.Name_box = wx.TextCtrl(self, value=str(self.this_Team['Team_Name'][0]), style=wx.TE_READONLY)
        self.add_widgets('Name:', self.Name_box)
        
        #Sets up Combo Box and View Button for Pokemon_1
        self.Pokemon_1_box = wx.ComboBox(self, size=(95, -1), choices=Pokemon_Team_Builder_Frame.pokemon_available, value=str(self.this_Team['Members']['Pokemon_1']['Name'][0]), style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
        #Defines View Button
        self.view_btn_1 = wx.Button(self, label='View')
        #Binds Button event to View Button
        self.view_btn_1.Bind(wx.EVT_BUTTON, self.on_view_pokemon_1)        
        self.add_widgets_view('Pokemon 1:', self.Pokemon_1_box, self.view_btn_1 )
        
        #Sets up Combo Box and View Button for Pokemon_2
        self.Pokemon_2_box = wx.ComboBox(self, size=(95, -1), choices=Pokemon_Team_Builder_Frame.pokemon_available, value=str(self.this_Team['Members']['Pokemon_2']['Name'][0]), style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
        #Defines View Button
        self.view_btn_2 = wx.Button(self, label='View')
        #Binds Button event to View Button
        self.view_btn_2.Bind(wx.EVT_BUTTON, self.on_view_pokemon_2)
        self.add_widgets_view('Pokemon 2:', self.Pokemon_2_box, self.view_btn_2 )
        
        #Sets up Combo Box and View Button for Pokemon_3
        self.Pokemon_3_box = wx.ComboBox(self, size=(95, -1), choices=Pokemon_Team_Builder_Frame.pokemon_available, value=str(self.this_Team['Members']['Pokemon_3']['Name'][0]), style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
        #Defines View Button
        self.view_btn_3 = wx.Button(self, label='View')
        #Binds Button event to View Button
        self.view_btn_3.Bind(wx.EVT_BUTTON, self.on_view_pokemon_3)
        self.add_widgets_view('Pokemon 3:', self.Pokemon_3_box, self.view_btn_3 )
        
        #Sets up Combo Box and View Button for Pokemon_4
        self.Pokemon_4_box = wx.ComboBox(self, size=(95, -1), choices=Pokemon_Team_Builder_Frame.pokemon_available, value=str(self.this_Team['Members']['Pokemon_4']['Name'][0]), style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
        #Defines View Button
        self.view_btn_4 = wx.Button(self, label='View')
        #Binds Button event to View Button
        self.view_btn_4.Bind(wx.EVT_BUTTON, self.on_view_pokemon_4)
        self.add_widgets_view('Pokemon 4:', self.Pokemon_4_box, self.view_btn_4 )
        
        #Sets up Combo Box and View Button for Pokemon_5
        self.Pokemon_5_box = wx.ComboBox(self, size=(95, -1), choices=Pokemon_Team_Builder_Frame.pokemon_available, value=str(self.this_Team['Members']['Pokemon_5']['Name'][0]), style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
        #Defines View Button
        self.view_btn_5 = wx.Button(self, label='View')
        #Binds Button event to View Button
        self.view_btn_5.Bind(wx.EVT_BUTTON, self.on_view_pokemon_5)
        self.add_widgets_view('Pokemon 5:', self.Pokemon_5_box, self.view_btn_5 )
        
        #Sets up Combo Box and View Button for Pokemon_6
        self.Pokemon_6_box = wx.ComboBox(self, size=(95, -1), choices=Pokemon_Team_Builder_Frame.pokemon_available, value=str(self.this_Team['Members']['Pokemon_6']['Name'][0]), style=wx.CB_DROPDOWN | wx.CB_READONLY | wx.CB_SORT)
        #Defines View Button
        self.view_btn_6 = wx.Button(self, label='View')
        #Binds Button event to View Button
        self.view_btn_6.Bind(wx.EVT_BUTTON, self.on_view_pokemon_6)
        self.add_widgets_view('Pokemon 6:', self.Pokemon_6_box, self.view_btn_6 )
        
        #Sets up a Static Text to be used as empty space and adds it to the main sizer
        empty_space_label = wx.StaticText(self, label="")
        empty_space_label.SetFont(font_moves)
        self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
        
        # ====================== #
        
        #Defines Button Sizer
        btn_sizer = wx.BoxSizer()
        #Creates Calculate Button
        calculate_btn = wx.Button(self, label='Calculate')
        #Binds Button event to Calculate Button
        calculate_btn.Bind(wx.EVT_BUTTON, self.on_calculate_team)
        #Adds Calculate Button to Button Sizer
        btn_sizer.Add(calculate_btn, 0, wx.ALL, 5)
        #Adds Cancel Button (pre defined) to Button Sizer
        btn_sizer.Add(wx.Button(self, id=wx.ID_CANCEL), 0, wx.ALL, 5)
        #Adds Button Sizer to Main Sizer
        self.main_sizer.Add(btn_sizer, 0, wx.CENTER)
        
        #Sets Main Sizer for user to see
        self.SetSizer(self.main_sizer)
              
     # ====================================== # Organizing layout   
    
    #This function helps position each box (Text Control, Combo Box, Radio Box) to entry information
    def add_widgets(self, label_text, entry_box):
        #Defines font style for widgets statict text
        font_labels = wx.Font(12, wx.DECORATIVE, wx.BOLD, wx.NORMAL)   
        #Defines Row Sizer for each "Line"
        row_sizer = wx.BoxSizer(wx.HORIZONTAL)
        #Uses Static Text to Label the boxes
        label = wx.StaticText(self, label=label_text, size=(135, -1))
        #Sets font style to label
        label.SetFont(font_labels)        
        #Adds Static Text Label to Row Sizer
        row_sizer.Add(label, 0, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Adds Box to add information to row sizer
        row_sizer.Add(entry_box, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Adds Row Sizer to Main Sizer
        self.main_sizer.Add(row_sizer, 0, wx.EXPAND)
        
    #This function helps position each box (Text Control, Combo Box, Radio Box) to entry information with the addition of a side button
    def add_widgets_view(self, label_text, entry_box, view_btn):
        #Defines font style for widgets statict text
        font_labels = wx.Font(12, wx.DECORATIVE, wx.BOLD, wx.NORMAL)   
        #Defines Row Sizer for each "Line"
        row_sizer = wx.BoxSizer(wx.HORIZONTAL)
        #Uses Static Text to Label the boxes
        label = wx.StaticText(self, label=label_text, size=(135, -1))
        #Sets font style to label
        label.SetFont(font_labels)  
        #Adds Static Text Label to Row Sizer
        row_sizer.Add(label, 0, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Adds Box to add information to row sizer
        row_sizer.Add(entry_box, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Adds Row Sizer to Main Sizer
        self.main_sizer.Add(row_sizer, 0, wx.EXPAND)
        #Adds View Button to Row Sizer
        row_sizer.Add(view_btn, 0, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        
    # ====================================== # When calculate button is pressed
        
    #This function is called when the calculate button is pressed. It gets the data from the pokemon names from the entry boxes and assembles a dictionary
    #It also displays the recently calculated team info into the bottom boxes
    def on_calculate_team(self, event):
        
        #Gets Team Name
        Team_name = self.Name_box.GetValue()
        
        #Creates/resets local pokemon_name_list variable
        pokemon_name_list = []        
        
        #Gets pokemon names from entry boxes and adds them to the pokemon_name_list
        pokemon_name_list.append(self.Pokemon_1_box.GetValue())
        pokemon_name_list.append(self.Pokemon_2_box.GetValue())
        pokemon_name_list.append(self.Pokemon_3_box.GetValue())
        pokemon_name_list.append(self.Pokemon_4_box.GetValue())
        pokemon_name_list.append(self.Pokemon_5_box.GetValue())
        pokemon_name_list.append(self.Pokemon_6_box.GetValue())
        
        #Checks if there was an empty field
        #If any of the variables is empty, the program tells the user to fill up the boxes
        if '' in pokemon_name_list or Team_name == '':
            #Displays a default error message dialog telling the user to fill up the fields/boxes
            #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
            dlg = wx.MessageDialog(self, caption='Error!', message = 'One or more fields are emtpy! Please fill up all the fields with Pokemon information.', style= wx.OK | wx.ICON_ERROR)
            dlg.ShowModal()
            #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
            #Closes the message dialog box    
            dlg.Destroy()
        
        #when all fields/boxes are filled, the program proceeds normally
        else:        
            #Checks if there is a repeated name in the pokemon_name_list
            #creates/resets helpful variable
            repeated_name = []
            for element in pokemon_name_list:
                #if a name appears more than once, the program displays a default error message telling the user to not repeat names
                if pokemon_name_list.count(element) > 1:
                    #Only adds repeated pokemon name to the helpful bariable for the first time, avoiding adding the same pokemon name twice
                    if element not in repeated_name:
                        repeated_name.append(element)
            #If there is a repeated pokemon name
            if repeated_name != []:
                #Displays the default error message dialog
                #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
                dlg = wx.MessageDialog(self, caption='Error!', message = 'One or more pokemon were selected multiple times! Please only select a pokemon name once.', style= wx.OK | wx.ICON_ERROR)
                dlg.ShowModal()
                #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
                #Closes the message dialog box    
                dlg.Destroy()

            #When there is no repeated pokemon name, the program proceeds normally
            else:
                #When the team entry already was created, provide the ok or cancel option in an attention dialog
                if Team_name in Pokemon_Team_Builder_Frame.teams_available:
                    #Calls a default attention Message Dialog
                    #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
                    dlg_attention = wx.MessageDialog(self, caption='Attention!', message = 'A Team Entry with this name already exists! Would you still like to proceed?', style= wx.OK | wx.CANCEL | wx.ICON_WARNING)
                    #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown
                    #If the user presses the attention dialog box’s OK button, the program can get the user’s path choice via the dialog’s .GetPath() function
                    if dlg_attention.ShowModal() == wx.ID_OK:
                        #Assembles a Team Dictionary and Calculates pokemon team information
                        self.this_Team = assemble_team(Pokemon_Team_Builder_Frame.data_path,Team_name,pokemon_name_list)
                        #Sets up PreViewTeamDialog Box while passing self.this_Team to it
                        dlg_preview = PreViewTeamDialog(self.this_Team, preview='yes')
                        #Displays ViewPokemonDialog Box using ShowModal(), so the user will only be able to interact  with this dialog box until it is closed
                        ret = dlg_preview.ShowModal()  #ret keeps track of the value returned by dlg_preview
                        #print(ret)
                        if ret == 1: #when save team is selected, dlg_preview returns a value (True)
                            #Closes AddTeamDialog Box
                            self.Destroy()
                        
                        #Closes the attention message dialog box    
                        dlg_attention.Destroy()

                    #When the user doesn't want to use the same team name, the message box closes and the use retuns to the AddTeam Dialog Box
                    else:
                        #Closes the attention message dialog box    
                        dlg_attention.Destroy()
                
                #When a team entry with the provided Team Name is not in the database (it is a new entry)
                else:
                    #Assembles a Team Dictionary and Calculates pokemon team information
                    self.this_Team = assemble_team(Pokemon_Team_Builder_Frame.data_path,Team_name,pokemon_name_list)
                    #Sets up PreViewTeamDialog Box while passing self.this_Team to it
                    dlg_preview = PreViewTeamDialog(self.this_Team, preview='yes')
                    #Displays ViewPokemonDialog Box using ShowModal(), so the user will only be able to interact  with this dialog box until it is closed
                    ret = dlg_preview.ShowModal()  #ret keeps track of the value returned by dlg_preview
                    #print(ret)
                    if ret == 1: #when save team is selected, dlg_preview returns a value (True)
                        #Closes AddTeamDialog Box
                        self.Destroy()
                    
                
    # ====================================== # When view pokemon 1 button is pressed
        
    #This function is called when the view pokemon 1 button is pressed. 
    #It gets the data from the pokemon name from the entry box in the same line and passes it to the ViewPokemonDialog
    def on_view_pokemon_1(self, event):
        
        #Gets Pokemon_1 Name
        Pokemon_1_name = self.Pokemon_1_box.GetValue()
        #Reads selected pokemon data into a python dictionary named this_Pokemon using given pokemon name
        this_Pokemon = read_pokemon(Pokemon_Team_Builder_Frame.data_path,Pokemon_1_name)
        
        #Passes it to the ViewPokemonDialog Box
        #Sets up ViewPokemonDialog Box
        dlg = ViewPokemonDialog_from_dict(this_Pokemon, preview='no')
        #Displays ViewPokemonDialog Box
        dlg.Show()
        
    # ====================================== # When view pokemon 2 button is pressed
        
    #This function is called when the view pokemon 2 button is pressed. 
    #It gets the data from the pokemon name from the entry box in the same line and passes it to the ViewPokemonDialog
    def on_view_pokemon_2(self, event):
        
        #Gets Pokemon_2 Name
        Pokemon_2_name = self.Pokemon_2_box.GetValue()
        #Reads selected pokemon data into a python dictionary named this_Pokemon using given pokemon name
        this_Pokemon = read_pokemon(Pokemon_Team_Builder_Frame.data_path,Pokemon_2_name)
        
        #Passes it to the ViewPokemonDialog Box
        #Sets up ViewPokemonDialog Box
        dlg = ViewPokemonDialog_from_dict(this_Pokemon, preview='no')
        #Displays ViewPokemonDialog Box
        dlg.Show()
        
# ====================================== # When view pokemon 3 button is pressed
        
    #This function is called when the view pokemon 3 button is pressed. 
    #It gets the data from the pokemon name from the entry box in the same line and passes it to the ViewPokemonDialog
    def on_view_pokemon_3(self, event):
        
        #Gets Pokemon_3 Name
        Pokemon_3_name = self.Pokemon_3_box.GetValue()
        #Reads selected pokemon data into a python dictionary named this_Pokemon using given pokemon name
        this_Pokemon = read_pokemon(Pokemon_Team_Builder_Frame.data_path,Pokemon_3_name)
        
        #Passes it to the ViewPokemonDialog Box
        #Sets up ViewPokemonDialog Box
        dlg = ViewPokemonDialog_from_dict(this_Pokemon, preview='no')
        #Displays ViewPokemonDialog Box
        dlg.Show()       

# ====================================== # When view pokemon 4 button is pressed
        
    #This function is called when the view pokemon 4 button is pressed. 
    #It gets the data from the pokemon name from the entry box in the same line and passes it to the ViewPokemonDialog
    def on_view_pokemon_4(self, event):
        
        #Gets Pokemon_4 Name
        Pokemon_4_name = self.Pokemon_4_box.GetValue()
        #Reads selected pokemon data into a python dictionary named this_Pokemon using given pokemon name
        this_Pokemon = read_pokemon(Pokemon_Team_Builder_Frame.data_path,Pokemon_4_name)
        
        #Passes it to the ViewPokemonDialog Box
        #Sets up ViewPokemonDialog Box
        dlg = ViewPokemonDialog_from_dict(this_Pokemon, preview='no')
        #Displays ViewPokemonDialog Box
        dlg.Show()
        
    # ====================================== # When view pokemon 5 button is pressed
        
    #This function is called when the view pokemon 5 button is pressed. 
    #It gets the data from the pokemon name from the entry box in the same line and passes it to the ViewPokemonDialog
    def on_view_pokemon_5(self, event):
        
        #Gets Pokemon_5 Name
        Pokemon_5_name = self.Pokemon_5_box.GetValue()
        #Reads selected pokemon data into a python dictionary named this_Pokemon using given pokemon name
        this_Pokemon = read_pokemon(Pokemon_Team_Builder_Frame.data_path,Pokemon_5_name)
        
        #Passes it to the ViewPokemonDialog Box
        #Sets up ViewPokemonDialog Box
        dlg = ViewPokemonDialog_from_dict(this_Pokemon, preview='no')
        #Displays ViewPokemonDialog Box
        dlg.Show()
        
    # ====================================== # When view pokemon 6 button is pressed
        
    #This function is called when the view pokemon 6 button is pressed. 
    #It gets the data from the pokemon name from the entry box in the same line and passes it to the ViewPokemonDialog
    def on_view_pokemon_6(self, event):
        
        #Gets Pokemon_6 Name
        Pokemon_6_name = self.Pokemon_6_box.GetValue()
        #Reads selected pokemon data into a python dictionary named this_Pokemon using given pokemon name
        this_Pokemon = read_pokemon(Pokemon_Team_Builder_Frame.data_path,Pokemon_6_name)
        
        #Passes it to the ViewPokemonDialog Box
        #Sets up ViewPokemonDialog Box
        dlg = ViewPokemonDialog_from_dict(this_Pokemon, preview='no')
        #Displays ViewPokemonDialog Box
        dlg.Show()
        
        
# ======================================================================================= # View Pokemon Entry using a pre-loaded pokemon dictionary
        
#Defines View Pokemon Entry as a Dialog Box using a pre-assembled pokemon dictionary
class ViewPokemonDialog_from_dict(wx.Dialog):
            
    def __init__(self, pokemon_dict, preview):
        
        if preview == 'yes':
            this_title = 'Preview Pokemon Entry'
        else:
            this_title = 'View Pokemon Entry'
        
        super().__init__(parent=None, title=this_title, style = wx.DEFAULT_DIALOG_STYLE | wx.RESIZE_BORDER, size = (1450,1030))   
        
        #Checks if the pokemon_dict provided is blank
        if pokemon_dict == '':
            #Displays a default error message dialog telling the user to select a pokemon name from database
            #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
            dlg = wx.MessageDialog(self, caption='Error!', message = 'The pokemon name is blank! Please select a valid pokemon name from the provided database.', style= wx.OK | wx.ICON_ERROR)
            dlg.ShowModal()
            #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
            #Closes the message dialog box    
            dlg.Destroy()
            
            #Closes ViewPokemonDialog Box
            self.Destroy()
            
        #When a pokemon_dict was properly provided    
        else:        
            #Updates this_Pokemon local dictionary with provided pokemon_dict
            this_Pokemon = pokemon_dict

            #Defines Helpful Variables
            self.all_types = ['Normal', 'Fighting', 'Flying', 'Poison', 'Ground', 'Rock', 'Bug', 'Ghost', 'Steel', 'Fire', 'Water', 'Grass', 'Electric', 'Psychic', 'Ice', 'Dragon', 'Dark', 'Fairy']
            self.all_types_2 = ['', 'Normal', 'Fighting', 'Flying', 'Poison', 'Ground', 'Rock', 'Bug', 'Ghost', 'Steel', 'Fire', 'Water', 'Grass', 'Electric', 'Psychic', 'Ice', 'Dragon', 'Dark', 'Fairy']
            self.all_modes = ['Physical', 'Special', 'Status']

            #Defines Global and Buttons Sizer, which will contain Global sizer on top and button sizer at the bottom
            self.global_and_buttons_sizer = wx.BoxSizer(wx.VERTICAL)
            
            #Defines Global sizer, which will contain Main sizer on the left for General Pokemon info 
            #and Battle sizer on the right for type advantages pokemon info
            self.global_sizer = wx.BoxSizer(wx.HORIZONTAL)
            
            #Adds Global Sizer to Global and Buttons Sizer
            self.global_and_buttons_sizer.Add(self.global_sizer, 0, wx.EXPAND)
            
            #Defines Main Sizer
            self.main_sizer = wx.BoxSizer(wx.VERTICAL)
            
            #Adds Main Sizer to Global Sizer
            self.global_sizer.Add(self.main_sizer, 0, wx.EXPAND)

            #Defines font style for the statict text that labels the Moves and Pokemon
            font_moves = wx.Font(12, wx.DECORATIVE, wx.BOLD, wx.NORMAL)

            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Sets up a Static Text for Pokemon and adds it to the main sizer
            pokemon_label = wx.StaticText(self, label="General Info")
            pokemon_label.SetFont(font_moves)
            self.main_sizer.Add(pokemon_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            

 
            # ============== # General
            
            #Defines Top_left_right Sizer
            self.Top_left_right_sizer = wx.BoxSizer(wx.HORIZONTAL)

            #Defines Top_left Sizer
            self.Top_left_sizer = wx.BoxSizer(wx.VERTICAL)

            #Defines Top_right Sizer
            self.Top_right_sizer = wx.BoxSizer(wx.VERTICAL)
            
            #Sets up a Static Text to be used as empty space and adds it to Top_left Sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.Top_left_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            
            #Sets up a Static Text to be used as empty space and adds it to Top_left Sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.Top_left_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            
            #Sets up Box for Name
            self.Name_box = wx.TextCtrl(self, value=this_Pokemon['Name'][0], style=wx.TE_READONLY)
            self.add_widgets('Name:', self.Name_box, self.Top_left_sizer)
            #self.add_widgets('Name:', self.Name_box, self.main_sizer)
            
            #Creates bitmap image box, starting empty
            self.png = wx.StaticBitmap(self, -1)
            #Adds bitmap Image box to Top_right Sizer
            self.Top_right_sizer.Add(self.png, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            #Loads pokemon entry's image file into bitmap image box
            #Loads selected image
            try: #Tries to load it from dex folder
                img = wx.Image(Pokemon_Team_Builder_Frame.dex_path+this_Pokemon['Image'][0], wx.BITMAP_TYPE_ANY)
            except: #Tries to load it from entry folder
                img = wx.Image(Pokemon_Team_Builder_Frame.data_path+this_Pokemon['Image'][0], wx.BITMAP_TYPE_ANY)            
            #Finally sets image into bitmap image box, if there was an image to begin with
            try:
                self.png.SetBitmap(wx.Bitmap(img))
            except:
                pass

            #Sets up Boxes for Species and Form 
            #Sets up Combo Box for Species
            self.Species_box = wx.TextCtrl(self, value=this_Pokemon['Species'][0], style=wx.TE_READONLY)                  
            #Sets up Box for Form
            self.Form_box = wx.TextCtrl(self, value=this_Pokemon['Form'][0], style=wx.TE_READONLY)
            self.add_widgets('Species:', self.Species_box, self.Top_left_sizer)
            self.add_widgets('Form:', self.Form_box, self.Top_left_sizer)            

            #Sets up Combo Boxes for Types
            #Sets up Combo Box for Type_1
            self.Type_1_box = wx.TextCtrl(self, value=this_Pokemon['Types'][0], style=wx.TE_READONLY)
            background_type(self.Type_1_box, this_Pokemon['Types'][0])       
            #Sets up Combo Box for Type_2
            self.Type_2_box = wx.TextCtrl(self, value=this_Pokemon['Types'][1], style=wx.TE_READONLY)
            background_type(self.Type_2_box, this_Pokemon['Types'][1])      
            self.add_widgets_4_boxes_horizontal('Type 1:', self.Type_1_box, 'Type 2:', self.Type_2_box, self.Top_left_sizer)

            #Sets up Box for Item
            self.Item_box = wx.TextCtrl(self, value=this_Pokemon['Item'][0], style=wx.TE_READONLY)
            self.add_widgets('Item:', self.Item_box, self.Top_left_sizer)
            
            #Sets up Box for Image
            self.Image_box = wx.TextCtrl(self, value=this_Pokemon['Image'][0], style=wx.TE_READONLY)
            self.add_widgets('Image:', self.Image_box, self.Top_left_sizer)
            
            #Sets up a Static Text to be used as empty space and adds it to Top_left Sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.Top_left_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            
            #Sets up a Static Text to be used as empty space and adds it to Top_left Sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.Top_left_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Adds Top_left Sizer to Top_left_right Sizer
            self.Top_left_right_sizer.Add(self.Top_left_sizer, 1, wx.EXPAND)
            
            #Sets up a Static Text to be used as empty space and adds it to Top_left_right Sizer
            empty_space_label = wx.StaticText(self, label="    ")
            empty_space_label.SetFont(font_moves)
            self.Top_left_right_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_VERTICAL)
            
            #Adds Top_right Sizer to Top_left_right Sizer
            self.Top_left_right_sizer.Add(self.Top_right_sizer, 1, wx.EXPAND)

            #Adds self.Top_left_right to Main Sizer
            self.main_sizer.Add(self.Top_left_right_sizer, 0, wx.EXPAND)

            #Sets up Boxes for Ability_Name and Ability_Info
            #Sets up a Static Text for Ability and adds it to the main sizer
            Ability_label = wx.StaticText(self, label="Ability")
            Ability_label.SetFont(font_moves)
            self.main_sizer.Add(Ability_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            
            #Sets up Box for Ability_Name
            self.Ability_Name_box = wx.TextCtrl(self, value=this_Pokemon['Ability']['Ability_Name'][0], style=wx.TE_READONLY)
            #Sets up Box for Ability_Info
            self.Ability_Info_box = wx.TextCtrl(self, value=this_Pokemon['Ability']['Ability_Info'][0], style=wx.TE_READONLY | wx.TE_MULTILINE | wx.HSCROLL)
            #self.add_widgets('Ability Info:', self.Ability_Info_box)
            self.add_widgets_4_boxes_horizontal('Name:', self.Ability_Name_box, 'Info:', self.Ability_Info_box, self.main_sizer)         

            # ============================ #  Moves 1 and 2

            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Defines Moves 1 and 2 Sizer
            self.moves_1_2_sizer = wx.BoxSizer(wx.HORIZONTAL)

            #Defines Move 1 Sizer
            self.move_1_sizer = wx.BoxSizer(wx.VERTICAL)

            #Defines Move 2 Sizer
            self.move_2_sizer = wx.BoxSizer(wx.VERTICAL)

            # ============== #  Move 1

            #Sets up a Static Text for Move 1 and adds it to the move 1 sizer
            Move_1_label = wx.StaticText(self, label="Move 1")
            Move_1_label.SetFont(font_moves)
            self.move_1_sizer.Add(Move_1_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Sets up Boxes for Name and Mode
            #Sets up Box for Move 1 Name
            self.Move_1_Name_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Name'][0], style=wx.TE_READONLY)
            
            #Sets up Box for Move 1 Mode
            self.Move_1_Mode_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Mode'][0], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Name:', self.Move_1_Name_box, 'Mode:', self.Move_1_Mode_box, self.move_1_sizer)

            #Sets up Boxes for Type and Method        
            #Sets up Combo Box for Move 1 Type
            self.Move_1_Type_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Type'][0], style=wx.TE_READONLY)
            background_type(self.Move_1_Type_box, this_Pokemon['Moves']['Move_Type'][0])
            #Sets up Static Text for Move 1 Method
            self.Move_1_Method_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Method'][0], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Type:', self.Move_1_Type_box, 'Method:', self.Move_1_Method_box, self.move_1_sizer)

            #Sets up Boxes for Power and Accuracy
            #Sets up Static Text for Move 1 Power
            self.Move_1_Power_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Power'][0], style=wx.TE_READONLY)       
            #Sets up Static Text for Move 1 Accuracy
            self.Move_1_Accuracy_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Accuracy'][0], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Power:', self.Move_1_Power_box, 'Acc.:', self.Move_1_Accuracy_box, self.move_1_sizer)

            #Sets up Boxes for PP and Effect
            #Sets up Static Text for Move 1 PP
            self.Move_1_PP_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_PP'][0], style=wx.TE_READONLY)      
            #Sets up Static Text for Move 1 Effect
            self.Move_1_Effect_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Effect'][0], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('PP:', self.Move_1_PP_box, 'Effect:', self.Move_1_Effect_box, self.move_1_sizer)

            #Sets up Static Text for Move 1 Info
            self.Move_1_Info_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Info'][0], style=wx.TE_READONLY | wx.TE_MULTILINE | wx.HSCROLL)
            self.add_widgets('Info:', self.Move_1_Info_box, self.move_1_sizer)

            # ============== #  Move 2

            #Sets up a Static Text for Move 2 and adds it to the move 2 sizer
            Move_2_label = wx.StaticText(self, label="Move 2")
            Move_2_label.SetFont(font_moves)
            self.move_2_sizer.Add(Move_2_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Sets up Boxes for Name and Mode
            #Sets up Box for Move 2 Name
            self.Move_2_Name_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Name'][1], style=wx.TE_READONLY)
            
            #Sets up Box for Move 2 Mode
            self.Move_2_Mode_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Mode'][1], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Name:', self.Move_2_Name_box, 'Mode:', self.Move_2_Mode_box, self.move_2_sizer)

            #Sets up Boxes for Type and Method        
            #Sets up Combo Box for Move 2 Type
            self.Move_2_Type_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Type'][1], style=wx.TE_READONLY)
            background_type(self.Move_2_Type_box, this_Pokemon['Moves']['Move_Type'][1])
            #Sets up Static Text for Move 2 Method
            self.Move_2_Method_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Method'][1], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Type:', self.Move_2_Type_box, 'Method:', self.Move_2_Method_box, self.move_2_sizer)

            #Sets up Boxes for Power and Accuracy
            #Sets up Static Text for Move 2 Power
            self.Move_2_Power_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Power'][1], style=wx.TE_READONLY)       
            #Sets up Static Text for Move 2 Accuracy
            self.Move_2_Accuracy_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Accuracy'][1], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Power:', self.Move_2_Power_box, 'Acc.:', self.Move_2_Accuracy_box, self.move_2_sizer)

            #Sets up Boxes for PP and Effect
            #Sets up Static Text for Move 2 PP
            self.Move_2_PP_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_PP'][1], style=wx.TE_READONLY)      
            #Sets up Static Text for Move 2 Effect
            self.Move_2_Effect_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Effect'][1], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('PP:', self.Move_2_PP_box, 'Effect:', self.Move_2_Effect_box, self.move_2_sizer)

            #Sets up Static Text for Move 2 Info
            self.Move_2_Info_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Info'][1], style=wx.TE_READONLY | wx.TE_MULTILINE | wx.HSCROLL)
            self.add_widgets('Info:', self.Move_2_Info_box, self.move_2_sizer)

            # ============== # 

            #Adds Move 1 Sizer to Moves 1 and 2 Sizer
            self.moves_1_2_sizer.Add(self.move_1_sizer, 1, wx.EXPAND)

            #Sets up a Static Text to be used as empty space and adds it to  Moves 1 and 2 Sizer
            empty_space_label = wx.StaticText(self, label="    ")
            empty_space_label.SetFont(font_moves)
            self.moves_1_2_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_VERTICAL)

            #Adds Move 2 Sizer to Moves 1 and 2 Sizer
            self.moves_1_2_sizer.Add(self.move_2_sizer, 1, wx.EXPAND)

            #Adds Moves 1 and 2 Sizer to Main Sizer
            self.main_sizer.Add(self.moves_1_2_sizer, 0, wx.EXPAND)   
            
            # ============================ #  Moves 3 and 4

            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Defines Moves 3 and 4 Sizer
            self.moves_3_4_sizer = wx.BoxSizer(wx.HORIZONTAL)

            #Defines Move 3 Sizer
            self.move_3_sizer = wx.BoxSizer(wx.VERTICAL)

            #Defines Move 4 Sizer
            self.move_4_sizer = wx.BoxSizer(wx.VERTICAL)

            # ============== #  Move 3

            #Sets up a Static Text for Move 3 and adds it to the move 3 sizer
            Move_3_label = wx.StaticText(self, label="Move 3")
            Move_3_label.SetFont(font_moves)
            self.move_3_sizer.Add(Move_3_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Sets up Boxes for Name and Mode
            #Sets up Box for Move 3 Name
            self.Move_3_Name_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Name'][2], style=wx.TE_READONLY)
            
            #Sets up Box for Move 3 Mode
            self.Move_3_Mode_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Mode'][2], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Name:', self.Move_3_Name_box, 'Mode:', self.Move_3_Mode_box, self.move_3_sizer)

            #Sets up Boxes for Type and Method        
            #Sets up Combo Box for Move 3 Type
            self.Move_3_Type_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Type'][2], style=wx.TE_READONLY) 
            background_type(self.Move_3_Type_box, this_Pokemon['Moves']['Move_Type'][2])
            #Sets up Static Text for Move 3 Method
            self.Move_3_Method_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Method'][2], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Type:', self.Move_3_Type_box, 'Method:', self.Move_3_Method_box, self.move_3_sizer)

            #Sets up Boxes for Power and Accuracy
            #Sets up Static Text for Move 3 Power
            self.Move_3_Power_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Power'][2], style=wx.TE_READONLY)       
            #Sets up Static Text for Move 3 Accuracy
            self.Move_3_Accuracy_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Accuracy'][2], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Power:', self.Move_3_Power_box, 'Acc.:', self.Move_3_Accuracy_box, self.move_3_sizer)

            #Sets up Boxes for PP and Effect
            #Sets up Static Text for Move 3 PP
            self.Move_3_PP_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_PP'][2], style=wx.TE_READONLY)      
            #Sets up Static Text for Move 3 Effect
            self.Move_3_Effect_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Effect'][2], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('PP:', self.Move_3_PP_box, 'Effect:', self.Move_3_Effect_box, self.move_3_sizer)

            #Sets up Static Text for Move 3 Info
            self.Move_3_Info_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Info'][2], style=wx.TE_READONLY | wx.TE_MULTILINE | wx.HSCROLL)
            self.add_widgets('Info:', self.Move_3_Info_box, self.move_3_sizer)

            # ============== #  Move 4

            #Sets up a Static Text for Move 4 and adds it to the move 4 sizer
            Move_4_label = wx.StaticText(self, label="Move 4")
            Move_4_label.SetFont(font_moves)
            self.move_4_sizer.Add(Move_4_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Sets up Boxes for Name and Mode
            #Sets up Box for Move 4 Name
            self.Move_4_Name_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Name'][3], style=wx.TE_READONLY)
            
            #Sets up Box for Move 4 Mode
            self.Move_4_Mode_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Mode'][3], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Name:', self.Move_4_Name_box, 'Mode:', self.Move_4_Mode_box, self.move_4_sizer)

            #Sets up Boxes for Type and Method        
            #Sets up Combo Box for Move 4 Type
            self.Move_4_Type_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Type'][3], style=wx.TE_READONLY) 
            background_type(self.Move_4_Type_box, this_Pokemon['Moves']['Move_Type'][3])
            #Sets up Static Text for Move 4 Method
            self.Move_4_Method_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Method'][3], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Type:', self.Move_4_Type_box, 'Method:', self.Move_4_Method_box, self.move_4_sizer)

            #Sets up Boxes for Power and Accuracy
            #Sets up Static Text for Move 4 Power
            self.Move_4_Power_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Power'][3], style=wx.TE_READONLY)       
            #Sets up Static Text for Move 4 Accuracy
            self.Move_4_Accuracy_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Accuracy'][3], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('Power:', self.Move_4_Power_box, 'Acc.:', self.Move_4_Accuracy_box, self.move_4_sizer)

            #Sets up Boxes for PP and Effect
            #Sets up Static Text for Move 4 PP
            self.Move_4_PP_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_PP'][3], style=wx.TE_READONLY)      
            #Sets up Static Text for Move 4 Effect
            self.Move_4_Effect_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Effect'][3], style=wx.TE_READONLY)
            self.add_widgets_4_boxes_horizontal('PP:', self.Move_4_PP_box, 'Effect:', self.Move_4_Effect_box, self.move_4_sizer)

            #Sets up Static Text for Move 4 Info
            self.Move_4_Info_box = wx.TextCtrl(self, value=this_Pokemon['Moves']['Move_Info'][3], style=wx.TE_READONLY | wx.TE_MULTILINE | wx.HSCROLL)
            self.add_widgets('Info:', self.Move_4_Info_box, self.move_4_sizer)

            # ============== # 

            #Adds Move 3 Sizer to Moves 3 and 4 Sizer
            self.moves_3_4_sizer.Add(self.move_3_sizer, 1, wx.EXPAND)

            #Sets up a Static Text to be used as empty space and adds it to  Moves 3 and 4 Sizer
            empty_space_label = wx.StaticText(self, label="    ")
            empty_space_label.SetFont(font_moves)
            self.moves_3_4_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_VERTICAL)

            #Adds Move 4 Sizer to Moves 3 and 4 Sizer
            self.moves_3_4_sizer.Add(self.move_4_sizer, 1, wx.EXPAND)

            #Adds Moves 3 and 4 Sizer to Main Sizer
            self.main_sizer.Add(self.moves_3_4_sizer, 0, wx.EXPAND)          
            
            

            # ============== # Battle
            
            #Sets up a Static Text to be used as empty space and adds it to the Global Sizer
            empty_space_label = wx.StaticText(self, label="      ")
            empty_space_label.SetFont(font_moves)
            self.global_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_VERTICAL)
            
            #Defines Battle Sizer, its sub-sizers and adds its sub-sizers to it
            self.battle_sizer = wx.BoxSizer(wx.VERTICAL)
            
            #Sets up a Static Text to be used as empty space and adds it to the battle_sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.battle_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            
            #Sets up a Static Text for Battle Info and adds it to the battle_sizer
            pokemon_label = wx.StaticText(self, label="Battle Info")
            pokemon_label.SetFont(font_moves)
            self.battle_sizer.Add(pokemon_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            
            #Offensive Sizer
            self.offensive_sizer = wx.BoxSizer(wx.VERTICAL)
            self.battle_sizer.Add(self.offensive_sizer, 0, wx.EXPAND) #1, wx.ALL, 5)
            
            #Sets up a Static Texts to be used as empty spaces and adds them to the battle sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.battle_sizer.Add(empty_space_label, 0, wx.EXPAND)
 
            #Defensive Sizer
            self.defensive_sizer = wx.BoxSizer(wx.VERTICAL)
            self.battle_sizer.Add(self.defensive_sizer, 0, wx.EXPAND)
            
            #Adds Battle sizer to Global Sizer
            self.global_sizer.Add(self.battle_sizer, 0, wx.EXPAND)

            # ============== #  Offensive Advantage
            
            #Defines/resets local helpful lists
            self.o_adv_list_times2 = []
            self.o_adv_list_times1 = []
            self.o_adv_list_times05 = []
            self.o_adv_list_times0 = []
            
            #Defines/resets local helpful counters
            self.o_counter = 0
            
            #Compiles all score = 2 from Pokemon['Match']['Offense'] 
            for this_type in this_Pokemon['Type_Match']['All_Types']:
                #print(this_Pokemon['Type_Match']['Offense'])
                #Checks if the score equals 2, meaning offensive advantage
                if this_Pokemon['Type_Match']['Offense'][self.o_counter] == 2:
                    #Adds the type name to helpful list
                    self.o_adv_list_times2.append(this_type)
               #Checks if the score equals 1, meaning regular offensive advantage
                if this_Pokemon['Type_Match']['Offense'][self.o_counter] == 1:
                    #Adds the type name to helpful list
                    self.o_adv_list_times1.append(this_type)
               #Checks if the score equals 0.5, meaning bad offensive advantage
                if this_Pokemon['Type_Match']['Offense'][self.o_counter] == 0.5:
                    #Adds the type name to helpful list
                    self.o_adv_list_times05.append(this_type)
                #Checks if the score equals 0, meaning no offensive advantage
                if this_Pokemon['Type_Match']['Offense'][self.o_counter] == 0:
                    #Adds the type name to helpful list
                    self.o_adv_list_times0.append(this_type)
                    
                #Moves counter to next iteration
                self.o_counter = self.o_counter + 1
            
            #For debugging:
            #print(self.o_adv_list_times2)
            #print(len(self.o_adv_list_times2))           
            

            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.offensive_sizer.Add(empty_space_label, 0, wx.EXPAND)
            
            #Sets up a Static Text for Moves and adds it to the main sizer
            pokemon_label = wx.StaticText(self, label="Offense")
            pokemon_label.SetFont(font_moves)
            self.offensive_sizer.Add(pokemon_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            
            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.offensive_sizer.Add(empty_space_label, 0, wx.EXPAND)
            
            #Sets up a Static Text for labeling Offensive Advantage and adds it to the main sizer
            o_Advantage_label = wx.StaticText(self, label="(2x) Offensive Advantage ["+str(len(self.o_adv_list_times2))+" types]", style=wx.ALIGN_CENTRE_HORIZONTAL)
            o_Advantage_label.SetFont(font_moves)
            self.offensive_sizer.Add(o_Advantage_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            #Calls function to organize and display the types from the list self.o_adv_list_times2
            self.add_widgets_neat_type_display(self.o_adv_list_times2, sizer="Offensive")
            
            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.offensive_sizer.Add(empty_space_label, 0, wx.EXPAND)
            
            #Sets up a Static Text for labeling Offensive Advantage and adds it to the main sizer
            o_Advantage_label = wx.StaticText(self, label="(1x) Offensive Advantage ["+str(len(self.o_adv_list_times1))+" types]", style=wx.ALIGN_CENTRE_HORIZONTAL)
            o_Advantage_label.SetFont(font_moves)
            self.offensive_sizer.Add(o_Advantage_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            #Calls function to organize and display the types from the list self.o_adv_list_times1
            self.add_widgets_neat_type_display(self.o_adv_list_times1, sizer="Offensive")
            
            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.offensive_sizer.Add(empty_space_label, 0, wx.EXPAND)
            
            #Sets up a Static Text for labeling Offensive Advantage and adds it to the main sizer
            o_Advantage_label = wx.StaticText(self, label="(0.5x) Offensive Advantage ["+str(len(self.o_adv_list_times05))+" types]", style=wx.ALIGN_CENTRE_HORIZONTAL)
            o_Advantage_label.SetFont(font_moves)
            self.offensive_sizer.Add(o_Advantage_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            #Calls function to organize and display the types from the list self.o_adv_list_times05
            self.add_widgets_neat_type_display(self.o_adv_list_times05, sizer="Offensive")
            
            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.offensive_sizer.Add(empty_space_label, 0, wx.EXPAND)
            
            #Sets up a Static Text for labeling Offensive Advantage and adds it to the main sizer
            o_Advantage_label = wx.StaticText(self, label="(0x) Offensive Advantage ["+str(len(self.o_adv_list_times0))+" types]", style=wx.ALIGN_CENTRE_HORIZONTAL)
            o_Advantage_label.SetFont(font_moves)
            self.offensive_sizer.Add(o_Advantage_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            #Calls function to organize and display the types from the list self.o_adv_list_times0
            self.add_widgets_neat_type_display(self.o_adv_list_times0, sizer="Offensive")

            # ============== #  Defensive Disadvantage
            
            #Defines/resets local helpful lists
            self.d_davt_list_times2 = []
            self.d_davt_list_times4 = []
            self.d_davt_list_times1 = []
            self.d_davt_list_times05 = []
            self.d_davt_list_times025 = []
            self.d_davt_list_times0 = []
            
            #Defines/resets local helpful counters
            self.d_counter = 0
            
            #Compiles all score = 2 from Pokemon['Match']['Defense'] 
            for this_type in this_Pokemon['Type_Match']['All_Types']:
                #print(this_Pokemon['Type_Match']['Defense'])
                #Checks if the score equals 2 (or more due to Ability Modifiers), meaning defensive disadvantage
                if this_Pokemon['Type_Match']['Defense'][self.d_counter] >= 2 and this_Pokemon['Type_Match']['Defense'][self.d_counter] < 4:
                    #Adds the type name to helpful list
                    self.d_davt_list_times2.append(this_type)
                #Checks if the score equals 4 (or more due to Ability Modifiers), meaning severe defensive disadvantage
                if this_Pokemon['Type_Match']['Defense'][self.d_counter] >= 4:
                    #Adds the type name to helpful list
                    self.d_davt_list_times4.append(this_type)
                #Checks if the score equals 1 (or more due to Ability Modifiers), meaning regular defensive disadvantage
                if this_Pokemon['Type_Match']['Defense'][self.d_counter] >= 1 and this_Pokemon['Type_Match']['Defense'][self.d_counter] <2:
                    #Adds the type name to helpful list
                    self.d_davt_list_times1.append(this_type)
                #Checks if the score equals 0.5 (or more due to Ability Modifiers), meaning light defensive disadvantage
                if this_Pokemon['Type_Match']['Defense'][self.d_counter] >= 0.5 and this_Pokemon['Type_Match']['Defense'][self.d_counter] < 1:
                    #Adds the type name to helpful list
                    self.d_davt_list_times05.append(this_type)
                #Checks if the score equals 0.25 (or around due to Ability Modifiers), meaning very little defensive disadvantage
                if this_Pokemon['Type_Match']['Defense'][self.d_counter] > 0 and this_Pokemon['Type_Match']['Defense'][self.d_counter] < 0.5:
                    #Adds the type name to helpful list
                    self.d_davt_list_times025.append(this_type)
                #Checks if the score equals 0, meaning no defensive disadvantage
                if this_Pokemon['Type_Match']['Defense'][self.d_counter] == 0:
                    #Adds the type name to helpful list
                    self.d_davt_list_times0.append(this_type)
                    
                #Moves counter to next iteration
                self.d_counter = self.d_counter + 1
            
            #For debugging:
            #print(self.d_davt_list_times2)
            #print(len(self.d_davt_list_times2))
            #print(self.d_davt_list_times4)
            #print(len(self.d_davt_list_times4))            
            
            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.defensive_sizer.Add(empty_space_label, 0, wx.EXPAND)
            
            #Sets up a Static Text for Moves and adds it to the main sizer
            pokemon_label = wx.StaticText(self, label="Defense")
            pokemon_label.SetFont(font_moves)
            self.defensive_sizer.Add(pokemon_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            
            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.defensive_sizer.Add(empty_space_label, 0, wx.EXPAND)
            
            #Sets up a Static Text for labeling Defensive Disadvantage (x4) and adds it to the main sizer
            d_Disadvantage_times4_label = wx.StaticText(self, label="(4x) Defensive Disadvantage ["+str(len(self.d_davt_list_times4))+" types]", style=wx.ALIGN_CENTRE_HORIZONTAL)
            d_Disadvantage_times4_label.SetFont(font_moves)
            self.defensive_sizer.Add(d_Disadvantage_times4_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            #Calls function to organize and display the types from the list self.d_davt_list_times4
            self.add_widgets_neat_type_display(self.d_davt_list_times4, sizer="Defensive")    
        
            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.defensive_sizer.Add(empty_space_label, 0, wx.EXPAND)     
            
            #Sets up a Static Text for labeling Defensive Disadvantage (x2) and adds it to the main sizer
            d_Disadvantage_times2_label = wx.StaticText(self, label="(2x) Defensive Disadvantage ["+str(len(self.d_davt_list_times2))+" types]", style=wx.ALIGN_CENTRE_HORIZONTAL)
            d_Disadvantage_times2_label.SetFont(font_moves)
            self.defensive_sizer.Add(d_Disadvantage_times2_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            #Calls function to organize and display the types from the list self.d_davt_list_times2
            self.add_widgets_neat_type_display(self.d_davt_list_times2, sizer="Defensive")
            
            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.defensive_sizer.Add(empty_space_label, 0, wx.EXPAND)     
            
            #Sets up a Static Text for labeling Defensive Disadvantage (x1) and adds it to the main sizer
            d_Disadvantage_times1_label = wx.StaticText(self, label="(1x) Defensive Disadvantage ["+str(len(self.d_davt_list_times1))+" types]", style=wx.ALIGN_CENTRE_HORIZONTAL)
            d_Disadvantage_times1_label.SetFont(font_moves)
            self.defensive_sizer.Add(d_Disadvantage_times1_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            #Calls function to organize and display the types from the list self.d_davt_list_times1
            self.add_widgets_neat_type_display(self.d_davt_list_times1, sizer="Defensive")
            
            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.defensive_sizer.Add(empty_space_label, 0, wx.EXPAND)     
            
            #Sets up a Static Text for labeling Defensive Disadvantage (x0.5) and adds it to the main sizer
            d_Disadvantage_times05_label = wx.StaticText(self, label="(0.5x) Defensive Disadvantage ["+str(len(self.d_davt_list_times05))+" types]", style=wx.ALIGN_CENTRE_HORIZONTAL)
            d_Disadvantage_times05_label.SetFont(font_moves)
            self.defensive_sizer.Add(d_Disadvantage_times05_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            #Calls function to organize and display the types from the list self.d_davt_list_times05
            self.add_widgets_neat_type_display(self.d_davt_list_times05, sizer="Defensive")          

            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.defensive_sizer.Add(empty_space_label, 0, wx.EXPAND)     
            
            #Sets up a Static Text for labeling Defensive Disadvantage (x0.25) and adds it to the main sizer
            d_Disadvantage_times025_label = wx.StaticText(self, label="(0.25x) Defensive Disadvantage ["+str(len(self.d_davt_list_times025))+" types]", style=wx.ALIGN_CENTRE_HORIZONTAL)
            d_Disadvantage_times025_label.SetFont(font_moves)
            self.defensive_sizer.Add(d_Disadvantage_times025_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            #Calls function to organize and display the types from the list self.d_davt_list_times025
            self.add_widgets_neat_type_display(self.d_davt_list_times025, sizer="Defensive")
            
            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.defensive_sizer.Add(empty_space_label, 0, wx.EXPAND)
                                                      
            #Sets up a Static Text for labeling Defensive Disadvantage (x0) and adds it to the main sizer
            d_Disadvantage_times0_label = wx.StaticText(self, label="(0x) Defensive Disadvantage ["+str(len(self.d_davt_list_times0))+" types]", style=wx.ALIGN_CENTRE_HORIZONTAL)
            d_Disadvantage_times0_label.SetFont(font_moves)
            self.defensive_sizer.Add(d_Disadvantage_times0_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            #Calls function to organize and display the types from the list self.d_davt_list_times0
            self.add_widgets_neat_type_display(self.d_davt_list_times0, sizer="Defensive")
            
            # ============== # Bottom Buttons
            
            #Sets up a Static Text to be used as empty space and adds it to the global_and_buttons_sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.global_and_buttons_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Sets up a Static Text to be used as empty space and adds it to the global_and_buttons_sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.global_and_buttons_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Defines Button Sizer
            btn_sizer = wx.BoxSizer()
            #Creates Close Button
            close_btn = wx.Button(self, label='Close')
            #Binds Button event to Close Button
            close_btn.Bind(wx.EVT_BUTTON, self.on_close_view_pokemon)
            #Adds Close Button to Button Sizer
            btn_sizer.Add(close_btn, 0, wx.ALL, 5)
            #Adds Button Sizer to global_and_buttons_sizer
            self.global_and_buttons_sizer.Add(btn_sizer, 0, wx.CENTER)

            #Sets global_and_buttons_sizer for user to see
            self.SetSizer(self.global_and_buttons_sizer)
              
     # ====================================== # Organizing layout   
    
    #This function helps position each box (Text Control, Combo Box, Radio Box) to entry pokemon information
    def add_widgets(self, label_text, entry_box, this_sizer):
        #Defines font style for widgets statict text
        font_labels = wx.Font(12, wx.DECORATIVE, wx.BOLD, wx.NORMAL)   
        #Defines Row Sizer for each "Line"
        row_sizer = wx.BoxSizer(wx.HORIZONTAL)
        #Uses Static Text to Label the boxes
        label = wx.StaticText(self, label=label_text, size=(60, -1))
        #Sets font style to label
        label.SetFont(font_labels)        
        #Adds Static Text Label to Row Sizer
        row_sizer.Add(label, 0, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Adds Box to add information to row sizer
        row_sizer.Add(entry_box, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Adds Row Sizer to Provided Sizer
        this_sizer.Add(row_sizer, 0, wx.EXPAND)
        #self.main_sizer.Add(row_sizer, 0, wx.EXPAND)
        
    #This function helps position each box (Text Control, Combo Box, Radio Box) to entry pokemon information
    def add_widgets_4_boxes_horizontal(self, text_1, box_1, text_2, box_2, this_sizer):
        #Defines font style for widgets statict text
        font_labels = wx.Font(12, wx.DECORATIVE, wx.BOLD, wx.NORMAL)
        #Uses Static Text to Label using text_1
        label_1 = wx.StaticText(self, label=text_1, size=(60, -1))
        #Sets font style to text_1
        label_1.SetFont(font_labels) 
        #Uses Static Text to Label using text_2
        label_2 = wx.StaticText(self, label=text_2, size=(60, -1))
        #Sets font style to label
        label_2.SetFont(font_labels) 
        #Defines Row Sizer for each "Line"
        row_sizer = wx.BoxSizer(wx.HORIZONTAL)        
        #Adds text_1 to Row Sizer
        row_sizer.Add(label_1, 0, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)        
        #Adds box_1 to Row Sizer
        row_sizer.Add(box_1, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Sets up and adds empty space
        empty_space_label = wx.StaticText(self, label="    ")
        row_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_VERTICAL)
        #Adds text_2 to Row Sizer
        row_sizer.Add(label_2, 0, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5) 
        #Adds box_2 to row sizer
        row_sizer.Add(box_2, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Adds Row Sizer to Provided Sizer
        this_sizer.Add(row_sizer, 0, wx.EXPAND)
        #self.main_sizer.Add(row_sizer, 0, wx.EXPAND)
    
    #This function helps position each box (Text Control, Combo Box, Radio Box) to entry pokemon information
    def add_widgets_2_boxes_horizontal(self, box_1, box_2):
      
        #Defines Row Sizer for each "Line"
        row_sizer = wx.BoxSizer(wx.HORIZONTAL)
        #Adds box_1 to Row Sizer
        row_sizer.Add(box_1, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Sets up and adds empty space
        empty_space_label = wx.StaticText(self, label="")
        row_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_VERTICAL)
        #Adds box_2 to row sizer
        row_sizer.Add(box_2, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Adds Row Sizer to Main Sizer
        self.main_sizer.Add(row_sizer, 0, wx.EXPAND)
        
    #This function helps position each box (Text Control, Combo Box, Radio Box) to entry pokemon information
    def add_widgets_3_boxes_horizontal(self, box_1, box_2, box_3):
      
        #Defines Row Sizer for each "Line"
        row_sizer = wx.BoxSizer(wx.HORIZONTAL)
        #Adds box_1 to Row Sizer
        row_sizer.Add(box_1, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Adds box_2 to row sizer
        row_sizer.Add(box_2, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Adds box_3 to Row Sizer
        row_sizer.Add(box_3, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Adds Row Sizer to Main Sizer
        self.main_sizer.Add(row_sizer, 0, wx.EXPAND)
        
    #This function helps position each box (Text Control, Combo Box, Radio Box) to entry pokemon information
    def add_widgets_neat_type_display(self, type_list, sizer):
        
        #Transfers list to a local variable
        self.this_list = type_list

        #Cheks how many types are in the list
        type_number = len(self.this_list)
        
        #Defines Row Sizers for each "Line" according to total number of types in list
        row_sizer_1 = wx.BoxSizer(wx.HORIZONTAL)
        if type_number > 5:
            row_sizer_2 = wx.BoxSizer(wx.HORIZONTAL)
            
            if type_number > 10:
                row_sizer_3 = wx.BoxSizer(wx.HORIZONTAL)
                
                if type_number > 15:
                    row_sizer_4 = wx.BoxSizer(wx.HORIZONTAL)
        
        
        #Defines/resets helpful counter
        this_list_counter = 0
        
        #Starts to go through the list
        for types in self.this_list:
            #Until the 5th type:
            if this_list_counter <5:
            
                #Sets up read only TextCtrl box for current type
                self.Type_box = wx.TextCtrl(self, value=str(types), style=wx.TE_READONLY | wx.TE_CENTRE)
                #Calls function to color background according to type from pokemon_functions.py
                background_type(self.Type_box, str(types))
                #Adds type TextCtrl box to Row Sizer
                row_sizer_1.Add(self.Type_box, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
                
            #From the 5th to the 10th type:
            if this_list_counter >= 5 and this_list_counter < 10:
                
                #Sets up read only TextCtrl box for current type
                self.Type_box = wx.TextCtrl(self, value=str(types), style=wx.TE_READONLY | wx.TE_CENTRE)
                #Calls function to color background according to type from pokemon_functions.py
                background_type(self.Type_box, str(types))
                #Adds type TextCtrl box to Row Sizer
                row_sizer_2.Add(self.Type_box, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
                
            #From the 10th to the 15th type:
            if this_list_counter >= 10 and this_list_counter < 15:
                
                #Sets up read only TextCtrl box for current type
                self.Type_box = wx.TextCtrl(self, value=str(types), style=wx.TE_READONLY | wx.TE_CENTRE)
                #Calls function to color background according to type from pokemon_functions.py
                background_type(self.Type_box, str(types))
                #Adds type TextCtrl box to Row Sizer
                row_sizer_3.Add(self.Type_box, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
                
            #From the 15th to the 18th type:
            if this_list_counter >= 15 and this_list_counter < 18:
                
                #Sets up read only TextCtrl box for current type
                self.Type_box = wx.TextCtrl(self, value=str(types), style=wx.TE_READONLY | wx.TE_CENTRE)
                #Calls function to color background according to type from pokemon_functions.py
                background_type(self.Type_box, str(types))
                #Adds type TextCtrl box to Row Sizer
                row_sizer_4.Add(self.Type_box, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
            
            #Moves counter to next iteration
            this_list_counter = this_list_counter + 1
        
        #Adds Row Sizers to Main Sizer according to number of types in list and the provided sizer
        if sizer == "Offensive":
            self.offensive_sizer.Add(row_sizer_1, 0, wx.EXPAND)
            if type_number > 5:
                self.offensive_sizer.Add(row_sizer_2, 0, wx.EXPAND)

                if type_number > 10:
                    self.offensive_sizer.Add(row_sizer_3, 0, wx.EXPAND)

                    if type_number > 15:
                        self.offensive_sizer.Add(row_sizer_4, 0, wx.EXPAND)
                        
        if sizer == "Defensive":
            self.defensive_sizer.Add(row_sizer_1, 0, wx.EXPAND)
            if type_number > 5:
                self.defensive_sizer.Add(row_sizer_2, 0, wx.EXPAND)

                if type_number > 10:
                    self.defensive_sizer.Add(row_sizer_3, 0, wx.EXPAND)

                    if type_number > 15:
                        self.defensive_sizer.Add(row_sizer_4, 0, wx.EXPAND)
        
        
        
    # ====================================== # When close button is pressed
        
    #This function is called when the close button is pressed. It closes the ViewPokemonDialog Box
    def on_close_view_pokemon(self, event):

        #Closes ViewPokemonDialog Box
        self.Destroy()

# ======================================================================================= # Preview Team Entry
        
#Defines PreView Pokemon Entry as a Dialog Box
class PreViewTeamDialog(wx.Dialog):
            
    def __init__(self, team, preview):
        
        if preview == 'yes':
            this_title = 'Preview Team Entry'
        else:
            this_title = 'View Team Entry'
                        
        super().__init__(parent=None, title=this_title, style = wx.DEFAULT_DIALOG_STYLE | wx.RESIZE_BORDER, size = (615,1000))   
        
        #Stores the provided team dictionary into a local dictionary so button events can access it
        self.this_team = team
        
        #Defines Main Sizer
        self.main_sizer = wx.BoxSizer(wx.VERTICAL)
        
        #Defines font style for the statict text that labels the Team Members and Team Info titles
        font_moves = wx.Font(12, wx.DECORATIVE, wx.BOLD, wx.NORMAL)
        
        #Sets up a Static Text to be used as empty space and adds it to the main sizer
        empty_space_label = wx.StaticText(self, label="")
        empty_space_label.SetFont(font_moves)
        self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
        
        #Sets up a Static Text for Team Members and adds it to the main sizer
        Team_Members_label = wx.StaticText(self, label="Team")
        Team_Members_label.SetFont(font_moves)
        self.main_sizer.Add(Team_Members_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
        
        #Sets up Box for Name
        self.Name_box = wx.TextCtrl(self, value=str(self.this_team['Team_Name'][0]), style=wx.TE_READONLY | wx.TE_CENTRE)
        self.main_sizer.Add(self.Name_box, 0, wx.ALIGN_CENTRE_HORIZONTAL)
        
        #Sets up Subtle View Button for Pokemon_1
        self.Pokemon_1_btn = wx.Button(self, label=str(self.this_team['Members']['Pokemon_1']['Name'][0]))
        #Binds Button event to Subtle View Button
        self.Pokemon_1_btn.Bind(wx.EVT_BUTTON, self.on_view_pokemon_team)
        
        #Sets up Subtle View Button for Pokemon_2
        self.Pokemon_2_btn = wx.Button(self, label=str(self.this_team['Members']['Pokemon_2']['Name'][0]))
        #Binds Button event to Subtle View Button
        self.Pokemon_2_btn.Bind(wx.EVT_BUTTON, self.on_view_pokemon_team)
        
        #Sets up Subtle View Button for Pokemon_3
        self.Pokemon_3_btn = wx.Button(self, label=str(self.this_team['Members']['Pokemon_3']['Name'][0]))
        #Binds Button event to Subtle View Button
        self.Pokemon_3_btn.Bind(wx.EVT_BUTTON, self.on_view_pokemon_team)
        
        #Sets up Subtle View Button for Pokemon_4
        self.Pokemon_4_btn = wx.Button(self, label=str(self.this_team['Members']['Pokemon_4']['Name'][0]))
        #Binds Button event to Subtle View Button
        self.Pokemon_4_btn.Bind(wx.EVT_BUTTON, self.on_view_pokemon_team)
        
        #Sets up Subtle View Button for Pokemon_5
        self.Pokemon_5_btn = wx.Button(self, label=str(self.this_team['Members']['Pokemon_5']['Name'][0]))
        #Binds Button event to Subtle View Button
        self.Pokemon_5_btn.Bind(wx.EVT_BUTTON, self.on_view_pokemon_team)
        
        #Sets up Subtle View Button for Pokemon_6
        self.Pokemon_6_btn = wx.Button(self, label=str(self.this_team['Members']['Pokemon_6']['Name'][0]))
        #Binds Button event to Subtle View Button
        self.Pokemon_6_btn.Bind(wx.EVT_BUTTON, self.on_view_pokemon_team)
        
        #Adjusts layout for all 6 pokemon
        self.add_widgets_pokemon_name_btns(self.Pokemon_1_btn, self.Pokemon_2_btn, self.Pokemon_3_btn, self.Pokemon_4_btn, self.Pokemon_5_btn, self.Pokemon_6_btn)
        
        #Sets up a Static Text to be used as empty space and adds it to the main sizer
        empty_space_label = wx.StaticText(self, label="")
        empty_space_label.SetFont(font_moves)
        self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
        
        #Sets up a Static Text to be used as empty space and adds it to the main sizer
        empty_space_label = wx.StaticText(self, label="")
        empty_space_label.SetFont(font_moves)
        self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
        
        #Sets up a Static Text for Team Info and adds it to the main sizer
        Team_Info_label = wx.StaticText(self, label="Team Info")
        Team_Info_label.SetFont(font_moves)
        self.main_sizer.Add(Team_Info_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
        
        #Sets up Boxes for Physical Attackers (READONLY)
        self.Physical_Attackers_box = wx.TextCtrl(self, value=str(self.this_team['Stats']['Physical_Moves']).strip('][').replace("'",""), style=wx.TE_READONLY)
        self.add_widgets('Physical Attackers:', self.Physical_Attackers_box)
        
        #Sets up Box for Special Attackers (READONLY), starting empty
        self.Special_Attackers_box = wx.TextCtrl(self, value=str(self.this_team['Stats']['Special_Moves']).strip('][').replace("'",""), style=wx.TE_READONLY)
        self.add_widgets('Special Attackers:', self.Special_Attackers_box)
        
        #Sets up Box for Status Attackers (READONLY), starting empty
        self.Status_Attackers_box = wx.TextCtrl(self, value=str(self.this_team['Stats']['Status_Moves']).strip('][').replace("'",""), style=wx.TE_READONLY)
        self.add_widgets('Status Attackers:', self.Status_Attackers_box)
        
        #Sets up Box for Repeated Items (READONLY), starting empty
        self.Repeated_Items_box = wx.TextCtrl(self, value=str(self.this_team['Stats']['Repeated_Items']).strip('][').replace("'",""), style=wx.TE_READONLY)
        self.add_widgets('Repeated Items:', self.Repeated_Items_box)

        # ============== # Battle

        #Sets up a Static Text to be used as empty space and adds it to the main sizer
        empty_space_label = wx.StaticText(self, label="")
        empty_space_label.SetFont(font_moves)
        self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

        #Sets up a Static Text to be used as empty space and adds it to the main sizer
        empty_space_label = wx.StaticText(self, label="")
        empty_space_label.SetFont(font_moves)
        self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

        #Sets up a Static Text for Moves and adds it to the main sizer
        pokemon_label = wx.StaticText(self, label="Battle")
        pokemon_label.SetFont(font_moves)
        self.main_sizer.Add(pokemon_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

        # ============== #  Offensive Advantage

        #Sets up a Static Text to be used as empty space and adds it to the main sizer
        empty_space_label = wx.StaticText(self, label="")
        empty_space_label.SetFont(font_moves)
        self.main_sizer.Add(empty_space_label, 0, wx.EXPAND)

        #Defines/resets local helpful lists
        self.o_adv_list_times2 = []
        self.o_adv_list_times1 = []
        self.o_adv_list_times05 = []
        self.o_adv_list_times0 = []

        #Defines/resets local helpful counters
        self.o_counter = 0

        #Compiles scores from self.this_team['Strengths'] into separate lists
        for this_type in self.this_team['Members']['Pokemon_1']['Type_Match']['All_Types']:
            #print(self.this_team['Strengths'])
            #Checks if the score can be approximated to 2, meaning offensive advantage
            if self.this_team['Strengths'][self.o_counter] > 1.5 and self.this_team['Strengths'][self.o_counter] <= 2:
                #Adds the type name to helpful list
                self.o_adv_list_times2.append(this_type)
           #Checks if the score can be approximated to 1, meaning regular offensive advantage
            if self.this_team['Strengths'][self.o_counter] > 0.5 and self.this_team['Strengths'][self.o_counter] <= 1.5:
                #Adds the type name to helpful list
                self.o_adv_list_times1.append(this_type)
           #Checks if the score can be approximated to 0.5, meaning bad offensive advantage
            if self.this_team['Strengths'][self.o_counter] > 0 and self.this_team['Strengths'][self.o_counter] <= 0.5:
                #Adds the type name to helpful list
                self.o_adv_list_times05.append(this_type)
            #Checks if the score equals 0, meaning no offensive advantage
            if self.this_team['Strengths'][self.o_counter] == 0:
                #Adds the type name to helpful list
                self.o_adv_list_times0.append(this_type)

            #Moves counter to next iteration
            self.o_counter = self.o_counter + 1

        #For debugging:
        #print(self.o_adv_list_times2)
        #print(len(self.o_adv_list_times2))

        #Sets up a Static Text for labeling Offensive Advantage and adds it to the main sizer
        o_Advantage_label = wx.StaticText(self, label="(2x) Offensive Advantage ["+str(len(self.o_adv_list_times2))+" types]", style=wx.ALIGN_CENTRE_HORIZONTAL)
        o_Advantage_label.SetFont(font_moves)
        self.main_sizer.Add(o_Advantage_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

        #Calls function to organize and display the types from the list self.o_adv_list_times2
        self.add_widgets_neat_type_display(self.o_adv_list_times2)

        # ============== #  Defensive Disadvantage

        #Sets up a Static Text to be used as empty space and adds it to the main sizer
        empty_space_label = wx.StaticText(self, label="")
        empty_space_label.SetFont(font_moves)
        self.main_sizer.Add(empty_space_label, 0, wx.EXPAND)

        #Defines/resets local helpful lists
        self.d_davt_list_times2 = []
        self.d_davt_list_times4 = []
        self.d_davt_list_times1 = []
        self.d_davt_list_times05 = []
        self.d_davt_list_times025 = []
        self.d_davt_list_times0 = []

        #Defines/resets local helpful counters
        self.d_counter = 0

        #Compiles all scores from self.this_team['Weaknesses'] into separate lists
        for this_type in self.this_team['Members']['Pokemon_1']['Type_Match']['All_Types']:
            #print(self.this_team['Weaknesses'])
            #Checks if the score can be approximated to 2, meaning defensive disadvantage
            if self.this_team['Weaknesses'][self.d_counter] >= 1.5 and self.this_team['Weaknesses'][self.d_counter] < 2.5:
                #Adds the type name to helpful list
                self.d_davt_list_times2.append(this_type)
            #Checks if the score can be approximated to 4, meaning severe defensive disadvantage
            if self.this_team['Weaknesses'][self.d_counter] >= 2.5 and self.this_team['Weaknesses'][self.d_counter] <= 4:
                #Adds the type name to helpful list
                self.d_davt_list_times4.append(this_type)
            #Checks if the score can be approximated to 1, meaning regular defensive disadvantage
            if self.this_team['Weaknesses'][self.d_counter] > 0.5 and self.this_team['Weaknesses'][self.d_counter] < 1.5:
                #Adds the type name to helpful list
                self.d_davt_list_times1.append(this_type)
            #Checks if the score can be approximated to 0.5, meaning light defensive disadvantage
            if self.this_team['Weaknesses'][self.d_counter] > 0.25 and self.this_team['Weaknesses'][self.d_counter] <= 0.5:
                #Adds the type name to helpful list
                self.d_davt_list_times05.append(this_type)
            #Checks if the score can be approximated to 0.25, meaning very little defensive disadvantage
            if self.this_team['Weaknesses'][self.d_counter] > 0 and self.this_team['Weaknesses'][self.d_counter] <= 0.25:
                #Adds the type name to helpful list
                self.d_davt_list_times025.append(this_type)
            #Checks if the score equals 0, meaning no defensive disadvantage
            if self.this_team['Weaknesses'][self.d_counter] == 0:
                #Adds the type name to helpful list
                self.d_davt_list_times0.append(this_type)

            #Moves counter to next iteration
            self.d_counter = self.d_counter + 1

        #For debugging:
        #print(self.d_davt_list_times2)
        #print(len(self.d_davt_list_times2))
        #print(self.d_davt_list_times4)
        #print(len(self.d_davt_list_times4))            

        #Sets up a Static Text for labeling Defensive Disadvantage (x2) and adds it to the main sizer
        d_Disadvantage_times2_label = wx.StaticText(self, label="(2x) Defensive Disadvantage ["+str(len(self.d_davt_list_times2))+" types]", style=wx.ALIGN_CENTRE_HORIZONTAL)
        d_Disadvantage_times2_label.SetFont(font_moves)
        self.main_sizer.Add(d_Disadvantage_times2_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

        #Calls function to organize and display the types from the list self.d_davt_list_times2
        self.add_widgets_neat_type_display(self.d_davt_list_times2)

        #Sets up a Static Text to be used as empty space and adds it to the main sizer
        empty_space_label = wx.StaticText(self, label="")
        empty_space_label.SetFont(font_moves)
        self.main_sizer.Add(empty_space_label, 0, wx.EXPAND)

        #Sets up a Static Text for labeling Defensive Disadvantage (x4) and adds it to the main sizer
        d_Disadvantage_times4_label = wx.StaticText(self, label="(4x) Defensive Disadvantage ["+str(len(self.d_davt_list_times4))+" types]", style=wx.ALIGN_CENTRE_HORIZONTAL)
        d_Disadvantage_times4_label.SetFont(font_moves)
        self.main_sizer.Add(d_Disadvantage_times4_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

        #Calls function to organize and display the types from the list self.d_davt_list_times4
        self.add_widgets_neat_type_display(self.d_davt_list_times4)        
                
        #Sets up a Static Text to be used as empty space and adds it to the main sizer
        empty_space_label = wx.StaticText(self, label="")
        empty_space_label.SetFont(font_moves)
        self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
        
        #Sets up a Static Text to be used as empty space and adds it to the main sizer
        empty_space_label = wx.StaticText(self, label="")
        empty_space_label.SetFont(font_moves)
        self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
        
        # ====================== #  Bottom Buttons
        
        #Defines Button Sizer
        btn_sizer = wx.BoxSizer()
        if preview == 'yes':
            #Creates Save Button (only if the window is working as a preview window)
            save_btn = wx.Button(self, label='Save')
            #Binds Button event to Save Button
            save_btn.Bind(wx.EVT_BUTTON, self.on_save_team)
            #Adds Save Button to Button Sizer
            btn_sizer.Add(save_btn, 0, wx.ALL, 5)
        #Creates More Button
        more_btn = wx.Button(self, label='More')
        #Binds Button event to More Button
        more_btn.Bind(wx.EVT_BUTTON, self.on_more_view_team)
        #Adds More Button to Button Sizer
        btn_sizer.Add(more_btn, 0, wx.ALL, 5)
        #Creates Close Button
        close_btn = wx.Button(self, label='Close')
        #Binds Button event to Close Button
        close_btn.Bind(wx.EVT_BUTTON, self.on_close_preview_team)
        #Adds Close Button to Button Sizer
        btn_sizer.Add(close_btn, 0, wx.ALL, 5)
        #Adds Button Sizer to Main Sizer
        self.main_sizer.Add(btn_sizer, 0, wx.CENTER)
        
        #Sets Main Sizer for user to see
        self.SetSizer(self.main_sizer)
              
     # ====================================== # Organizing layout   
    
    #This function helps position each box (Text Control, Combo Box, Radio Box) to entry information
    def add_widgets(self, label_text, entry_box):
        #Defines font style for widgets statict text
        font_labels = wx.Font(12, wx.DECORATIVE, wx.BOLD, wx.NORMAL)   
        #Defines Row Sizer for each "Line"
        row_sizer = wx.BoxSizer(wx.HORIZONTAL)
        #Uses Static Text to Label the boxes
        label = wx.StaticText(self, label=label_text, size=(135, -1))
        #Sets font style to label
        label.SetFont(font_labels)        
        #Adds Static Text Label to Row Sizer
        row_sizer.Add(label, 0, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Adds Box to add information to row sizer
        row_sizer.Add(entry_box, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Adds Row Sizer to Main Sizer
        self.main_sizer.Add(row_sizer, 0, wx.EXPAND)
        
    #This function helps position subtle button containing the pokemon names
    def add_widgets_pokemon_name_btns(self, btn_1, btn_2, btn_3, btn_4, btn_5, btn_6):
        #Defines 1st Row Sizer for each "Line"
        row_sizer_1 = wx.BoxSizer(wx.HORIZONTAL)
        #Adds first 3 buttons to 1st row sizer
        row_sizer_1.Add(btn_1, 0, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        row_sizer_1.Add(btn_2, 0, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        row_sizer_1.Add(btn_3, 0, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Adds 1st Row Sizer to Main Sizer
        self.main_sizer.Add(row_sizer_1, 0, wx.ALL | wx.ALIGN_CENTRE_HORIZONTAL)
        #Defines 2nd Row Sizer for each "Line"
        row_sizer_2 = wx.BoxSizer(wx.HORIZONTAL)
        #Adds last 3 buttons to 2nd row sizer
        row_sizer_2.Add(btn_4, 0, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        row_sizer_2.Add(btn_5, 0, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        row_sizer_2.Add(btn_6, 0, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Adds 2nd Row Sizer to Main Sizer
        self.main_sizer.Add(row_sizer_2, 0, wx.ALL | wx.ALIGN_CENTRE_HORIZONTAL)
        
    #This function helps position each box (Text Control, Combo Box, Radio Box) to entry pokemon information
    def add_widgets_neat_type_display(self, type_list):
        
        #Transfers list to a local variable
        self.this_list = type_list

        #Cheks how many types are in the list
        type_number = len(self.this_list)
        
        #Defines Row Sizers for each "Line" according to total number of types in list
        row_sizer_1 = wx.BoxSizer(wx.HORIZONTAL)
        if type_number > 5:
            row_sizer_2 = wx.BoxSizer(wx.HORIZONTAL)
            
            if type_number > 10:
                row_sizer_3 = wx.BoxSizer(wx.HORIZONTAL)
                
                if type_number > 15:
                    row_sizer_4 = wx.BoxSizer(wx.HORIZONTAL)
        
        
        #Defines/resets helpful counter
        this_list_counter = 0
        
        #Starts to go through the list
        for types in self.this_list:
            #Until the 5th type:
            if this_list_counter <5:
            
                #Sets up read only TextCtrl box for current type
                self.Type_box = wx.TextCtrl(self, value=str(types), style=wx.TE_READONLY | wx.TE_CENTRE)
                #Calls function to color background according to type from pokemon_functions.py
                background_type(self.Type_box, str(types))
                #Adds type TextCtrl box to Row Sizer
                row_sizer_1.Add(self.Type_box, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
                
            #From the 5th to the 10th type:
            if this_list_counter >= 5 and this_list_counter < 10:
                
                #Sets up read only TextCtrl box for current type
                self.Type_box = wx.TextCtrl(self, value=str(types), style=wx.TE_READONLY | wx.TE_CENTRE)
                #Calls function to color background according to type from pokemon_functions.py
                background_type(self.Type_box, str(types))
                #Adds type TextCtrl box to Row Sizer
                row_sizer_2.Add(self.Type_box, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
                
            #From the 10th to the 15th type:
            if this_list_counter >= 10 and this_list_counter < 15:
                
                #Sets up read only TextCtrl box for current type
                self.Type_box = wx.TextCtrl(self, value=str(types), style=wx.TE_READONLY | wx.TE_CENTRE)
                #Calls function to color background according to type from pokemon_functions.py
                background_type(self.Type_box, str(types))
                #Adds type TextCtrl box to Row Sizer
                row_sizer_3.Add(self.Type_box, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
                
            #From the 15th to the 18th type:
            if this_list_counter >= 15 and this_list_counter < 18:
                
                #Sets up read only TextCtrl box for current type
                self.Type_box = wx.TextCtrl(self, value=str(types), style=wx.TE_READONLY | wx.TE_CENTRE)
                #Calls function to color background according to type from pokemon_functions.py
                background_type(self.Type_box, str(types))
                #Adds type TextCtrl box to Row Sizer
                row_sizer_4.Add(self.Type_box, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
            
            #Moves counter to next iteration
            this_list_counter = this_list_counter + 1
        
        #Adds Row Sizers to Main Sizer according to number of types in list        
        self.main_sizer.Add(row_sizer_1, 0, wx.EXPAND)
        if type_number > 5:
            self.main_sizer.Add(row_sizer_2, 0, wx.EXPAND)
            
            if type_number > 10:
                self.main_sizer.Add(row_sizer_3, 0, wx.EXPAND)
                
                if type_number > 15:
                    self.main_sizer.Add(row_sizer_4, 0, wx.EXPAND)        
        
    # ====================================== # When save button is pressed
        
    #This function is called when the save button is pressed. It gets the data from the dictionary (self.this_Team) and saves it into a .team file
    def on_save_team(self, event):
        
        #Checks if the team dictionary is not assembled yet (calculate button wasn't pressed)
        if self.this_team == {}:
            #Displays a default error message dialog telling the user to click on calculate
            #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
            dlg = wx.MessageDialog(self, caption='Error!', message = 'The team has not been assembled yet! Please fill up all the fields with Pokemon names and click on calculate.', style= wx.OK | wx.ICON_ERROR)
            dlg.ShowModal()
            #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
            #Closes the message dialog box    
            dlg.Destroy()
        
        #when calculate was already clicked and the team assembled, the .team data can be saved
        else:            
            #If there is a .team entry with the same name in the database, the user can choose to overwrite it or not
            if self.this_team['Team_Name'] in Pokemon_Team_Builder_Frame.teams_available:
                    #Calls a default attention Message Dialog
                    #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
                    dlg_attention = wx.MessageDialog(self, caption='Attention!', message = 'A Team Entry with this name already exists! Would you like to overwrite it?', style= wx.OK | wx.CANCEL | wx.ICON_WARNING)
                    #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown
                    #If the user presses the attention dialog box’s OK button, the program can get the user’s path choice via the dialog’s .GetPath() function
                    if dlg_attention.ShowModal() == wx.ID_OK:
                        #Overwrites pokemon entry in the specified database
                        register_team(Pokemon_Team_Builder_Frame.data_path,self.this_team)
                        #Displays a default message to say that the Entry was Successfully Overwritten in the Database
                        #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
                        dlg = wx.MessageDialog(self, caption='Success!', message = 'Team entry was successfully overwritten in the database!', style= wx.OK | wx.ICON_INFORMATION)
                        dlg.ShowModal()
                        #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
                        #Returns True when the team was saved
                        self.EndModal(True)
                        #Closes the message dialog box    
                        dlg.Destroy()
                        #Closes the attention message dialog box    
                        dlg_attention.Destroy()
                        #Closes AddTeamDialog Box
                        self.Destroy()
                    #When the user doesn't want to overwrite data, the message box closes and the use retuns to the AddPokemon Dialog Box
                    else:
                        #Closes the attention message dialog box    
                        dlg_attention.Destroy()
            
            #If there is no .team entry in the database with this name, the program adds the entry straight away
            else:
                #Adds .team file to database
                register_team(Pokemon_Team_Builder_Frame.data_path,self.this_team)
                #Displays a default message to say that the Entry was Successfully added
                #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
                dlg = wx.MessageDialog(self, caption='Success!', message = 'Team entry was successfully added to the database!', style= wx.OK | wx.ICON_INFORMATION)
                dlg.ShowModal()
                #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
                #Closes the message dialog box    
                dlg.Destroy()
                #Returns True when the team was saved
                self.EndModal(True)
                #Closes AddTeamDialog Box
                self.Destroy()
                
    # ====================================== # When more button is pressed
        
    #This function is called when the more button is pressed. It opens the ViewTeamDialog_extended Box
    def on_more_view_team(self, event):
        
        #Passes info to the ViewTeamDialog_extended
        #Sets up ViewTeamDialog_extended
        dlg = ViewTeamDialog_extended(self.this_team, self.o_adv_list_times2, self.o_adv_list_times1, self.o_adv_list_times05, self.o_adv_list_times0, self.d_davt_list_times2, self.d_davt_list_times4, self.d_davt_list_times1, self.d_davt_list_times05, self.d_davt_list_times025, self.d_davt_list_times0)
        #Displays ViewTeamDialog_extended
        dlg.Show()
                
    # ====================================== # When close button is pressed
        
    #This function is called when the close button is pressed. It closes the PreviewTeamDialog Box
    def on_close_preview_team(self, event):

        #Closes PreviewTeamDialog Box
        self.Destroy()
                
    # ====================================== # When a pokemon subtle button is pressed
        
    #This function is called when a pokemon subtle button is pressed. 
    #It passes the name of the pokemon in the button to the ViewPokemonDialog
    def on_view_pokemon_team(self, event):
        
        #Gets the pokemon name from the button's label
        poke_name = event.GetEventObject().GetLabel()
        
        #Looks for that pokemon in the team dictionary
        for pokemon in self.this_team['Members']:
            #when the correct name is found
            if self.this_team['Members'][pokemon]['Name'][0] == poke_name:        
                #Passes pre-loaded pokemon sub-dictionary to the ViewPokemonDialog_from_dict Box
                #Sets up ViewPokemonDialog_from_dict Box
                dlg = ViewPokemonDialog_from_dict(self.this_team['Members'][pokemon], preview='no')
                #Displays ViewPokemonDialog_from_dict Box
                dlg.Show()
                
                
# ======================================================================================= # View Extended Pokemon Entry using a pre-loaded pokemon dictionary and type match lists
        
#Defines View Extended Team Entry as a Dialog Box using a pre-assembled team dictionary and type match lists
class ViewTeamDialog_extended(wx.Dialog):
            
    def __init__(self, team_dict, o_adv_list_times2, o_adv_list_times1, o_adv_list_times05, o_adv_list_times0, d_davt_list_times2, d_davt_list_times4, d_davt_list_times1, d_davt_list_times05, d_davt_list_times025, d_davt_list_times0):          
                
        super().__init__(parent=None, title="Extended Team Entry", style = wx.DEFAULT_DIALOG_STYLE | wx.RESIZE_BORDER, size = (1230,1000))   
        
        #Checks if the team_dict provided is blank
        if team_dict == '':
            #Displays a default error message dialog telling the user to select a team name from database
            #(this type of dialog box allows the user to choose simple options like yes, no, ok, cancel)
            dlg = wx.MessageDialog(self, caption='Error!', message = 'The team name is blank! Please select a valid team name from the provided database.', style= wx.OK | wx.ICON_ERROR)
            dlg.ShowModal()
            #ShowModal() causes the dialog to show modally, which means that the user won’t be able to interact with the main application while the dialog is shown           
            #Closes the message dialog box    
            dlg.Destroy()
            
            #Closes ViewTeamDialog_extended
            self.Destroy()
            
        #When a pokemon_dict was properly provided    
        else:        
            #Updates this_team local dictionary with provided team_dict
            this_team = team_dict
            
            #Updates local helpful lists
            self.o_adv_list_times2 = o_adv_list_times2
            self.o_adv_list_times1 = o_adv_list_times1
            self.o_adv_list_times05 = o_adv_list_times05
            self.o_adv_list_times0 = o_adv_list_times0
            
            #Updates local helpful lists
            self.d_davt_list_times2 = d_davt_list_times2
            self.d_davt_list_times4 = d_davt_list_times4
            self.d_davt_list_times1 = d_davt_list_times1
            self.d_davt_list_times05 = d_davt_list_times05
            self.d_davt_list_times025 = d_davt_list_times025
            self.d_davt_list_times0 = d_davt_list_times0

            #Defines Helpful Variables
            self.all_types = ['Normal', 'Fighting', 'Flying', 'Poison', 'Ground', 'Rock', 'Bug', 'Ghost', 'Steel', 'Fire', 'Water', 'Grass', 'Electric', 'Psychic', 'Ice', 'Dragon', 'Dark', 'Fairy']
            self.all_types_2 = ['', 'Normal', 'Fighting', 'Flying', 'Poison', 'Ground', 'Rock', 'Bug', 'Ghost', 'Steel', 'Fire', 'Water', 'Grass', 'Electric', 'Psychic', 'Ice', 'Dragon', 'Dark', 'Fairy']
            self.all_modes = ['Physical', 'Special', 'Status']

            #Defines Main Sizer
            self.main_sizer = wx.BoxSizer(wx.VERTICAL)

            #Defines font style for the statict text that labels team name
            font_moves = wx.Font(12, wx.DECORATIVE, wx.BOLD, wx.NORMAL)
            #Defines font style for the statict text for smaller labels
            font_moves_2 = wx.Font(10, wx.DECORATIVE, wx.BOLD, wx.NORMAL)

            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Sets up a Static Text for Team and adds it to the main sizer
            team_label = wx.StaticText(self, label="Team")
            team_label.SetFont(font_moves)
            self.main_sizer.Add(team_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            
            #Sets up a Static Text for Name label and adds it to the main sizer
            team_label = wx.StaticText(self, label="Name")
            team_label.SetFont(font_moves_2)
            self.main_sizer.Add(team_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Sets up Box for Name (READONLY), starting with former information
            self.Name_box = wx.TextCtrl(self, value=str(this_team['Team_Name'][0]), style=wx.TE_READONLY | wx.TE_CENTRE)
            empty_space_label_1 = wx.StaticText(self, label="")
            empty_space_label_2 = wx.StaticText(self, label="")
            self.add_widgets_3_boxes_horizontal(empty_space_label_1, self.Name_box , empty_space_label_2)

            # ============== # Battle
            
            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Sets up a Static Text for Moves and adds it to the main sizer
            pokemon_label = wx.StaticText(self, label="Battle")
            pokemon_label.SetFont(font_moves)
            self.main_sizer.Add(pokemon_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            
            #Defines Battle Sizer, its sub-sizers and adds its sub-sizers to it
            self.battle_sizer = wx.BoxSizer(wx.HORIZONTAL)
            
            #Offensive Sizer
            self.offensive_sizer = wx.BoxSizer(wx.VERTICAL)
            self.battle_sizer.Add(self.offensive_sizer, 1, wx.ALL, 5)
            
            #Sets up a Static Texts to be used as empty spaces and adds them to the battle sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.battle_sizer.Add(empty_space_label, 0, wx.EXPAND)
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.battle_sizer.Add(empty_space_label, 0, wx.EXPAND)
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.battle_sizer.Add(empty_space_label, 0, wx.EXPAND) 
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.battle_sizer.Add(empty_space_label, 0, wx.EXPAND) 
 
            #Defensive Sizer
            self.defensive_sizer = wx.BoxSizer(wx.VERTICAL)
            self.battle_sizer.Add(self.defensive_sizer, 1, wx.ALL, 5)
            
            #Adds Battle sizer to main sizer
            self.main_sizer.Add(self.battle_sizer, 0, wx.EXPAND)

            # ============== #  Offensive Advantage

            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.offensive_sizer.Add(empty_space_label, 0, wx.EXPAND)
            
            #Sets up a Static Text for Moves and adds it to the main sizer
            pokemon_label = wx.StaticText(self, label="Offense")
            pokemon_label.SetFont(font_moves)
            self.offensive_sizer.Add(pokemon_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            
            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.offensive_sizer.Add(empty_space_label, 0, wx.EXPAND)
            
            #Sets up a Static Text for labeling Offensive Advantage and adds it to the main sizer
            o_Advantage_label = wx.StaticText(self, label="(2x) Offensive Advantage ["+str(len(self.o_adv_list_times2))+" types]", style=wx.ALIGN_CENTRE_HORIZONTAL)
            o_Advantage_label.SetFont(font_moves)
            self.offensive_sizer.Add(o_Advantage_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            #Calls function to organize and display the types from the list self.o_adv_list_times2
            self.add_widgets_neat_type_display(self.o_adv_list_times2, sizer="Offensive")
            
            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.offensive_sizer.Add(empty_space_label, 0, wx.EXPAND)
            
            #Sets up a Static Text for labeling Offensive Advantage and adds it to the main sizer
            o_Advantage_label = wx.StaticText(self, label="(1x) Offensive Advantage ["+str(len(self.o_adv_list_times1))+" types]", style=wx.ALIGN_CENTRE_HORIZONTAL)
            o_Advantage_label.SetFont(font_moves)
            self.offensive_sizer.Add(o_Advantage_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            #Calls function to organize and display the types from the list self.o_adv_list_times1
            self.add_widgets_neat_type_display(self.o_adv_list_times1, sizer="Offensive")
            
            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.offensive_sizer.Add(empty_space_label, 0, wx.EXPAND)
            
            #Sets up a Static Text for labeling Offensive Advantage and adds it to the main sizer
            o_Advantage_label = wx.StaticText(self, label="(0.5x) Offensive Advantage ["+str(len(self.o_adv_list_times05))+" types]", style=wx.ALIGN_CENTRE_HORIZONTAL)
            o_Advantage_label.SetFont(font_moves)
            self.offensive_sizer.Add(o_Advantage_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            #Calls function to organize and display the types from the list self.o_adv_list_times05
            self.add_widgets_neat_type_display(self.o_adv_list_times05, sizer="Offensive")
            
            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.offensive_sizer.Add(empty_space_label, 0, wx.EXPAND)
            
            #Sets up a Static Text for labeling Offensive Advantage and adds it to the main sizer
            o_Advantage_label = wx.StaticText(self, label="(0x) Offensive Advantage ["+str(len(self.o_adv_list_times0))+" types]", style=wx.ALIGN_CENTRE_HORIZONTAL)
            o_Advantage_label.SetFont(font_moves)
            self.offensive_sizer.Add(o_Advantage_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            #Calls function to organize and display the types from the list self.o_adv_list_times0
            self.add_widgets_neat_type_display(self.o_adv_list_times0, sizer="Offensive")

            # ============== #  Defensive Disadvantage
            
            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.defensive_sizer.Add(empty_space_label, 0, wx.EXPAND)
            
            #Sets up a Static Text for Moves and adds it to the main sizer
            pokemon_label = wx.StaticText(self, label="Defense")
            pokemon_label.SetFont(font_moves)
            self.defensive_sizer.Add(pokemon_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            
            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.defensive_sizer.Add(empty_space_label, 0, wx.EXPAND)
            
            #Sets up a Static Text for labeling Defensive Disadvantage (x4) and adds it to the main sizer
            d_Disadvantage_times4_label = wx.StaticText(self, label="(4x) Defensive Disadvantage ["+str(len(self.d_davt_list_times4))+" types]", style=wx.ALIGN_CENTRE_HORIZONTAL)
            d_Disadvantage_times4_label.SetFont(font_moves)
            self.defensive_sizer.Add(d_Disadvantage_times4_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            #Calls function to organize and display the types from the list self.d_davt_list_times4
            self.add_widgets_neat_type_display(self.d_davt_list_times4, sizer="Defensive")    
        
            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.defensive_sizer.Add(empty_space_label, 0, wx.EXPAND)     
            
            #Sets up a Static Text for labeling Defensive Disadvantage (x2) and adds it to the main sizer
            d_Disadvantage_times2_label = wx.StaticText(self, label="(2x) Defensive Disadvantage ["+str(len(self.d_davt_list_times2))+" types]", style=wx.ALIGN_CENTRE_HORIZONTAL)
            d_Disadvantage_times2_label.SetFont(font_moves)
            self.defensive_sizer.Add(d_Disadvantage_times2_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            #Calls function to organize and display the types from the list self.d_davt_list_times2
            self.add_widgets_neat_type_display(self.d_davt_list_times2, sizer="Defensive")
            
            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.defensive_sizer.Add(empty_space_label, 0, wx.EXPAND)     
            
            #Sets up a Static Text for labeling Defensive Disadvantage (x1) and adds it to the main sizer
            d_Disadvantage_times1_label = wx.StaticText(self, label="(1x) Defensive Disadvantage ["+str(len(self.d_davt_list_times1))+" types]", style=wx.ALIGN_CENTRE_HORIZONTAL)
            d_Disadvantage_times1_label.SetFont(font_moves)
            self.defensive_sizer.Add(d_Disadvantage_times1_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            #Calls function to organize and display the types from the list self.d_davt_list_times1
            self.add_widgets_neat_type_display(self.d_davt_list_times1, sizer="Defensive")
            
            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.defensive_sizer.Add(empty_space_label, 0, wx.EXPAND)     
            
            #Sets up a Static Text for labeling Defensive Disadvantage (x0.5) and adds it to the main sizer
            d_Disadvantage_times05_label = wx.StaticText(self, label="(0.5x) Defensive Disadvantage ["+str(len(self.d_davt_list_times05))+" types]", style=wx.ALIGN_CENTRE_HORIZONTAL)
            d_Disadvantage_times05_label.SetFont(font_moves)
            self.defensive_sizer.Add(d_Disadvantage_times05_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            #Calls function to organize and display the types from the list self.d_davt_list_times05
            self.add_widgets_neat_type_display(self.d_davt_list_times05, sizer="Defensive")          

            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.defensive_sizer.Add(empty_space_label, 0, wx.EXPAND)     
            
            #Sets up a Static Text for labeling Defensive Disadvantage (x0.25) and adds it to the main sizer
            d_Disadvantage_times025_label = wx.StaticText(self, label="(0.25x) Defensive Disadvantage ["+str(len(self.d_davt_list_times025))+" types]", style=wx.ALIGN_CENTRE_HORIZONTAL)
            d_Disadvantage_times025_label.SetFont(font_moves)
            self.defensive_sizer.Add(d_Disadvantage_times025_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            #Calls function to organize and display the types from the list self.d_davt_list_times025
            self.add_widgets_neat_type_display(self.d_davt_list_times025, sizer="Defensive")
            
            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.defensive_sizer.Add(empty_space_label, 0, wx.EXPAND)
                                                      
            #Sets up a Static Text for labeling Defensive Disadvantage (x0) and adds it to the main sizer
            d_Disadvantage_times0_label = wx.StaticText(self, label="(0x) Defensive Disadvantage ["+str(len(self.d_davt_list_times0))+" types]", style=wx.ALIGN_CENTRE_HORIZONTAL)
            d_Disadvantage_times0_label.SetFont(font_moves)
            self.defensive_sizer.Add(d_Disadvantage_times0_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)
            #Calls function to organize and display the types from the list self.d_davt_list_times0
            self.add_widgets_neat_type_display(self.d_davt_list_times0, sizer="Defensive")
            
            # ============== # Bottom Buttons
            
            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Sets up a Static Text to be used as empty space and adds it to the main sizer
            empty_space_label = wx.StaticText(self, label="")
            empty_space_label.SetFont(font_moves)
            self.main_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_HORIZONTAL)

            #Defines Button Sizer
            btn_sizer = wx.BoxSizer()
            #Creates Close Button
            close_btn = wx.Button(self, label='Close')
            #Binds Button event to Close Button
            close_btn.Bind(wx.EVT_BUTTON, self.on_close_view_team)
            #Adds Close Button to Button Sizer
            btn_sizer.Add(close_btn, 0, wx.ALL, 5)
            #Adds Button Sizer to Main Sizer
            self.main_sizer.Add(btn_sizer, 0, wx.CENTER)

            #Sets Main Sizer for user to see
            self.SetSizer(self.main_sizer)
              
     # ====================================== # Organizing layout   
    
    #This function helps position each box (Text Control, Combo Box, Radio Box) to entry pokemon information
    def add_widgets_2_boxes_horizontal(self, box_1, box_2):
      
        #Defines Row Sizer for each "Line"
        row_sizer = wx.BoxSizer(wx.HORIZONTAL)
        #Adds box_1 to Row Sizer
        row_sizer.Add(box_1, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Sets up and adds empty space
        empty_space_label = wx.StaticText(self, label="")
        row_sizer.Add(empty_space_label, 0, wx.ALIGN_CENTRE_VERTICAL)
        #Adds box_2 to row sizer
        row_sizer.Add(box_2, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Adds Row Sizer to Main Sizer
        self.main_sizer.Add(row_sizer, 0, wx.EXPAND)
        
    #This function helps position each box (Text Control, Combo Box, Radio Box) to entry pokemon information
    def add_widgets_3_boxes_horizontal(self, box_1, box_2, box_3):
      
        #Defines Row Sizer for each "Line"
        row_sizer = wx.BoxSizer(wx.HORIZONTAL)
        #Adds box_1 to Row Sizer
        row_sizer.Add(box_1, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Adds box_2 to row sizer
        row_sizer.Add(box_2, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Adds box_3 to Row Sizer
        row_sizer.Add(box_3, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
        #Adds Row Sizer to Main Sizer
        self.main_sizer.Add(row_sizer, 0, wx.EXPAND)
        
    #This function helps position each box (Text Control, Combo Box, Radio Box) to entry pokemon information
    def add_widgets_neat_type_display(self, type_list, sizer):
        
        #Transfers list to a local variable
        self.this_list = type_list

        #Cheks how many types are in the list
        type_number = len(self.this_list)
        
        #Defines Row Sizers for each "Line" according to total number of types in list
        row_sizer_1 = wx.BoxSizer(wx.HORIZONTAL)
        if type_number > 5:
            row_sizer_2 = wx.BoxSizer(wx.HORIZONTAL)
            
            if type_number > 10:
                row_sizer_3 = wx.BoxSizer(wx.HORIZONTAL)
                
                if type_number > 15:
                    row_sizer_4 = wx.BoxSizer(wx.HORIZONTAL)
        
        
        #Defines/resets helpful counter
        this_list_counter = 0
        
        #Starts to go through the list
        for types in self.this_list:
            #Until the 5th type:
            if this_list_counter <5:
            
                #Sets up read only TextCtrl box for current type
                self.Type_box = wx.TextCtrl(self, value=str(types), style=wx.TE_READONLY | wx.TE_CENTRE)
                #Calls function to color background according to type from pokemon_functions.py
                background_type(self.Type_box, str(types))
                #Adds type TextCtrl box to Row Sizer
                row_sizer_1.Add(self.Type_box, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
                
            #From the 5th to the 10th type:
            if this_list_counter >= 5 and this_list_counter < 10:
                
                #Sets up read only TextCtrl box for current type
                self.Type_box = wx.TextCtrl(self, value=str(types), style=wx.TE_READONLY | wx.TE_CENTRE)
                #Calls function to color background according to type from pokemon_functions.py
                background_type(self.Type_box, str(types))
                #Adds type TextCtrl box to Row Sizer
                row_sizer_2.Add(self.Type_box, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
                
            #From the 10th to the 15th type:
            if this_list_counter >= 10 and this_list_counter < 15:
                
                #Sets up read only TextCtrl box for current type
                self.Type_box = wx.TextCtrl(self, value=str(types), style=wx.TE_READONLY | wx.TE_CENTRE)
                #Calls function to color background according to type from pokemon_functions.py
                background_type(self.Type_box, str(types))
                #Adds type TextCtrl box to Row Sizer
                row_sizer_3.Add(self.Type_box, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
                
            #From the 15th to the 18th type:
            if this_list_counter >= 15 and this_list_counter < 18:
                
                #Sets up read only TextCtrl box for current type
                self.Type_box = wx.TextCtrl(self, value=str(types), style=wx.TE_READONLY | wx.TE_CENTRE)
                #Calls function to color background according to type from pokemon_functions.py
                background_type(self.Type_box, str(types))
                #Adds type TextCtrl box to Row Sizer
                row_sizer_4.Add(self.Type_box, 1, wx.ALL | wx.ALIGN_CENTRE_VERTICAL, 5)
            
            #Moves counter to next iteration
            this_list_counter = this_list_counter + 1
        
        #Adds Row Sizers to Main Sizer according to number of types in list and the provided sizer
        if sizer == "Offensive":
            self.offensive_sizer.Add(row_sizer_1, 0, wx.EXPAND)
            if type_number > 5:
                self.offensive_sizer.Add(row_sizer_2, 0, wx.EXPAND)

                if type_number > 10:
                    self.offensive_sizer.Add(row_sizer_3, 0, wx.EXPAND)

                    if type_number > 15:
                        self.offensive_sizer.Add(row_sizer_4, 0, wx.EXPAND)
                        
        if sizer == "Defensive":
            self.defensive_sizer.Add(row_sizer_1, 0, wx.EXPAND)
            if type_number > 5:
                self.defensive_sizer.Add(row_sizer_2, 0, wx.EXPAND)

                if type_number > 10:
                    self.defensive_sizer.Add(row_sizer_3, 0, wx.EXPAND)

                    if type_number > 15:
                        self.defensive_sizer.Add(row_sizer_4, 0, wx.EXPAND)
        
        
        
    # ====================================== # When close button is pressed
        
    #This function is called when the close button is pressed. It closes the ViewPokemonDialog Box
    def on_close_view_team(self, event):

        #Closes ViewTeamDialog_Extended Box
        self.Destroy()
        


In [2]:
#Runs Program

if __name__ == '__main__':
    app = wx.App(False)
    frame = Pokemon_Team_Builder_Frame()
    app.MainLoop()

In [ ]:
#Check geodude family at some point. They are only showing the Kantonian form as 'Only Form'
